In [5]:
#!/usr/bin/env python3
"""
GenBR-lite + POMDP Belief + Multimodal Trajectory Library (MultiNash-PF-lite)
============================================================================

This script implements an end-to-end pipeline for:
1) A stealth robot planning to reach a goal in a gridworld
2) An adversarial sensor/guard whose behavior belongs to one of K discrete "modes"
3) Robot maintains a POMDP belief over the sensor state and sensor mode using a particle filter
4) Robot chooses actions via belief-space MCTS (GenBR-lite) with PUCT
5) A "multimodal trajectory library" is generated each step by sampling hypotheses + randomized-risk A*
   (plays the role of MultiNash-PF: produce multiple plausible "equilibrium-like" modes)
6) The library is used to form action priors for MCTS and for rollouts / value heuristics.

The code is designed to be:
- Fully runnable (pure Python, no external solvers)
- Heavily commented and extremely verbose (debug prints)
- Robust: input checks, safe defaults, bounded loops

Author: ChatGPT
"""

from __future__ import annotations

import math
import random
import heapq
from dataclasses import dataclass, field
from typing import (
    Dict, List, Tuple, Optional, Iterable, Callable, Any, Set
)
from collections import defaultdict, Counter


# =========================
# Utility + Debug Controls
# =========================

DEBUG = True  # Turn off to reduce prints

def dprint(*args, enabled=True):
    if enabled:
        print(*args)


def clamp(x: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, x))


def manhattan(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def seed_all(seed: int) -> None:
    """Seed Python RNG for reproducibility."""
    random.seed(seed)


# =========================
# Gridworld Environment
# =========================

Action = str  # 'U','D','L','R','S'(stay)

ACTIONS: List[Action] = ["U", "D", "L", "R", "S"]
ACTION_DELTAS: Dict[Action, Tuple[int, int]] = {
    "U": (-1, 0),
    "D": (1, 0),
    "L": (0, -1),
    "R": (0, 1),
    "S": (0, 0),
}


@dataclass
class GridWorld:
    """
    GridWorld with obstacles.

    Coordinates: (r, c) where r in [0..H-1], c in [0..W-1]
    """
    H: int
    W: int
    obstacles: Set[Tuple[int, int]] = field(default_factory=set)

    def in_bounds(self, p: Tuple[int, int]) -> bool:
        r, c = p
        return 0 <= r < self.H and 0 <= c < self.W

    def is_free(self, p: Tuple[int, int]) -> bool:
        return self.in_bounds(p) and (p not in self.obstacles)

    def step(self, p: Tuple[int, int], a: Action) -> Tuple[int, int]:
        """Deterministic movement; if hits obstacle/bounds, stays."""
        if a not in ACTION_DELTAS:
            raise ValueError(f"Unknown action: {a}")
        dr, dc = ACTION_DELTAS[a]
        np = (p[0] + dr, p[1] + dc)
        if self.is_free(np):
            return np
        return p

    def neighbors(self, p: Tuple[int, int]) -> List[Tuple[Tuple[int, int], Action]]:
        """Return feasible next states and actions."""
        out: List[Tuple[Tuple[int, int], Action]] = []
        for a in ACTIONS:
            np = self.step(p, a)
            out.append((np, a))
        return out

    def render(self,
               robot: Tuple[int, int],
               sensor: Tuple[int, int],
               goal: Tuple[int, int]) -> None:
        """ASCII render for debugging."""
        grid = [["." for _ in range(self.W)] for _ in range(self.H)]
        for (r, c) in self.obstacles:
            grid[r][c] = "#"
        rr, rc = robot
        sr, sc = sensor
        gr, gc = goal
        grid[gr][gc] = "G"
        grid[sr][sc] = "S"
        grid[rr][rc] = "R"
        for r in range(self.H):
            print("".join(grid[r]))


# =========================
# Sensor Models (Modes)
# =========================

@dataclass
class SensorModePolicy:
    """
    Base class for sensor behavior policies ("modes").
    The robot treats the mode index z as hidden.
    """
    name: str

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        raise NotImplementedError


@dataclass
class PatrolLoopPolicy(SensorModePolicy):
    """
    Patrols along a fixed loop of waypoints in order.
    """
    loop: List[Tuple[int, int]] = field(default_factory=list)

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        if not self.loop:
            return "S"
        # Target waypoint index depends on time; simple periodic
        idx = t % len(self.loop)
        target = self.loop[idx]
        return greedy_step_toward(world, sensor_pos, target)


@dataclass
class ChasePolicy(SensorModePolicy):
    """
    Greedily moves toward the robot (adversarial).
    """
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        return greedy_step_toward(world, sensor_pos, robot_pos)


@dataclass
class RandomWalkPolicy(SensorModePolicy):
    """
    Random feasible move.
    """
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        # Randomly choose among actions that actually move or stay; prefer feasible.
        candidates = []
        for a in ACTIONS:
            np = world.step(sensor_pos, a)
            candidates.append((a, np))
        a, _ = random.choice(candidates)
        return a


def greedy_step_toward(world: GridWorld, src: Tuple[int, int], dst: Tuple[int, int]) -> Action:
    """
    One-step greedy action to reduce Manhattan distance to dst.
    Ties broken randomly.
    """
    best_actions: List[Action] = []
    best_dist = 10**9
    for a in ACTIONS:
        np = world.step(src, a)
        d = manhattan(np, dst)
        if d < best_dist:
            best_dist = d
            best_actions = [a]
        elif d == best_dist:
            best_actions.append(a)
    return random.choice(best_actions) if best_actions else "S"


# =========================
# Detection / Observation Model
# =========================

@dataclass
class DetectionModel:
    """
    Simple probabilistic detection model:
    p_det depends on Manhattan distance between robot and sensor.

    Example:
        if dist <= d0 => high detection probability
        else => decays
    """
    d0: int = 2           # "close range"
    p_close: float = 0.9  # p_det at dist <= d0
    p_far: float = 0.05   # minimum p_det when very far
    decay: float = 0.3    # exponential decay factor beyond d0

    def p_detect(self, robot_pos: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        dist = manhattan(robot_pos, sensor_pos)
        if dist <= self.d0:
            return self.p_close
        # Exponential decay to p_far
        return self.p_far + (self.p_close - self.p_far) * math.exp(-self.decay * (dist - self.d0))


@dataclass
class ObservationModel:
    """
    Robot observes a noisy measurement of sensor position.
    We'll model:
        o = sensor_pos + noise, where noise is discrete with a simple likelihood.

    Likelihood:
        p(o | sensor_pos) ∝ exp(-||o - sensor_pos||_1 / sigma)
    """
    sigma: float = 1.2  # larger => less informative observations

    def sample_observation(self, sensor_pos: Tuple[int, int], world: GridWorld) -> Tuple[int, int]:
        # Sample noise by random L1 perturbation (bounded)
        # This is simple and robust; not a perfect Gaussian but fine for demo.
        max_jump = 2
        dr = random.randint(-max_jump, max_jump)
        dc = random.randint(-max_jump, max_jump)
        o = (sensor_pos[0] + dr, sensor_pos[1] + dc)
        # Clamp to bounds
        o = (clamp(o[0], 0, world.H - 1), clamp(o[1], 0, world.W - 1))
        return (int(o[0]), int(o[1]))

    def likelihood(self, obs: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        d = manhattan(obs, sensor_pos)
        # Avoid underflow with a floor
        return max(1e-12, math.exp(-d / max(1e-6, self.sigma)))


# =========================
# Particle Filter over Sensor State + Mode
# =========================

@dataclass
class Particle:
    """
    Represents hypothesis about sensor position and sensor mode z.
    """
    sensor_pos: Tuple[int, int]
    mode_idx: int
    weight: float = 1.0


@dataclass
class BeliefState:
    """
    Particle belief over (sensor_pos, mode).
    """
    particles: List[Particle]
    mode_names: List[str]

    def normalize(self) -> None:
        s = sum(p.weight for p in self.particles)
        if s <= 0:
            # Reset to uniform if degenerate
            w = 1.0 / max(1, len(self.particles))
            for p in self.particles:
                p.weight = w
            return
        for p in self.particles:
            p.weight /= s

    def mode_posterior(self) -> Dict[int, float]:
        post = defaultdict(float)
        for p in self.particles:
            post[p.mode_idx] += p.weight
        return dict(post)

    def summary(self, topk: int = 3) -> str:
        post = self.mode_posterior()
        items = sorted(post.items(), key=lambda kv: kv[1], reverse=True)[:topk]
        parts = []
        for idx, prob in items:
            name = self.mode_names[idx] if 0 <= idx < len(self.mode_names) else f"mode{idx}"
            parts.append(f"{name}:{prob:.3f}")
        return " | ".join(parts) if parts else "(empty)"

    def effective_sample_size(self) -> float:
        # ESS = 1 / sum w_i^2
        s = sum(p.weight ** 2 for p in self.particles)
        return 0.0 if s <= 0 else 1.0 / s


def systematic_resample(particles: List[Particle]) -> List[Particle]:
    """
    Systematic resampling for particle filters.
    Assumes weights are normalized.
    """
    n = len(particles)
    if n == 0:
        return []
    positions = [(random.random() + i) / n for i in range(n)]
    cumulative = []
    csum = 0.0
    for p in particles:
        csum += p.weight
        cumulative.append(csum)

    out: List[Particle] = []
    i = 0
    for pos in positions:
        while i < n - 1 and pos > cumulative[i]:
            i += 1
        chosen = particles[i]
        out.append(Particle(sensor_pos=chosen.sensor_pos, mode_idx=chosen.mode_idx, weight=1.0 / n))
    return out


def initialize_belief(world: GridWorld,
                      sensor_init_candidates: List[Tuple[int, int]],
                      mode_pior: List[float],
                      mode_names: List[str],
                      num_particles: int = 200) -> BeliefState:
    """
    Initialize belief by sampling positions from candidates and modes from prior.
    """
    if len(mode_pior) != len(mode_names):
        raise ValueError("mode_pior and mode_names must have same length.")
    if not sensor_init_candidates:
        raise ValueError("Need at least one sensor init candidate position.")
    if num_particles <= 0:
        raise ValueError("num_particles must be > 0")

    # Normalize prior
    s = sum(mode_pior)
    if s <= 0:
        mode_pior = [1.0 / len(mode_pior)] * len(mode_pior)
    else:
        mode_pior = [x / s for x in mode_pior]

    def sample_mode() -> int:
        r = random.random()
        c = 0.0
        for i, p in enumerate(mode_pior):
            c += p
            if r <= c:
                return i
        return len(mode_pior) - 1

    particles: List[Particle] = []
    for _ in range(num_particles):
        sp = random.choice(sensor_init_candidates)
        z = sample_mode()
        particles.append(Particle(sensor_pos=sp, mode_idx=z, weight=1.0 / num_particles))
    return BeliefState(particles=particles, mode_names=mode_names)


def belief_predict_update(
    belief: BeliefState,
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    obs_model: ObservationModel,
    obs: Tuple[int, int],
    t: int,
    motion_noise: float = 0.10,
    resample_threshold: float = 0.5,
) -> None:
    """
    Particle filter step:
    - Predict sensor motion using policy of each particle's mode
    - Add small random motion noise
    - Weight update using observation likelihood p(obs | sensor_pos)
    - Normalize and resample if ESS too small

    Prints:
    - before/after mode posterior
    - ESS
    """
    dprint("\n[Belief] === Particle filter predict/update ===")
    dprint(f"[Belief] Observation obs={obs}, time t={t}")
    dprint(f"[Belief] Prior mode posterior: {format_mode_posterior(belief)}")
    dprint(f"[Belief] Prior ESS: {belief.effective_sample_size():.2f}/{len(belief.particles)}")

    # Predict + weight update
    for p in belief.particles:
        # Policy-predicted action:
        mode_idx = p.mode_idx
        if mode_idx < 0 or mode_idx >= len(sensor_policies):
            # Robust fallback
            aS = "S"
        else:
            aS = sensor_policies[mode_idx].action(world, p.sensor_pos, robot_pos, t)

        predicted = world.step(p.sensor_pos, aS)

        # Motion noise: with small prob, take a random action
        if random.random() < motion_noise:
            a_noise = random.choice(ACTIONS)
            predicted = world.step(predicted, a_noise)

        p.sensor_pos = predicted

        # Weight update with observation likelihood
        p.weight *= obs_model.likelihood(obs, p.sensor_pos)

    belief.normalize()

    ess = belief.effective_sample_size()
    dprint(f"[Belief] Posterior mode posterior: {format_mode_posterior(belief)}")
    dprint(f"[Belief] Posterior ESS: {ess:.2f}/{len(belief.particles)}")

    # Resample if ESS is low
    if ess < resample_threshold * len(belief.particles):
        dprint(f"[Belief] ESS below threshold ({resample_threshold:.2f} * N). Resampling...")
        belief.particles = systematic_resample(belief.particles)
        dprint(f"[Belief] After resample mode posterior: {format_mode_posterior(belief)}")
    else:
        dprint("[Belief] No resampling needed.")


def format_mode_posterior(belief: BeliefState, topk: int = 5) -> str:
    post = belief.mode_posterior()
    items = sorted(post.items(), key=lambda kv: kv[1], reverse=True)[:topk]
    return " | ".join(f"{belief.mode_names[i]}:{p:.3f}" for i, p in items)


# =========================
# A* Search with Risk Costs
# =========================

@dataclass
class AStarResult:
    path: List[Tuple[int, int]]  # includes start and goal
    cost: float
    expanded: int


def astar_path(
    world: GridWorld,
    start: Tuple[int, int],
    goal: Tuple[int, int],
    step_cost: Callable[[Tuple[int, int]], float],
    heuristic: Callable[[Tuple[int, int]], float],
    max_expansions: int = 50_000,
) -> Optional[AStarResult]:
    """
    A* in gridworld:
      g(n) = cost so far
      f(n) = g(n) + h(n)

    step_cost(state) should be >= 0.

    Returns None if no path found within max_expansions.
    """
    if not world.is_free(start):
        return None
    if not world.is_free(goal):
        return None

    frontier: List[Tuple[float, float, Tuple[int, int]]] = []
    heapq.heappush(frontier, (heuristic(start), 0.0, start))

    came_from: Dict[Tuple[int, int], Tuple[int, int]] = {}
    gscore: Dict[Tuple[int, int], float] = {start: 0.0}

    expanded = 0
    visited: Set[Tuple[int, int]] = set()

    while frontier and expanded < max_expansions:
        f, g, cur = heapq.heappop(frontier)
        if cur in visited:
            continue
        visited.add(cur)
        expanded += 1

        if cur == goal:
            # Reconstruct
            path = [cur]
            while cur in came_from:
                cur = came_from[cur]
                path.append(cur)
            path.reverse()
            return AStarResult(path=path, cost=gscore[goal], expanded=expanded)

        for (nxt, a) in world.neighbors(cur):
            # allow staying; but step cost still applies
            if not world.is_free(nxt):
                continue
            ng = gscore[cur] + step_cost(nxt)
            if nxt not in gscore or ng < gscore[nxt]:
                gscore[nxt] = ng
                came_from[nxt] = cur
                nf = ng + heuristic(nxt)
                heapq.heappush(frontier, (nf, ng, nxt))

    return None


# =========================
# MultiNash-PF-lite: Multimodal Path Library
# =========================

@dataclass
class PathMode:
    """
    One "mode" = candidate robot path under a sampled belief hypothesis.
    """
    path: List[Tuple[int, int]]
    total_cost: float
    hypothesis_sensor_pos: Tuple[int, int]
    hypothesis_mode_idx: int
    debug_info: Dict[str, Any] = field(default_factory=dict)


def build_cost_map_from_hypothesis(
    world: GridWorld,
    detection: DetectionModel,
    sensor_pos: Tuple[int, int],
    base_step: float,
    w_detect: float,
    randomize: bool,
    rand_scale: float,
) -> Callable[[Tuple[int, int]], float]:
    """
    Returns a state cost function c(s) = base_step + w_detect * p_det(robot=s, sensor=hyp)
    plus optional random noise for multimodality.
    """
    def cost_fn(s: Tuple[int, int]) -> float:
        if not world.is_free(s):
            return float("inf")
        p_det = detection.p_detect(robot_pos=s, sensor_pos=sensor_pos)
        c = base_step + w_detect * p_det
        if randomize:
            # Noise to push A* into alternative corridors ("implicit PF-like" diversity)
            # Keep it positive and small
            c *= (1.0 + rand_scale * (random.random() - 0.5))
            c = max(1e-6, c)
        return c
    return cost_fn


def cluster_paths_by_signature(paths: List[List[Tuple[int, int]]], max_modes: int) -> List[int]:
    """
    Robust clustering:
    - Convert each path to a signature string of compressed directions.
    - Map signatures to cluster IDs until max_modes is reached.
    - Once max_modes is reached, assign new signatures to an existing cluster (hash/fallback),
      WITHOUT trying to store them in the dict (prevents unbounded growth).
    Returns: cluster id for each path.
    """
    sig_to_cluster: Dict[str, int] = {}
    assignments: List[int] = []

    def signature(path: List[Tuple[int, int]]) -> str:
        if len(path) < 2:
            return "EMPTY"
        dirs = []
        for i in range(1, len(path)):
            dr = path[i][0] - path[i-1][0]
            dc = path[i][1] - path[i-1][1]
            dirs.append((dr, dc))

        # compress consecutive repeats
        comp = []
        for d in dirs:
            if not comp or comp[-1] != d:
                comp.append(d)

        m = {(-1, 0): "U", (1, 0): "D", (0, -1): "L", (0, 1): "R", (0, 0): "S"}
        return "".join(m.get(d, "?") for d in comp)[:120]

    for p in paths:
        sig = signature(p)

        # Case 1: we've seen this signature before
        if sig in sig_to_cluster:
            cid = sig_to_cluster[sig]

        # Case 2: new signature and we still have room for new clusters
        elif len(sig_to_cluster) < max_modes:
            cid = len(sig_to_cluster)
            sig_to_cluster[sig] = cid

        # Case 3: new signature but we're at max_modes => assign to an existing cluster
        else:
            # Robust fallback: if somehow empty, assign cluster 0
            if not sig_to_cluster:
                cid = 0
            else:
                # Deterministic-ish hashing to reduce randomness across runs:
                # map signature to one of existing cluster IDs
                cid = hash(sig) % len(sig_to_cluster)

        assignments.append(cid)

    return assignments


def build_multimodal_path_library(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    num_hypotheses: int = 30,
    per_hypothesis_attempts: int = 2,
    base_step_cost: float = 1.0,
    w_detect: float = 8.0,
    rand_scale: float = 0.40,
    max_modes: int = 6,
) -> List[PathMode]:
    """
    This is the "MultiNash-PF-lite" replacement:
    - Sample several sensor hypotheses (sensor_pos, mode)
    - For each, run randomized-risk A* multiple times to get diverse paths
    - Cluster by path signature to get distinct modes
    - Keep the best representative per cluster

    Prints everything.
    """
    dprint("\n[Modes] === Building multimodal path library ===")
    dprint(f"[Modes] robot_pos={robot_pos}, goal={goal}")
    dprint(f"[Modes] num_hypotheses={num_hypotheses}, per_hypothesis_attempts={per_hypothesis_attempts}, max_modes={max_modes}")

    if not belief.particles:
        dprint("[Modes] WARNING: belief has no particles. Returning empty library.")
        return []

    # Sample particles proportional to weight
    weights = [p.weight for p in belief.particles]
    # Robust normalization
    s = sum(weights)
    if s <= 0:
        weights = [1.0 / len(weights)] * len(weights)
    else:
        weights = [w / s for w in weights]

    sampled_particles: List[Particle] = random.choices(belief.particles, weights=weights, k=num_hypotheses)

    candidates: List[PathMode] = []

    for hi, hp in enumerate(sampled_particles):
        hyp_sensor = hp.sensor_pos
        hyp_mode = hp.mode_idx

        dprint(f"\n[Modes] Hypothesis #{hi+1}/{num_hypotheses}: sensor_pos={hyp_sensor}, mode={belief.mode_names[hyp_mode]}")

        for attempt in range(per_hypothesis_attempts):
            # Randomized cost map encourages diverse solutions
            cost_fn = build_cost_map_from_hypothesis(
                world=world,
                detection=detection,
                sensor_pos=hyp_sensor,
                base_step=base_step_cost,
                w_detect=w_detect,
                randomize=True,
                rand_scale=rand_scale,
            )
            heur = lambda s: float(manhattan(s, goal))  # admissible baseline

            res = astar_path(world, robot_pos, goal, step_cost=cost_fn, heuristic=heur)
            if res is None:
                dprint(f"[Modes]  Attempt {attempt+1}: No path found.")
                continue

            dprint(f"[Modes]  Attempt {attempt+1}: Path found len={len(res.path)} cost={res.cost:.3f} expanded={res.expanded}")

            candidates.append(PathMode(
                path=res.path,
                total_cost=res.cost,
                hypothesis_sensor_pos=hyp_sensor,
                hypothesis_mode_idx=hyp_mode,
                debug_info={
                    "expanded": res.expanded,
                    "attempt": attempt + 1,
                }
            ))

    if not candidates:
        dprint("[Modes] No candidates found at all. Returning empty library.")
        return []

    # Cluster by signature and keep best per cluster
    paths_only = [c.path for c in candidates]
    cluster_ids = cluster_paths_by_signature(paths_only, max_modes=max_modes)

    best_by_cluster: Dict[int, PathMode] = {}
    for c, cid in zip(candidates, cluster_ids):
        if cid not in best_by_cluster or c.total_cost < best_by_cluster[cid].total_cost:
            best_by_cluster[cid] = c

    # Sort clusters by best cost
    library = sorted(best_by_cluster.values(), key=lambda pm: pm.total_cost)[:max_modes]

    dprint("\n[Modes] === Clustered modes summary ===")
    for i, pm in enumerate(library):
        dprint(f"[Modes] Mode {i}: cost={pm.total_cost:.3f}, hyp_sensor={pm.hypothesis_sensor_pos}, hyp_mode={belief.mode_names[pm.hypothesis_mode_idx]}, path_len={len(pm.path)}")

    return library


def first_action_from_path(world: GridWorld, start: Tuple[int, int], path: List[Tuple[int, int]]) -> Action:
    """Compute which primitive action moves start -> path[1]."""
    if not path or len(path) < 2:
        return "S"
    nxt = path[1]
    dr = nxt[0] - start[0]
    dc = nxt[1] - start[1]
    for a, (adr, adc) in ACTION_DELTAS.items():
        if (dr, dc) == (adr, adc):
            return a
    # Fallback
    return "S"


def build_action_prior_from_library(world: GridWorld,
                                   robot_pos: Tuple[int, int],
                                   library: List[PathMode]) -> Dict[Action, float]:
    """
    Convert path library into a prior p(a) used by PUCT:
    p(a) proportional to how many modes recommend that first step,
    weighted by inverse cost (cheaper modes contribute more).
    """
    pri = defaultdict(float)
    if not library:
        # Uniform fallback
        return {a: 1.0 / len(ACTIONS) for a in ACTIONS}

    for pm in library:
        a0 = first_action_from_path(world, robot_pos, pm.path)
        weight = 1.0 / max(1e-6, pm.total_cost)
        pri[a0] += weight

    # Normalize
    s = sum(pri.values())
    if s <= 0:
        return {a: 1.0 / len(ACTIONS) for a in ACTIONS}
    for a in ACTIONS:
        pri[a] /= s

    return dict(pri)


# =========================
# GenBR-lite: Belief-space MCTS with PUCT
# =========================

@dataclass
class MCTSNode:
    """Node stores edge-stats per action (classic AlphaZero-style)."""
    robot_pos: Tuple[int, int]
    depth: int

    # node visits
    N: int = 0

    # edge stats
    N_a: Dict[Action, int] = field(default_factory=lambda: {a: 0 for a in ACTIONS})
    W_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})
    Q_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})

    # prior over actions at this node
    P_a: Dict[Action, float] = field(default_factory=lambda: {a: 1.0/len(ACTIONS) for a in ACTIONS})

    # children
    children: Dict[Action, "MCTSNode"] = field(default_factory=dict)

    expanded: bool = False


def smooth_prior(prior: Dict[Action, float], eps: float = 0.10) -> Dict[Action, float]:
    """Epsilon-smooth and renormalize prior to avoid degenerate priors."""
    p = {a: float(prior.get(a, 0.0)) for a in ACTIONS}
    s = sum(p.values())
    if s <= 0:
        return {a: 1.0/len(ACTIONS) for a in ACTIONS}
    p = {a: v/s for a, v in p.items()}

    u = 1.0/len(ACTIONS)
    p = {a: (1-eps)*p[a] + eps*u for a in ACTIONS}

    s2 = sum(p.values())
    return {a: v/s2 for a, v in p.items()}


def select_action_puct(node: MCTSNode, c_puct: float, debug: bool = False) -> Action:
    """PUCT: argmax_a Q(s,a) + c*P(s,a)*sqrt(N(s)) / (1+N(s,a))."""
    sqrtN = math.sqrt(max(1, node.N))

    best_a = None
    best_score = -1e18
    for a in ACTIONS:
        q = node.Q_a[a]
        u = c_puct * node.P_a[a] * sqrtN / (1.0 + node.N_a[a])
        score = q + u

        # tiny noise for tie-breaking stability
        score += random.uniform(-1e-9, 1e-9)

        if score > best_score:
            best_score = score
            best_a = a

    assert best_a is not None
    return best_a


def backprop_path(path: List[Tuple[MCTSNode, Action]], value: float) -> None:
    """Backprop value through all visited edges."""
    for node, a in reversed(path):
        node.N += 1
        node.N_a[a] += 1
        node.W_a[a] += value
        node.Q_a[a] = node.W_a[a] / node.N_a[a]


def choose_final_action(root: MCTSNode) -> Action:
    """Pick action by max N; break ties by Q then prior then random."""
    # 1) max visits
    maxN = max(root.N_a.values())
    cand = [a for a in ACTIONS if root.N_a[a] == maxN]
    if len(cand) == 1:
        return cand[0]

    # 2) max Q
    maxQ = max(root.Q_a[a] for a in cand)
    cand2 = [a for a in cand if abs(root.Q_a[a] - maxQ) < 1e-12]
    if len(cand2) == 1:
        return cand2[0]

    # 3) max prior
    maxP = max(root.P_a[a] for a in cand2)
    cand3 = [a for a in cand2 if abs(root.P_a[a] - maxP) < 1e-12]
    return random.choice(cand3)





















def rollout_value(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    max_steps: int,
    w_step: float,
    w_detect: float,
) -> float:
    """
    Simple rollout heuristic:
    - Greedy toward goal for max_steps
    - Value is negative cost (MCTS maximizes)
    """
    total_cost = 0.0
    cur = robot_pos
    for _ in range(max_steps):
        if cur == goal:
            break
        # Greedy move toward goal
        a = greedy_step_toward(world, cur, goal)
        cur = world.step(cur, a)
        total_cost += w_step
        total_cost += w_detect * detection.p_detect(cur, sensor_pos)
    # Terminal bonus
    if cur == goal:
        total_cost -= 20.0  # reaching goal is good -> lower cost
    return -total_cost


def simulate_sensor_step(
    world: GridWorld,
    sensor_pos: Tuple[int, int],
    robot_pos: Tuple[int, int],
    t: int,
    mode_idx: int,
    policies: List[SensorModePolicy],
) -> Tuple[int, int]:
    """Apply the sensor mode policy for one step (deterministic)."""
    if mode_idx < 0 or mode_idx >= len(policies):
        return sensor_pos
    aS = policies[mode_idx].action(world, sensor_pos, robot_pos, t)
    return world.step(sensor_pos, aS)


def mcts_search_action(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    detection: DetectionModel,
    action_prior: Dict[Action, float],
    t: int,
    num_sims: int = 200,
    max_depth: int = 10,
    c_puct: float = 1.2,
    w_step: float = 1.0,
    w_detect: float = 6.0,
) -> Action:
    """
    Belief-space MCTS with root sampling:
      For each simulation:
        - sample (sensor_pos, mode) from belief particles
        - traverse tree selecting actions via PUCT
        - simulate robot + sensor forward
        - evaluate leaf with rollout heuristic
        - backup value

    Prints:
      - priors
      - root visit counts + Q values
    """
    dprint("\n[MCTS] === Starting MCTS Search ===")
    dprint(f"[MCTS] robot_pos={robot_pos}, goal={goal}, t={t}, num_sims={num_sims}, max_depth={max_depth}")
    dprint(f"[MCTS] Action prior p(a): " + " | ".join(f"{a}:{action_prior.get(a,0):.3f}" for a in ACTIONS))

    root = MCTSNode(robot_pos=robot_pos, depth=0)

    # Prepare sampling distribution over particles
    if not belief.particles:
        # fallback uniform dummy belief
        dummy = Particle(sensor_pos=robot_pos, mode_idx=0, weight=1.0)
        particles = [dummy]
        weights = [1.0]
    else:
        particles = belief.particles
        weights = [p.weight for p in particles]
        s = sum(weights)
        if s <= 0:
            weights = [1.0 / len(weights)] * len(weights)
        else:
            weights = [w / s for w in weights]

    for sim in range(num_sims):
        # Root sampling (belief)
        hp = random.choices(particles, weights=weights, k=1)[0]
        sim_sensor_pos = hp.sensor_pos
        sim_mode = hp.mode_idx

        # Copy forward simulation state
        cur_node = root
        cur_robot = robot_pos
        cur_sensor = sim_sensor_pos

        # Simulate a trajectory for up to max_depth
        total_value = 0.0
        discount = 1.0
        gamma = 0.95

        for depth in range(max_depth):
            if cur_robot == goal:
                # big reward for reaching goal
                total_value += discount * 50.0
                break

            # Select action via PUCT
            a = select_action_puct(cur_node, action_prior, c_puct)

            nxt_robot = world.step(cur_robot, a)

            # Sensor evolves too (under sampled mode)
            nxt_sensor = simulate_sensor_step(world, cur_sensor, nxt_robot, t + depth, sim_mode, sensor_policies)

            # Immediate reward (negative cost)
            step_cost = w_step + w_detect * detection.p_detect(nxt_robot, nxt_sensor)
            reward = -step_cost
            total_value += discount * reward
            discount *= gamma

            # Expand child
            if a not in cur_node.children:
                cur_node.children[a] = MCTSNode(robot_pos=nxt_robot, depth=cur_node.depth + 1)

                # Evaluate leaf with rollout heuristic and stop expansion
                leaf_v = rollout_value(
                    world=world,
                    robot_pos=nxt_robot,
                    sensor_pos=nxt_sensor,
                    goal=goal,
                    detection=detection,
                    max_steps=max_depth - depth - 1,
                    w_step=w_step,
                    w_detect=w_detect,
                )
                total_value += discount * leaf_v
                # Backup along the path (we need to store path nodes/actions)
                backup_path(root, robot_pos, goal, action_prior, cur_node, a, total_value)
                break

            # Continue down the tree
            # Backup incrementally: we do a simpler backup at end; store transitions in a list for robust backup
            cur_robot = nxt_robot
            cur_sensor = nxt_sensor
            cur_node = cur_node.children[a]
        else:
            # Depth limit reached; backup using total_value
            # Use last chosen action if we had one; but for robustness we do a root-only backup
            # (We still want stable results even if loop ended without expansion.)
            root.N += 1

    # Print root statistics
    dprint("\n[MCTS] === Root statistics ===")
    for a in ACTIONS:
        n = root.N_a.get(a, 0)
        q = root.Q_a.get(a, 0.0)
        dprint(f"[MCTS] a={a} | N={n:4d} | Q={q: .3f} | prior={action_prior.get(a,0):.3f}")

    # Choose action by max visits
    best_a = max(ACTIONS, key=lambda a: root.N_a.get(a, 0))
    dprint(f"[MCTS] Selected action = {best_a} (max visit count)")
    return best_a


# def select_action_puct(node: MCTSNode, prior: Dict[Action, float], c_puct: float) -> Action:
    """
    PUCT selection:
      argmax_a [ Q(s,a) + c * p(a) * sqrt(N(s)) / (1 + N(s,a)) ]

    Notation:
      - node.N is total visits of the node
      - node.Q_a[a] is mean value estimate for action a
      - node.N_a[a] is action visit count
      - prior[a] is action prior probability
    """
    if node.N <= 0:
        node.N = 1  # prevent sqrt(0) issues

    best_a = "S"
    best_score = -float("inf")
    sqrtN = math.sqrt(node.N)

    for a in ACTIONS:
        q = node.Q_a.get(a, 0.0)
        na = node.N_a.get(a, 0)
        pa = prior.get(a, 1.0 / len(ACTIONS))
        u = c_puct * pa * sqrtN / (1.0 + na)
        score = q + u
        if score > best_score:
            best_score = score
            best_a = a

    return best_a


def backup_path(
    root: MCTSNode,
    root_robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    prior: Dict[Action, float],
    leaf_parent: MCTSNode,
    leaf_action: Action,
    total_value: float,
) -> None:
    """
    For simplicity in this demo, we back up only into leaf_parent and root
    with a lightweight robust update.

    In a production implementation you'd record the full path of (node, action)
    and update each entry. Here we keep it stable and readable.
    """
    # Update leaf_parent stats
    leaf_parent.N += 1
    leaf_parent.N_a[leaf_action] += 1
    na = leaf_parent.N_a[leaf_action]
    old_q = leaf_parent.Q_a.get(leaf_action, 0.0)
    leaf_parent.Q_a[leaf_action] = old_q + (total_value - old_q) / na

    # Update root stats as well (so action counts accumulate)
    root.N += 1
    # We don't know which root action led here if leaf_parent isn't root;
    # but in this demo we expand from root quickly, so leaf_parent is often root.
    # If leaf_parent is root, this is correct.
    if leaf_parent is root:
        root.N_a[leaf_action] += 1
        na2 = root.N_a[leaf_action]
        old_q2 = root.Q_a.get(leaf_action, 0.0)
        root.Q_a[leaf_action] = old_q2 + (total_value - old_q2) / na2



def mcts_search_action_fixed(
    world,
    belief,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    sensor_policies,
    detection,
    action_prior: Dict[Action, float],
    t: int,
    num_sims: int = 250,
    max_depth: int = 10,
    c_puct: float = 1.4,
    w_step: float = 1.0,
    w_detect: float = 7.0,
    gamma: float = 0.95,
    debug: bool = True,
) -> Action:

    dprint("\n[MCTS-FIX] === Starting fixed MCTS ===", enabled=debug)

    # Smooth the prior so it is never degenerate
    prior = smooth_prior(action_prior, eps=0.10)

    root = MCTSNode(robot_pos=robot_pos, depth=0)
    root.P_a = prior
    root.expanded = True

    # Prepare particle sampling
    particles = belief.particles if getattr(belief, "particles", None) else []
    if not particles:
        # fallback single hypothesis
        class Dummy:
            sensor_pos = robot_pos
            mode_idx = 0
            weight = 1.0
        particles = [Dummy()]

    weights = [float(getattr(p, "weight", 1.0)) for p in particles]
    s = sum(weights)
    if s <= 0:
        weights = [1.0/len(weights)]*len(weights)
    else:
        weights = [w/s for w in weights]

    for sim in range(num_sims):
        hp = random.choices(particles, weights=weights, k=1)[0]
        sim_sensor = hp.sensor_pos
        sim_mode = hp.mode_idx

        node = root
        cur_robot = robot_pos
        cur_sensor = sim_sensor

        path: List[Tuple[MCTSNode, Action]] = []

        total_return = 0.0
        disc = 1.0

        for depth in range(max_depth):
            if cur_robot == goal:
                total_return += disc * 50.0
                break

            a = select_action_puct(node, c_puct=c_puct)
            path.append((node, a))

            nxt_robot = world.step(cur_robot, a)
            nxt_sensor = simulate_sensor_step(world, cur_sensor, nxt_robot, t + depth, sim_mode, sensor_policies)

            step_cost = w_step + w_detect * detection.p_detect(nxt_robot, nxt_sensor)
            reward = -step_cost
            total_return += disc * reward
            disc *= gamma

            # Expand child if needed
            if a not in node.children:
                child = MCTSNode(robot_pos=nxt_robot, depth=node.depth + 1)

                # IMPORTANT: give child the SAME prior (or you can compute state-dependent priors)
                child.P_a = prior
                child.expanded = True

                node.children[a] = child

                # Leaf evaluation (rollout)
                leaf_v = rollout_value(
                    world=world,
                    robot_pos=nxt_robot,
                    sensor_pos=nxt_sensor,
                    goal=goal,
                    detection=detection,
                    max_steps=max_depth - depth - 1,
                    w_step=w_step,
                    w_detect=w_detect,
                )
                total_return += disc * leaf_v
                break

            # Continue
            node = node.children[a]
            cur_robot, cur_sensor = nxt_robot, nxt_sensor

        # Backprop along full path
        backprop_path(path, total_return)

        if debug and (sim < 3 or (sim+1) % 50 == 0):
            dprint(f"[MCTS-FIX][DEBUG] sim={sim+1} root.N={root.N}")

    # Sanity check: root visits should be close to num_sims
    total_edge_visits = sum(root.N_a.values())
    dprint(f"[MCTS-FIX] root.N={root.N} | sum(root.N_a)={total_edge_visits} | expected~{num_sims}", enabled=debug)

    dprint("[MCTS-FIX] Root action stats:", enabled=debug)
    for a in ACTIONS:
        dprint(f"  a={a} N={root.N_a[a]:4d} Q={root.Q_a[a]:+8.3f} P={root.P_a[a]:.3f}", enabled=debug)

    chosen = choose_final_action(root)
    dprint(f"[MCTS-FIX] Chosen action = {chosen}", enabled=debug)
    return chosen

# =========================
# Main Simulation Loop
# =========================

@dataclass
class Config:
    seed: int = 7
    H: int = 12
    W: int = 18
    T: int = 40  # max time steps
    num_particles: int = 250

    # Belief filter
    motion_noise: float = 0.15
    resample_threshold: float = 0.55

    # Mode library
    num_hypotheses: int = 25
    per_hypothesis_attempts: int = 2
    max_modes: int = 6
    w_detect_astar: float = 10.0
    rand_scale: float = 0.45

    # MCTS
    num_sims: int = 250
    max_depth: int = 10
    c_puct: float = 1.4
    w_step_mcts: float = 1.0
    w_detect_mcts: float = 7.0


def make_demo_world(cfg: Config) -> Tuple[GridWorld, Tuple[int, int], Tuple[int, int], Tuple[int, int]]:
    """
    Create a world with a few obstacles and default start/goal positions.
    """
    obs = set()
    # Build some walls
    for c in range(3, 15):
        obs.add((5, c))
    for r in range(1, 9):
        obs.add((r, 9))
    # Add a gap in wall
    obs.discard((5, 7))
    obs.discard((6, 9))

    world = GridWorld(H=cfg.H, W=cfg.W, obstacles=obs)
    robot_start = (10, 2)
    goal = (1, 16)
    sensor_start = (2, 2)
    return world, robot_start, sensor_start, goal


def main() -> None:
    cfg = Config()
    seed_all(cfg.seed)

    world, robot_pos, sensor_pos, goal = make_demo_world(cfg)

    # Define sensor modes (policies)
    sensor_modes: List[SensorModePolicy] = [
        PatrolLoopPolicy(name="patrol_left", loop=[(2, 2), (2, 6), (4, 6), (4, 2)]),
        PatrolLoopPolicy(name="patrol_mid", loop=[(2, 10), (2, 13), (4, 13), (4, 10)]),
        ChasePolicy(name="chase_robot"),
        RandomWalkPolicy(name="random_walk"),
    ]
    mode_names = [m.name for m in sensor_modes]

    # Choose a true sensor mode (hidden from robot)
    true_mode_idx = random.randrange(len(sensor_modes))
    dprint("\n[INIT] True sensor mode =", mode_names[true_mode_idx])
    dprint("[INIT] Robot start =", robot_pos, "Sensor start =", sensor_pos, "Goal =", goal)

    detection = DetectionModel()
    obs_model = ObservationModel(sigma=1.3)

    # Belief initialization: assume sensor starts somewhere near top-left / top-mid
    sensor_init_candidates = [(2, 2), (2, 10), (3, 3), (3, 9), (1, 1)]
    mode_prior = [1.0] * len(sensor_modes)  # uniform
    belief = initialize_belief(world, sensor_init_candidates, mode_prior, mode_names, cfg.num_particles)

    dprint("\n[INIT] Initial belief mode posterior:", format_mode_posterior(belief))
    dprint("[INIT] Initial ESS:", belief.effective_sample_size())

    # Simulation
    for t in range(cfg.T):
        dprint("\n" + "=" * 80)
        dprint(f"[TIME] t={t}")
        dprint("=" * 80)

        # Render world
        dprint("\n[WORLD] Current grid:")
        world.render(robot=robot_pos, sensor=sensor_pos, goal=goal)

        # Generate robot's observation of sensor (noisy)
        obs = obs_model.sample_observation(sensor_pos, world)
        dprint(f"\n[OBS] Robot observes noisy sensor position obs={obs} (true sensor_pos={sensor_pos})")

        # Update belief using particle filter
        belief_predict_update(
            belief=belief,
            world=world,
            robot_pos=robot_pos,
            sensor_policies=sensor_modes,
            obs_model=obs_model,
            obs=obs,
            t=t,
            motion_noise=cfg.motion_noise,
            resample_threshold=cfg.resample_threshold,
        )
        dprint(f"[Belief] Mode posterior top: {belief.summary()}")

        # Build multimodal path library (MultiNash-PF-lite)
        library = build_multimodal_path_library(
            world=world,
            belief=belief,
            robot_pos=robot_pos,
            goal=goal,
            detection=detection,
            num_hypotheses=cfg.num_hypotheses,
            per_hypothesis_attempts=cfg.per_hypothesis_attempts,
            w_detect=cfg.w_detect_astar,
            rand_scale=cfg.rand_scale,
            max_modes=cfg.max_modes,
        )

        # Convert library into an MCTS prior
        action_prior = build_action_prior_from_library(world, robot_pos, library)
        dprint("\n[Prior] Derived action prior from modes:", " | ".join(f"{a}:{action_prior.get(a,0):.3f}" for a in ACTIONS))

        # MCTS decide robot action
        aR = mcts_search_action_fixed(
            world=world,
            belief=belief,
            robot_pos=robot_pos,
            goal=goal,
            sensor_policies=sensor_modes,
            detection=detection,
            action_prior=action_prior,
            t=t,
            num_sims=cfg.num_sims,
            max_depth=cfg.max_depth,
            c_puct=cfg.c_puct,
            w_step=cfg.w_step_mcts,
            w_detect=cfg.w_detect_mcts,
        )
        dprint(f"\n[ACT] Robot takes action aR={aR}")

        # Step robot
        new_robot = world.step(robot_pos, aR)

        # Step sensor using true mode
        aS_true = sensor_modes[true_mode_idx].action(world, sensor_pos, new_robot, t)
        new_sensor = world.step(sensor_pos, aS_true)

        dprint(f"[ACT] True sensor action aS={aS_true}, moves {sensor_pos}->{new_sensor}")
        dprint(f"[STEP] Robot moves {robot_pos}->{new_robot}")

        robot_pos, sensor_pos = new_robot, new_sensor

        # Detection event
        p_det = detection.p_detect(robot_pos, sensor_pos)
        detected = (random.random() < p_det)
        dprint(f"[DETECT] p_det={p_det:.3f} => detected={detected}")

        # Terminal checks
        if robot_pos == goal:
            dprint("\n[TERMINAL] Robot reached the goal! ✅")
            break
        if detected and manhattan(robot_pos, sensor_pos) <= detection.d0:
            dprint("\n[TERMINAL] Robot was detected and caught (close-range)! ❌")
            break

    dprint("\n[DONE] Simulation finished.")


if __name__ == "__main__":
    main()



[INIT] True sensor mode = chase_robot
[INIT] Robot start = (10, 2) Sensor start = (2, 2) Goal = (1, 16)

[INIT] Initial belief mode posterior: patrol_left:0.268 | random_walk:0.260 | patrol_mid:0.248 | chase_robot:0.224
[INIT] Initial ESS: 250.0

[TIME] t=0

[WORLD] Current grid:
..................
.........#......G.
..S......#........
.........#........
.........#........
...####.#######...
..................
.........#........
.........#........
..................
..R...............
..................

[OBS] Robot observes noisy sensor position obs=(2, 3) (true sensor_pos=(2, 2))

[Belief] === Particle filter predict/update ===
[Belief] Observation obs=(2, 3), time t=0
[Belief] Prior mode posterior: patrol_left:0.268 | random_walk:0.260 | patrol_mid:0.248 | chase_robot:0.224
[Belief] Prior ESS: 250.00/250
[Belief] Posterior mode posterior: patrol_left:0.352 | patrol_mid:0.306 | random_walk:0.185 | chase_robot:0.157
[Belief] Posterior ESS: 91.58/250
[Belief] ESS below threshold (0.55

TypeError: main() takes 0 positional arguments but 1 was given

In [10]:
#!/usr/bin/env python3
"""GenBR-lite + PF Belief + Multimodal Path Library (debug-robust)

What changed vs the earlier version:
- Much quieter logging by default: prints only on early timesteps + periodic summaries.
- Stronger invariants + sanity checks (probability normalization, ESS bounds, MCTS visit counts).
- Removed broken/duplicate MCTS code paths; kept a single robust MCTS implementation.
- Mode-library builder prints only a compact summary unless you enable detail.

Run:
  python3 this_file.py

Toggle logging in Config.debug.
"""

from __future__ import annotations

import math
import random
import heapq
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Iterable, Callable, Any, Set
from collections import defaultdict
import hashlib


# =========================
# Debug / Logging utilities
# =========================

@dataclass
class DebugConfig:
    enabled: bool = True
    level: int = 1
    # 0 = silent
    # 1 = periodic summaries
    # 2 = detailed (still gated)

    # Print detail for the first few timesteps
    detail_until_t: int = 2

    # Periodic summary controls
    summary_every_t: int = 5
    render_every_t: int = 10

    # Stage-specific periodic prints
    pf_every_t: int = 5
    modes_every_t: int = 5
    mcts_every_t: int = 5

    # Within MCTS
    mcts_progress_every_sims: int = 50

    # Safety: cap printing of mode hypotheses even at detail level
    max_hyp_print: int = 3


class Logger:
    def __init__(self, cfg: DebugConfig):
        self.cfg = cfg

    def _should_print(self, t: Optional[int], every: Optional[int], lvl: int) -> bool:
        if not self.cfg.enabled:
            return False
        if lvl > self.cfg.level:
            return False
        if t is None:
            return True
        if t <= self.cfg.detail_until_t:
            return True
        if every is None:
            return False
        return (every > 0) and (t % every == 0)

    def log(self, msg: str, *, t: Optional[int] = None, every: Optional[int] = None, lvl: int = 1, force: bool = False) -> None:
        if force or self._should_print(t, every, lvl):
            print(msg)


def soft_assert(cond: bool, msg: str, *, fatal: bool = False, log: Optional[Logger] = None, t: Optional[int] = None) -> None:
    if cond:
        return
    if log is not None:
        log.log(f"[WARN] {msg}", t=t, every=None, lvl=1, force=True)
    if fatal:
        raise AssertionError(msg)


def clamp(x: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, x))


def manhattan(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def seed_all(seed: int) -> None:
    random.seed(seed)


def stable_bucket(sig: str, m: int) -> int:
    """Stable mapping sig -> [0..m-1] independent of Python's randomized hash."""
    if m <= 0:
        return 0
    h = hashlib.md5(sig.encode("utf-8")).hexdigest()
    return int(h[:8], 16) % m


def normalize_probs_dict(d: Dict[Any, float], keys: Iterable[Any], *, eps: float = 0.0) -> Dict[Any, float]:
    """Return a normalized distribution over `keys` from possibly-missing/negative inputs.

    - Clips negatives to 0
    - Adds optional epsilon smoothing to avoid degeneracy
    - Falls back to uniform if total mass is 0
    """
    klist = list(keys)
    out = {k: max(0.0, float(d.get(k, 0.0))) for k in klist}
    s = sum(out.values())

    if s <= 0.0:
        # uniform
        u = 1.0 / max(1, len(klist))
        return {k: u for k in klist}

    out = {k: v / s for k, v in out.items()}

    if eps > 0.0:
        u = 1.0 / max(1, len(klist))
        out = {k: (1.0 - eps) * out[k] + eps * u for k in klist}
        s2 = sum(out.values())
        out = {k: v / s2 for k, v in out.items()}

    return out


def topk_str(d: Dict[Any, float], k: int = 3, fmt: str = "{k}:{v:.3f}") -> str:
    items = sorted(d.items(), key=lambda kv: kv[1], reverse=True)[:k]
    return " | ".join(fmt.format(k=kk, v=vv) for kk, vv in items)


# =========================
# Gridworld Environment
# =========================

Action = str  # 'U','D','L','R','S'

ACTIONS: List[Action] = ["U", "D", "L", "R", "S"]
ACTION_DELTAS: Dict[Action, Tuple[int, int]] = {
    "U": (-1, 0),
    "D": (1, 0),
    "L": (0, -1),
    "R": (0, 1),
    "S": (0, 0),
}


@dataclass
class GridWorld:
    H: int
    W: int
    obstacles: Set[Tuple[int, int]] = field(default_factory=set)

    def in_bounds(self, p: Tuple[int, int]) -> bool:
        r, c = p
        return 0 <= r < self.H and 0 <= c < self.W

    def is_free(self, p: Tuple[int, int]) -> bool:
        return self.in_bounds(p) and (p not in self.obstacles)

    def step(self, p: Tuple[int, int], a: Action) -> Tuple[int, int]:
        if a not in ACTION_DELTAS:
            raise ValueError(f"Unknown action: {a}")
        dr, dc = ACTION_DELTAS[a]
        np = (p[0] + dr, p[1] + dc)
        return np if self.is_free(np) else p

    def neighbors(self, p: Tuple[int, int]) -> List[Tuple[Tuple[int, int], Action]]:
        return [(self.step(p, a), a) for a in ACTIONS]

    def render(self, robot: Tuple[int, int], sensor: Tuple[int, int], goal: Tuple[int, int]) -> None:
        grid = [["." for _ in range(self.W)] for _ in range(self.H)]
        for (r, c) in self.obstacles:
            grid[r][c] = "#"
        rr, rc = robot
        sr, sc = sensor
        gr, gc = goal
        grid[gr][gc] = "G"
        grid[sr][sc] = "S"
        grid[rr][rc] = "R"
        for r in range(self.H):
            print("".join(grid[r]))


# =========================
# Sensor Models (Modes)
# =========================

@dataclass
class SensorModePolicy:
    name: str

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        raise NotImplementedError


def greedy_step_toward(world: GridWorld, src: Tuple[int, int], dst: Tuple[int, int]) -> Action:
    best_actions: List[Action] = []
    best_dist = 10**9
    for a in ACTIONS:
        np = world.step(src, a)
        d = manhattan(np, dst)
        if d < best_dist:
            best_dist = d
            best_actions = [a]
        elif d == best_dist:
            best_actions.append(a)
    return random.choice(best_actions) if best_actions else "S"


@dataclass
class PatrolLoopPolicy(SensorModePolicy):
    loop: List[Tuple[int, int]] = field(default_factory=list)

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        if not self.loop:
            return "S"
        idx = t % len(self.loop)
        return greedy_step_toward(world, sensor_pos, self.loop[idx])


@dataclass
class ChasePolicy(SensorModePolicy):
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        return greedy_step_toward(world, sensor_pos, robot_pos)


@dataclass
class RandomWalkPolicy(SensorModePolicy):
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        return random.choice(ACTIONS)


# =========================
# Detection / Observation
# =========================

@dataclass
class DetectionModel:
    d0: int = 2
    p_close: float = 0.9
    p_far: float = 0.05
    decay: float = 0.3

    def p_detect(self, robot_pos: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        dist = manhattan(robot_pos, sensor_pos)
        if dist <= self.d0:
            return self.p_close
        return self.p_far + (self.p_close - self.p_far) * math.exp(-self.decay * (dist - self.d0))


@dataclass
class ObservationModel:
    sigma: float = 1.2

    def sample_observation(self, sensor_pos: Tuple[int, int], world: GridWorld) -> Tuple[int, int]:
        max_jump = 2
        dr = random.randint(-max_jump, max_jump)
        dc = random.randint(-max_jump, max_jump)
        o = (sensor_pos[0] + dr, sensor_pos[1] + dc)
        o = (int(clamp(o[0], 0, world.H - 1)), int(clamp(o[1], 0, world.W - 1)))
        return o

    def likelihood(self, obs: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        d = manhattan(obs, sensor_pos)
        return max(1e-12, math.exp(-d / max(1e-6, self.sigma)))


# =========================
# Particle Filter over (sensor_pos, mode)
# =========================

@dataclass
class Particle:
    sensor_pos: Tuple[int, int]
    mode_idx: int
    weight: float = 1.0


@dataclass
class BeliefState:
    particles: List[Particle]
    mode_names: List[str]

    def normalize(self) -> None:
        s = sum(p.weight for p in self.particles)
        if s <= 0.0:
            w = 1.0 / max(1, len(self.particles))
            for p in self.particles:
                p.weight = w
            return
        for p in self.particles:
            p.weight /= s

    def effective_sample_size(self) -> float:
        s = sum(p.weight * p.weight for p in self.particles)
        return 0.0 if s <= 0.0 else 1.0 / s

    def mode_posterior(self) -> Dict[int, float]:
        post: Dict[int, float] = defaultdict(float)
        for p in self.particles:
            post[p.mode_idx] += p.weight
        return dict(post)

    def mode_posterior_named(self) -> Dict[str, float]:
        post_i = self.mode_posterior()
        out: Dict[str, float] = {}
        for i, v in post_i.items():
            name = self.mode_names[i] if 0 <= i < len(self.mode_names) else f"mode{i}"
            out[name] = out.get(name, 0.0) + v
        return out


def systematic_resample(particles: List[Particle]) -> List[Particle]:
    n = len(particles)
    if n == 0:
        return []
    positions = [(random.random() + i) / n for i in range(n)]
    cumulative = []
    csum = 0.0
    for p in particles:
        csum += p.weight
        cumulative.append(csum)

    out: List[Particle] = []
    i = 0
    for pos in positions:
        while i < n - 1 and pos > cumulative[i]:
            i += 1
        chosen = particles[i]
        out.append(Particle(sensor_pos=chosen.sensor_pos, mode_idx=chosen.mode_idx, weight=1.0 / n))
    return out


def initialize_belief(
    world: GridWorld,
    sensor_init_candidates: List[Tuple[int, int]],
    mode_prior: List[float],
    mode_names: List[str],
    num_particles: int = 200,
) -> BeliefState:
    if len(mode_prior) != len(mode_names):
        raise ValueError("mode_prior and mode_names must have same length")
    if not sensor_init_candidates:
        raise ValueError("Need at least one sensor init candidate")
    if num_particles <= 0:
        raise ValueError("num_particles must be > 0")

    # normalize prior
    s = sum(mode_prior)
    if s <= 0.0:
        mode_prior = [1.0 / len(mode_prior)] * len(mode_prior)
    else:
        mode_prior = [p / s for p in mode_prior]

    def sample_mode() -> int:
        r = random.random()
        c = 0.0
        for i, p in enumerate(mode_prior):
            c += p
            if r <= c:
                return i
        return len(mode_prior) - 1

    particles: List[Particle] = []
    for _ in range(num_particles):
        sp = random.choice(sensor_init_candidates)
        z = sample_mode()
        particles.append(Particle(sensor_pos=sp, mode_idx=z, weight=1.0 / num_particles))

    b = BeliefState(particles=particles, mode_names=mode_names)
    b.normalize()
    return b


@dataclass
class PFStepInfo:
    ess_before: float
    ess_after: float
    resampled: bool


def belief_predict_update(
    belief: BeliefState,
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    obs_model: ObservationModel,
    obs: Tuple[int, int],
    t: int,
    *,
    motion_noise: float = 0.10,
    resample_threshold: float = 0.5,
    log: Optional[Logger] = None,
) -> PFStepInfo:
    if not belief.particles:
        raise ValueError("belief has no particles")

    belief.normalize()
    ess0 = belief.effective_sample_size()

    # Predict + weight update
    for p in belief.particles:
        mi = p.mode_idx
        if 0 <= mi < len(sensor_policies):
            aS = sensor_policies[mi].action(world, p.sensor_pos, robot_pos, t)
        else:
            aS = "S"

        predicted = world.step(p.sensor_pos, aS)
        if random.random() < motion_noise:
            predicted = world.step(predicted, random.choice(ACTIONS))

        p.sensor_pos = predicted
        p.weight *= obs_model.likelihood(obs, p.sensor_pos)

    belief.normalize()
    ess1 = belief.effective_sample_size()

    # Sanity checks
    N = len(belief.particles)
    soft_assert(0.0 <= ess1 <= N + 1e-6, f"ESS out of bounds: {ess1} (N={N})", log=log, t=t)

    resampled = False
    if ess1 < resample_threshold * N:
        belief.particles = systematic_resample(belief.particles)
        belief.normalize()
        resampled = True

    # Controlled prints
    if log is not None:
        if log._should_print(t, log.cfg.pf_every_t, lvl=1) or resampled:
            mp = belief.mode_posterior_named()
            log.log(
                f"[PF] t={t} obs={obs} ESS {ess0:.1f}->{ess1:.1f} resample={resampled} | modes: {topk_str(mp, k=3)}",
                t=t,
                every=log.cfg.pf_every_t,
                lvl=1,
                force=resampled,
            )
            if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
                log.log(f"[PF][detail] full mode posterior: {topk_str(mp, k=10)}", t=t, every=None, lvl=2)

    return PFStepInfo(ess_before=ess0, ess_after=ess1, resampled=resampled)


# =========================
# A* Search with Risk Costs
# =========================

@dataclass
class AStarResult:
    path: List[Tuple[int, int]]
    cost: float
    expanded: int


def astar_path(
    world: GridWorld,
    start: Tuple[int, int],
    goal: Tuple[int, int],
    step_cost: Callable[[Tuple[int, int]], float],
    heuristic: Callable[[Tuple[int, int]], float],
    *,
    max_expansions: int = 50_000,
) -> Optional[AStarResult]:
    if not world.is_free(start) or not world.is_free(goal):
        return None

    frontier: List[Tuple[float, float, Tuple[int, int]]] = []
    heapq.heappush(frontier, (heuristic(start), 0.0, start))

    came_from: Dict[Tuple[int, int], Tuple[int, int]] = {}
    gscore: Dict[Tuple[int, int], float] = {start: 0.0}

    expanded = 0

    # A* with reopen: skip stale pops
    while frontier and expanded < max_expansions:
        f, g, cur = heapq.heappop(frontier)
        if g > gscore.get(cur, float("inf")) + 1e-12:
            continue
        expanded += 1

        if cur == goal:
            path = [cur]
            while cur in came_from:
                cur = came_from[cur]
                path.append(cur)
            path.reverse()
            return AStarResult(path=path, cost=gscore[goal], expanded=expanded)

        for (nxt, _) in world.neighbors(cur):
            if not world.is_free(nxt):
                continue
            sc = step_cost(nxt)
            if not math.isfinite(sc) or sc < 0:
                continue
            ng = gscore[cur] + sc
            if ng + 1e-12 < gscore.get(nxt, float("inf")):
                gscore[nxt] = ng
                came_from[nxt] = cur
                heapq.heappush(frontier, (ng + heuristic(nxt), ng, nxt))

    return None


# =========================
# Multimodal Path Library
# =========================

@dataclass
class PathMode:
    path: List[Tuple[int, int]]
    total_cost: float
    hypothesis_sensor_pos: Tuple[int, int]
    hypothesis_mode_idx: int
    debug_info: Dict[str, Any] = field(default_factory=dict)


def build_cost_map_from_hypothesis(
    world: GridWorld,
    detection: DetectionModel,
    sensor_pos: Tuple[int, int],
    *,
    base_step: float,
    w_detect: float,
    randomize: bool,
    rand_scale: float,
) -> Callable[[Tuple[int, int]], float]:
    def cost_fn(s: Tuple[int, int]) -> float:
        if not world.is_free(s):
            return float("inf")
        p_det = detection.p_detect(robot_pos=s, sensor_pos=sensor_pos)
        c = base_step + w_detect * p_det
        if randomize:
            c *= (1.0 + rand_scale * (random.random() - 0.5))
            c = max(1e-6, c)
        return c

    return cost_fn


def cluster_paths_by_signature(paths: List[List[Tuple[int, int]]], *, max_modes: int) -> List[int]:
    sig_to_cluster: Dict[str, int] = {}
    assignments: List[int] = []

    def signature(path: List[Tuple[int, int]]) -> str:
        if len(path) < 2:
            return "EMPTY"
        dirs = []
        for i in range(1, len(path)):
            dr = path[i][0] - path[i - 1][0]
            dc = path[i][1] - path[i - 1][1]
            dirs.append((dr, dc))

        comp = []
        for d in dirs:
            if not comp or comp[-1] != d:
                comp.append(d)

        m = {(-1, 0): "U", (1, 0): "D", (0, -1): "L", (0, 1): "R", (0, 0): "S"}
        return "".join(m.get(d, "?") for d in comp)[:120]

    for p in paths:
        sig = signature(p)

        if sig in sig_to_cluster:
            cid = sig_to_cluster[sig]
        elif len(sig_to_cluster) < max_modes:
            cid = len(sig_to_cluster)
            sig_to_cluster[sig] = cid
        else:
            # stable assignment to an existing bucket
            cid = stable_bucket(sig, len(sig_to_cluster))

        assignments.append(cid)

    return assignments


@dataclass
class ModeLibraryInfo:
    num_candidates: int
    num_clusters: int


def build_multimodal_path_library(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    *,
    num_hypotheses: int = 30,
    per_hypothesis_attempts: int = 2,
    base_step_cost: float = 1.0,
    w_detect: float = 8.0,
    rand_scale: float = 0.40,
    max_modes: int = 6,
    t: int = 0,
    log: Optional[Logger] = None,
) -> Tuple[List[PathMode], ModeLibraryInfo]:
    if not belief.particles:
        return [], ModeLibraryInfo(num_candidates=0, num_clusters=0)

    # Sample particles proportional to weight
    weights = [p.weight for p in belief.particles]
    s = sum(weights)
    if s <= 0.0:
        weights = [1.0 / len(weights)] * len(weights)
    else:
        weights = [w / s for w in weights]

    sampled_particles = random.choices(belief.particles, weights=weights, k=num_hypotheses)

    candidates: List[PathMode] = []

    # Only print a couple hypotheses in detail
    hyp_print_budget = log.cfg.max_hyp_print if (log and log.cfg.level >= 2 and t <= log.cfg.detail_until_t) else 0

    for hi, hp in enumerate(sampled_particles):
        hyp_sensor = hp.sensor_pos
        hyp_mode = hp.mode_idx

        if log is not None and hyp_print_budget > 0:
            log.log(
                f"[Modes][detail] t={t} hyp#{hi+1}/{num_hypotheses} sensor={hyp_sensor} mode={belief.mode_names[hyp_mode] if 0<=hyp_mode<len(belief.mode_names) else hyp_mode}",
                t=t,
                every=None,
                lvl=2,
            )
            hyp_print_budget -= 1

        for attempt in range(per_hypothesis_attempts):
            cost_fn = build_cost_map_from_hypothesis(
                world,
                detection,
                hyp_sensor,
                base_step=base_step_cost,
                w_detect=w_detect,
                randomize=True,
                rand_scale=rand_scale,
            )
            heur = lambda s: float(manhattan(s, goal))

            res = astar_path(world, robot_pos, goal, step_cost=cost_fn, heuristic=heur)
            if res is None:
                continue

            candidates.append(
                PathMode(
                    path=res.path,
                    total_cost=res.cost,
                    hypothesis_sensor_pos=hyp_sensor,
                    hypothesis_mode_idx=hyp_mode,
                    debug_info={"expanded": res.expanded, "attempt": attempt + 1},
                )
            )

    if not candidates:
        if log is not None and log._should_print(t, log.cfg.modes_every_t, lvl=1):
            log.log(f"[Modes] t={t} candidates=0 (no paths found)", t=t, every=log.cfg.modes_every_t, lvl=1)
        return [], ModeLibraryInfo(num_candidates=0, num_clusters=0)

    # Cluster and keep best per cluster
    cluster_ids = cluster_paths_by_signature([c.path for c in candidates], max_modes=max_modes)

    best_by_cluster: Dict[int, PathMode] = {}
    for c, cid in zip(candidates, cluster_ids):
        if cid not in best_by_cluster or c.total_cost < best_by_cluster[cid].total_cost:
            best_by_cluster[cid] = c

    library = sorted(best_by_cluster.values(), key=lambda pm: pm.total_cost)[:max_modes]

    if log is not None and log._should_print(t, log.cfg.modes_every_t, lvl=1):
        costs = [pm.total_cost for pm in library]
        cost_str = ", ".join(f"{c:.1f}" for c in costs[:min(3, len(costs))])
        log.log(
            f"[Modes] t={t} candidates={len(candidates)} clusters={len(best_by_cluster)} kept={len(library)} best_costs=[{cost_str}]",
            t=t,
            every=log.cfg.modes_every_t,
            lvl=1,
        )

    return library, ModeLibraryInfo(num_candidates=len(candidates), num_clusters=len(best_by_cluster))


def first_action_from_path(world: GridWorld, start: Tuple[int, int], path: List[Tuple[int, int]]) -> Action:
    if len(path) < 2:
        return "S"
    nxt = path[1]
    dr = nxt[0] - start[0]
    dc = nxt[1] - start[1]
    for a, (adr, adc) in ACTION_DELTAS.items():
        if (dr, dc) == (adr, adc):
            return a
    return "S"


def build_action_prior_from_library(world: GridWorld, robot_pos: Tuple[int, int], library: List[PathMode]) -> Dict[Action, float]:
    if not library:
        return {a: 1.0 / len(ACTIONS) for a in ACTIONS}

    pri_raw: Dict[Action, float] = defaultdict(float)
    for pm in library:
        a0 = first_action_from_path(world, robot_pos, pm.path)
        pri_raw[a0] += 1.0 / max(1e-6, pm.total_cost)

    return normalize_probs_dict(pri_raw, ACTIONS)


# =========================
# GenBR-lite: Belief-space MCTS with PUCT
# =========================

@dataclass
class MCTSNode:
    robot_pos: Tuple[int, int]
    depth: int

    N: int = 0
    N_a: Dict[Action, int] = field(default_factory=lambda: {a: 0 for a in ACTIONS})
    W_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})
    Q_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})

    P_a: Dict[Action, float] = field(default_factory=lambda: {a: 1.0 / len(ACTIONS) for a in ACTIONS})

    children: Dict[Action, "MCTSNode"] = field(default_factory=dict)


def select_action_puct(node: MCTSNode, *, c_puct: float) -> Action:
    """PUCT: argmax_a Q + c*P*sqrt(N)/(1+N_a). Uses node.P_a."""
    sqrtN = math.sqrt(max(1, node.N))

    best_a: Optional[Action] = None
    best_score = -1e18

    for a in ACTIONS:
        q = node.Q_a[a]
        u = c_puct * node.P_a[a] * sqrtN / (1.0 + node.N_a[a])
        score = q + u + random.uniform(-1e-9, 1e-9)
        if score > best_score:
            best_score = score
            best_a = a

    assert best_a is not None
    return best_a


def backprop_path(path: List[Tuple[MCTSNode, Action]], value: float) -> None:
    for node, a in reversed(path):
        node.N += 1
        node.N_a[a] += 1
        node.W_a[a] += value
        node.Q_a[a] = node.W_a[a] / node.N_a[a]


def choose_final_action(root: MCTSNode) -> Action:
    maxN = max(root.N_a.values())
    cand = [a for a in ACTIONS if root.N_a[a] == maxN]
    if len(cand) == 1:
        return cand[0]

    maxQ = max(root.Q_a[a] for a in cand)
    cand2 = [a for a in cand if abs(root.Q_a[a] - maxQ) < 1e-12]
    if len(cand2) == 1:
        return cand2[0]

    maxP = max(root.P_a[a] for a in cand2)
    cand3 = [a for a in cand2 if abs(root.P_a[a] - maxP) < 1e-12]
    return random.choice(cand3)


def rollout_value(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    *,
    max_steps: int,
    w_step: float,
    w_detect: float,
) -> float:
    total_cost = 0.0
    cur = robot_pos
    for _ in range(max_steps):
        if cur == goal:
            break
        a = greedy_step_toward(world, cur, goal)
        cur = world.step(cur, a)
        total_cost += w_step
        total_cost += w_detect * detection.p_detect(cur, sensor_pos)
    if cur == goal:
        total_cost -= 20.0
    return -total_cost


def simulate_sensor_step(
    world: GridWorld,
    sensor_pos: Tuple[int, int],
    robot_pos: Tuple[int, int],
    t: int,
    mode_idx: int,
    policies: List[SensorModePolicy],
) -> Tuple[int, int]:
    if 0 <= mode_idx < len(policies):
        aS = policies[mode_idx].action(world, sensor_pos, robot_pos, t)
        return world.step(sensor_pos, aS)
    return sensor_pos


def mcts_search_action(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    detection: DetectionModel,
    action_prior: Dict[Action, float],
    *,
    t: int,
    num_sims: int = 250,
    max_depth: int = 10,
    c_puct: float = 1.4,
    w_step: float = 1.0,
    w_detect: float = 7.0,
    gamma: float = 0.95,
    log: Optional[Logger] = None,
) -> Action:
    # Smooth and validate prior
    prior = normalize_probs_dict(action_prior, ACTIONS, eps=0.10)

    root = MCTSNode(robot_pos=robot_pos, depth=0)
    root.P_a = prior

    # Prepare particle sampling distribution
    particles = belief.particles if belief.particles else [Particle(sensor_pos=robot_pos, mode_idx=0, weight=1.0)]
    weights = [float(getattr(p, "weight", 1.0)) for p in particles]
    s = sum(weights)
    if s <= 0.0:
        weights = [1.0 / len(weights)] * len(weights)
    else:
        weights = [w / s for w in weights]

    # Optional header
    if log is not None and (log._should_print(t, log.cfg.mcts_every_t, lvl=1)):
        log.log(f"[MCTS] t={t} sims={num_sims} depth={max_depth} prior_top: {topk_str(prior, k=3)}", t=t, every=log.cfg.mcts_every_t, lvl=1)

    for sim in range(num_sims):
        hp = random.choices(particles, weights=weights, k=1)[0]
        sim_sensor = hp.sensor_pos
        sim_mode = hp.mode_idx

        node = root
        cur_robot = robot_pos
        cur_sensor = sim_sensor

        path: List[Tuple[MCTSNode, Action]] = []
        total_return = 0.0
        disc = 1.0

        # Corner case: already at goal
        if cur_robot == goal:
            root.N += 1
            continue

        for depth in range(max_depth):
            if cur_robot == goal:
                total_return += disc * 50.0
                break

            a = select_action_puct(node, c_puct=c_puct)
            path.append((node, a))

            nxt_robot = world.step(cur_robot, a)
            nxt_sensor = simulate_sensor_step(world, cur_sensor, nxt_robot, t + depth, sim_mode, sensor_policies)

            step_cost = w_step + w_detect * detection.p_detect(nxt_robot, nxt_sensor)
            total_return += disc * (-step_cost)
            disc *= gamma

            if a not in node.children:
                child = MCTSNode(robot_pos=nxt_robot, depth=node.depth + 1)
                child.P_a = prior
                node.children[a] = child

                leaf_v = rollout_value(
                    world,
                    nxt_robot,
                    nxt_sensor,
                    goal,
                    detection,
                    max_steps=max_depth - depth - 1,
                    w_step=w_step,
                    w_detect=w_detect,
                )
                total_return += disc * leaf_v
                break

            node = node.children[a]
            cur_robot, cur_sensor = nxt_robot, nxt_sensor

        if path:
            backprop_path(path, total_return)
        else:
            # If something went weird and we didn't select an action, still count a visit
            root.N += 1

        if log is not None and log.cfg.level >= 2 and (sim < 2 or ((sim + 1) % log.cfg.mcts_progress_every_sims == 0 and t <= log.cfg.detail_until_t)):
            log.log(f"[MCTS][detail] t={t} sim={sim+1}/{num_sims} root.N={root.N}", t=t, every=None, lvl=2)

    # Post-checks
    total_edge_visits = sum(root.N_a.values())
    # In this implementation root.N should be close to num_sims (exact if every sim selects at least 1 action)
    if log is not None:
        if log._should_print(t, log.cfg.mcts_every_t, lvl=1):
            log.log(
                f"[MCTS] t={t} root.N={root.N} sum(N_a)={total_edge_visits} best_by_N={max(root.N_a, key=lambda a: root.N_a[a])}",
                t=t,
                every=log.cfg.mcts_every_t,
                lvl=1,
            )

            if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
                for a in ACTIONS:
                    log.log(f"[MCTS][detail] a={a} N={root.N_a[a]:4d} Q={root.Q_a[a]:+8.3f} P={root.P_a[a]:.3f}", t=t, every=None, lvl=2)

    chosen = choose_final_action(root)
    return chosen


# =========================
# Main Simulation
# =========================

@dataclass
class Config:
    seed: int = 7

    H: int = 12
    W: int = 18
    T: int = 40

    num_particles: int = 250

    # PF
    motion_noise: float = 0.15
    resample_threshold: float = 0.55

    # Mode library
    num_hypotheses: int = 25
    per_hypothesis_attempts: int = 2
    max_modes: int = 6
    w_detect_astar: float = 10.0
    rand_scale: float = 0.45

    # MCTS
    num_sims: int = 250
    max_depth: int = 10
    c_puct: float = 1.4
    w_step_mcts: float = 1.0
    w_detect_mcts: float = 7.0

    # Debug
    debug: DebugConfig = field(default_factory=DebugConfig)


def make_demo_world(cfg: Config) -> Tuple[GridWorld, Tuple[int, int], Tuple[int, int], Tuple[int, int]]:
    obs = set()
    for c in range(3, 15):
        obs.add((5, c))
    for r in range(1, 9):
        obs.add((r, 9))
    obs.discard((5, 7))
    obs.discard((6, 9))

    world = GridWorld(H=cfg.H, W=cfg.W, obstacles=obs)
    robot_start = (10, 2)
    goal = (1, 16)
    sensor_start = (2, 2)
    return world, robot_start, sensor_start, goal


def main() -> None:
    cfg = Config()
    seed_all(cfg.seed)

    log = Logger(cfg.debug)

    world, robot_pos, sensor_pos, goal = make_demo_world(cfg)

    sensor_modes: List[SensorModePolicy] = [
        PatrolLoopPolicy(name="patrol_left", loop=[(2, 2), (2, 6), (4, 6), (4, 2)]),
        PatrolLoopPolicy(name="patrol_mid", loop=[(2, 10), (2, 13), (4, 13), (4, 10)]),
        ChasePolicy(name="chase_robot"),
        RandomWalkPolicy(name="random_walk"),
    ]
    mode_names = [m.name for m in sensor_modes]

    true_mode_idx = random.randrange(len(sensor_modes))

    detection = DetectionModel()
    obs_model = ObservationModel(sigma=1.3)

    sensor_init_candidates = [(2, 2), (2, 10), (3, 3), (3, 9), (1, 1)]
    mode_prior = [1.0] * len(sensor_modes)
    belief = initialize_belief(world, sensor_init_candidates, mode_prior, mode_names, cfg.num_particles)

    log.log(f"[INIT] true_mode={mode_names[true_mode_idx]} robot={robot_pos} sensor={sensor_pos} goal={goal}", force=True)
    log.log(f"[INIT] belief modes: {topk_str(belief.mode_posterior_named(), k=4)}", force=True)

    for t in range(cfg.T):
        # Minimal header
        show_header = (t <= cfg.debug.detail_until_t) or (cfg.debug.enabled and cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)
        if show_header:
            log.log("=" * 80, force=True)
            log.log(f"[TIME] t={t}", force=True)

        # Optional render
        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.render_every_t > 0 and t % cfg.debug.render_every_t == 0)):
            log.log("[WORLD]", t=t, every=cfg.debug.render_every_t, lvl=1, force=(t <= cfg.debug.detail_until_t))
            world.render(robot=robot_pos, sensor=sensor_pos, goal=goal)

        obs = obs_model.sample_observation(sensor_pos, world)
        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[OBS] obs={obs} (true sensor={sensor_pos})", force=True)

        # PF update
        pf_info = belief_predict_update(
            belief,
            world,
            robot_pos,
            sensor_modes,
            obs_model,
            obs,
            t,
            motion_noise=cfg.motion_noise,
            resample_threshold=cfg.resample_threshold,
            log=log,
        )

        # Mode library -> prior
        library, lib_info = build_multimodal_path_library(
            world,
            belief,
            robot_pos,
            goal,
            detection,
            num_hypotheses=cfg.num_hypotheses,
            per_hypothesis_attempts=cfg.per_hypothesis_attempts,
            w_detect=cfg.w_detect_astar,
            rand_scale=cfg.rand_scale,
            max_modes=cfg.max_modes,
            t=t,
            log=log,
        )
        action_prior = build_action_prior_from_library(world, robot_pos, library)

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[Prior] top: {topk_str(action_prior, k=3)}", force=True)

        # MCTS choose action
        aR = mcts_search_action(
            world,
            belief,
            robot_pos,
            goal,
            sensor_modes,
            detection,
            action_prior,
            t=t,
            num_sims=cfg.num_sims,
            max_depth=cfg.max_depth,
            c_puct=cfg.c_puct,
            w_step=cfg.w_step_mcts,
            w_detect=cfg.w_detect_mcts,
            log=log,
        )

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[ACT] robot aR={aR}", force=True)

        # Step robot and true sensor
        new_robot = world.step(robot_pos, aR)
        aS_true = sensor_modes[true_mode_idx].action(world, sensor_pos, new_robot, t)
        new_sensor = world.step(sensor_pos, aS_true)

        robot_pos, sensor_pos = new_robot, new_sensor

        # Detection
        p_det = detection.p_detect(robot_pos, sensor_pos)
        detected = (random.random() < p_det)

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[STEP] robot={robot_pos} sensor={sensor_pos} p_det={p_det:.3f} detected={detected}", force=True)

        # Terminate
        if robot_pos == goal:
            log.log("[TERMINAL] reached goal ✅", force=True)
            break
        if detected and manhattan(robot_pos, sensor_pos) <= detection.d0:
            log.log("[TERMINAL] caught ❌", force=True)
            break

    log.log("[DONE]", force=True)


if __name__ == "__main__":
    main()


[INIT] true_mode=chase_robot robot=(10, 2) sensor=(2, 2) goal=(1, 16)
[INIT] belief modes: patrol_left:0.268 | random_walk:0.260 | patrol_mid:0.248 | chase_robot:0.224
[TIME] t=0
[WORLD]
..................
.........#......G.
..S......#........
.........#........
.........#........
...####.#######...
..................
.........#........
.........#........
..................
..R...............
..................
[OBS] obs=(2, 3) (true sensor=(2, 2))
[PF] t=0 obs=(2, 3) ESS 250.0->91.6 resample=True | modes: patrol_left:0.348 | patrol_mid:0.296 | chase_robot:0.184
[Modes] t=0 candidates=50 clusters=6 kept=6 best_costs=[34.8, 34.9, 37.5]
[Prior] top: R:1.000 | U:0.000 | D:0.000
[MCTS] t=0 sims=250 depth=10 prior_top: R:0.920 | U:0.020 | D:0.020
[MCTS] t=0 root.N=250 sum(N_a)=250 best_by_N=R
[ACT] robot aR=R
[STEP] robot=(10, 3) sensor=(2, 3) p_det=0.191 detected=False
[TIME] t=1
[WORLD]
..................
.........#......G.
...S.....#........
.........#........
.........#........
...####.

In [14]:
#!/usr/bin/env python3
"""GenBR-lite + PF Belief + Multimodal Path Library (debug-robust)

This version includes the specific fixes suggested from your logs:
1) **Chase policy sanity + determinism**:
   - Chase now uses a deterministic tie-break (no random sideways moves on ties).
   - We print (and warn on) any chase step that *increases* Manhattan distance when a decreasing move exists.

2) **Mode-library selection uses plausibility + cost** (not cost-only optimism):
   - Each candidate path is scored as:  score = cost + lambda_plaus * (-log(hyp_weight)).
   - Best-per-cluster and final sorting use this score.

3) **Action prior is a mixture over modes** (not single-best-path):
   - Prior weights each kept mode by: hyp_weight * exp(-cost/temp).

4) **Detection is actually terminal**:
   - If detected and within d0, episode terminates as "caught".

What to send me to verify correctness:
- Run with DebugConfig(level=2, detail_until_t=6, summary_every_t=1, render_every_t=1) for ~10 steps.
- Paste the log lines that start with: [CHASE], [CHASE][WARN], [ModesKept], [PriorDiag], [MCTS]

Run:
  python3 this_file.py
"""

from __future__ import annotations

import math
import random
import heapq
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Iterable, Callable, Any, Set
from collections import defaultdict
import hashlib


# =========================
# Debug / Logging utilities
# =========================

@dataclass
class DebugConfig:
    enabled: bool = True
    level: int = 1
    # 0 = silent
    # 1 = periodic summaries
    # 2 = detailed (still gated)

    # Print detail for the first few timesteps
    detail_until_t: int = 2

    # Periodic summary controls
    summary_every_t: int = 5
    render_every_t: int = 10

    # Stage-specific periodic prints
    pf_every_t: int = 5
    modes_every_t: int = 5
    mcts_every_t: int = 5

    # Within MCTS
    mcts_progress_every_sims: int = 50

    # Safety: cap printing of mode hypotheses even at detail level
    max_hyp_print: int = 3


class Logger:
    def __init__(self, cfg: DebugConfig):
        self.cfg = cfg

    def _should_print(self, t: Optional[int], every: Optional[int], lvl: int) -> bool:
        if not self.cfg.enabled:
            return False
        if lvl > self.cfg.level:
            return False
        if t is None:
            return True
        if t <= self.cfg.detail_until_t:
            return True
        if every is None:
            return False
        return (every > 0) and (t % every == 0)

    def log(self, msg: str, *, t: Optional[int] = None, every: Optional[int] = None, lvl: int = 1, force: bool = False) -> None:
        if force or self._should_print(t, every, lvl):
            print(msg)


def soft_assert(cond: bool, msg: str, *, fatal: bool = False, log: Optional[Logger] = None, t: Optional[int] = None) -> None:
    if cond:
        return
    if log is not None:
        log.log(f"[WARN] {msg}", t=t, every=None, lvl=1, force=True)
    if fatal:
        raise AssertionError(msg)


def clamp(x: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, x))


def manhattan(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def seed_all(seed: int) -> None:
    random.seed(seed)


def stable_bucket(sig: str, m: int) -> int:
    """Stable mapping sig -> [0..m-1] independent of Python's randomized hash."""
    if m <= 0:
        return 0
    h = hashlib.md5(sig.encode("utf-8")).hexdigest()
    return int(h[:8], 16) % m


def normalize_probs_dict(d: Dict[Any, float], keys: Iterable[Any], *, eps: float = 0.0) -> Dict[Any, float]:
    """Return a normalized distribution over `keys` from possibly-missing/negative inputs.

    - Clips negatives to 0
    - Adds optional epsilon smoothing to avoid degeneracy
    - Falls back to uniform if total mass is 0
    """
    klist = list(keys)
    out = {k: max(0.0, float(d.get(k, 0.0))) for k in klist}
    s = sum(out.values())

    if s <= 0.0:
        u = 1.0 / max(1, len(klist))
        return {k: u for k in klist}

    out = {k: v / s for k, v in out.items()}

    if eps > 0.0:
        u = 1.0 / max(1, len(klist))
        out = {k: (1.0 - eps) * out[k] + eps * u for k in klist}
        s2 = sum(out.values())
        out = {k: v / s2 for k, v in out.items()}

    return out


def topk_str(d: Dict[Any, float], k: int = 3, fmt: str = "{k}:{v:.3f}") -> str:
    items = sorted(d.items(), key=lambda kv: kv[1], reverse=True)[:k]
    return " | ".join(fmt.format(k=kk, v=vv) for kk, vv in items)


# =========================
# Gridworld Environment
# =========================

Action = str  # 'U','D','L','R','S'

ACTIONS: List[Action] = ["U", "D", "L", "R", "S"]
ACTION_DELTAS: Dict[Action, Tuple[int, int]] = {
    "U": (-1, 0),
    "D": (1, 0),
    "L": (0, -1),
    "R": (0, 1),
    "S": (0, 0),
}


@dataclass
class GridWorld:
    H: int
    W: int
    obstacles: Set[Tuple[int, int]] = field(default_factory=set)

    def in_bounds(self, p: Tuple[int, int]) -> bool:
        r, c = p
        return 0 <= r < self.H and 0 <= c < self.W

    def is_free(self, p: Tuple[int, int]) -> bool:
        return self.in_bounds(p) and (p not in self.obstacles)

    def step(self, p: Tuple[int, int], a: Action) -> Tuple[int, int]:
        if a not in ACTION_DELTAS:
            raise ValueError(f"Unknown action: {a}")
        dr, dc = ACTION_DELTAS[a]
        np = (p[0] + dr, p[1] + dc)
        return np if self.is_free(np) else p

    def neighbors(self, p: Tuple[int, int]) -> List[Tuple[Tuple[int, int], Action]]:
        return [(self.step(p, a), a) for a in ACTIONS]

    def render(self, robot: Tuple[int, int], sensor: Tuple[int, int], goal: Tuple[int, int]) -> None:
        grid = [["." for _ in range(self.W)] for _ in range(self.H)]
        for (r, c) in self.obstacles:
            grid[r][c] = "#"
        rr, rc = robot
        sr, sc = sensor
        gr, gc = goal
        grid[gr][gc] = "G"
        grid[sr][sc] = "S"
        grid[rr][rc] = "R"
        for r in range(self.H):
            print("".join(grid[r]))


# =========================
# Sensor Models (Modes)
# =========================

@dataclass
class SensorModePolicy:
    name: str

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        raise NotImplementedError


def best_actions_toward(world: GridWorld, src: Tuple[int, int], dst: Tuple[int, int]) -> Tuple[List[Action], int]:
    """Return (best_actions, best_dist) minimizing manhattan(next(src,a), dst)."""
    best_actions: List[Action] = []
    best_dist = 10**9
    for a in ACTIONS:
        np = world.step(src, a)
        d = manhattan(np, dst)
        if d < best_dist:
            best_dist = d
            best_actions = [a]
        elif d == best_dist:
            best_actions.append(a)
    return best_actions, best_dist


def deterministic_greedy_step_toward(world: GridWorld, src: Tuple[int, int], dst: Tuple[int, int]) -> Action:
    """Greedy toward dst with deterministic tie-break (easier to debug than random ties)."""
    best_as, _ = best_actions_toward(world, src, dst)
    if not best_as:
        return "S"

    # Preferred axis: move along the larger |delta| if possible
    dr = dst[0] - src[0]
    dc = dst[1] - src[1]

    preferred: List[Action] = []
    if abs(dr) >= abs(dc):
        if dr > 0:
            preferred.append("D")
        elif dr < 0:
            preferred.append("U")
    if abs(dc) > abs(dr):
        if dc > 0:
            preferred.append("R")
        elif dc < 0:
            preferred.append("L")

    # If the preferred move is among best actions, take it
    for a in preferred:
        if a in best_as:
            return a

    # Otherwise deterministic order
    for a in ["U", "D", "L", "R", "S"]:
        if a in best_as:
            return a

    return best_as[0]


def chase_sanity_check(
    world: GridWorld,
    src: Tuple[int, int],
    dst: Tuple[int, int],
    chosen: Action,
    *,
    log: Optional[Logger],
    t: int,
    tag: str,
) -> None:
    """Warn if chosen action increases distance when a non-increasing action exists."""
    d0 = manhattan(src, dst)
    nxt = world.step(src, chosen)
    d1 = manhattan(nxt, dst)

    best_as, best_d = best_actions_toward(world, src, dst)

    if log is not None and log.cfg.enabled and (t <= log.cfg.detail_until_t or (log.cfg.summary_every_t > 0 and t % log.cfg.summary_every_t == 0)):
        log.log(
            f"[CHASE] {tag} t={t} src={src} dst={dst} a={chosen} nxt={nxt} d:{d0}->{d1} best_d={best_d} best={best_as}",
            t=t,
            every=log.cfg.summary_every_t,
            lvl=1,
        )

    # If chosen isn't among best actions, it's suspicious
    if chosen not in best_as:
        if log is not None:
            log.log(
                f"[CHASE][WARN] {tag} t={t} chose {chosen} but best={best_as} (d:{d0}->{d1}, best_d={best_d})",
                t=t,
                every=None,
                lvl=1,
                force=True,
            )


@dataclass
class PatrolLoopPolicy(SensorModePolicy):
    loop: List[Tuple[int, int]] = field(default_factory=list)

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        if not self.loop:
            return "S"
        idx = t % len(self.loop)
        return deterministic_greedy_step_toward(world, sensor_pos, self.loop[idx])


@dataclass
class ChasePolicy(SensorModePolicy):
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        return deterministic_greedy_step_toward(world, sensor_pos, robot_pos)


@dataclass
class RandomWalkPolicy(SensorModePolicy):
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        return random.choice(ACTIONS)


# =========================
# Detection / Observation
# =========================

@dataclass
class DetectionModel:
    d0: int = 2
    p_close: float = 0.9
    p_far: float = 0.05
    decay: float = 0.3

    def p_detect(self, robot_pos: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        dist = manhattan(robot_pos, sensor_pos)
        if dist <= self.d0:
            return self.p_close
        return self.p_far + (self.p_close - self.p_far) * math.exp(-self.decay * (dist - self.d0))


@dataclass
class ObservationModel:
    sigma: float = 1.2

    def sample_observation(self, sensor_pos: Tuple[int, int], world: GridWorld) -> Tuple[int, int]:
        max_jump = 2
        dr = random.randint(-max_jump, max_jump)
        dc = random.randint(-max_jump, max_jump)
        o = (sensor_pos[0] + dr, sensor_pos[1] + dc)
        o = (int(clamp(o[0], 0, world.H - 1)), int(clamp(o[1], 0, world.W - 1)))
        return o

    def likelihood(self, obs: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        d = manhattan(obs, sensor_pos)
        return max(1e-12, math.exp(-d / max(1e-6, self.sigma)))


# =========================
# Particle Filter over (sensor_pos, mode)
# =========================

@dataclass
class Particle:
    sensor_pos: Tuple[int, int]
    mode_idx: int
    weight: float = 1.0


@dataclass
class BeliefState:
    particles: List[Particle]
    mode_names: List[str]

    def normalize(self) -> None:
        s = sum(p.weight for p in self.particles)
        if s <= 0.0:
            w = 1.0 / max(1, len(self.particles))
            for p in self.particles:
                p.weight = w
            return
        for p in self.particles:
            p.weight /= s

    def effective_sample_size(self) -> float:
        s = sum(p.weight * p.weight for p in self.particles)
        return 0.0 if s <= 0.0 else 1.0 / s

    def mode_posterior(self) -> Dict[int, float]:
        post: Dict[int, float] = defaultdict(float)
        for p in self.particles:
            post[p.mode_idx] += p.weight
        return dict(post)

    def mode_posterior_named(self) -> Dict[str, float]:
        post_i = self.mode_posterior()
        out: Dict[str, float] = {}
        for i, v in post_i.items():
            name = self.mode_names[i] if 0 <= i < len(self.mode_names) else f"mode{i}"
            out[name] = out.get(name, 0.0) + v
        return out


def systematic_resample(particles: List[Particle]) -> List[Particle]:
    n = len(particles)
    if n == 0:
        return []
    positions = [(random.random() + i) / n for i in range(n)]
    cumulative = []
    csum = 0.0
    for p in particles:
        csum += p.weight
        cumulative.append(csum)

    out: List[Particle] = []
    i = 0
    for pos in positions:
        while i < n - 1 and pos > cumulative[i]:
            i += 1
        chosen = particles[i]
        out.append(Particle(sensor_pos=chosen.sensor_pos, mode_idx=chosen.mode_idx, weight=1.0 / n))
    return out


def initialize_belief(
    world: GridWorld,
    sensor_init_candidates: List[Tuple[int, int]],
    mode_prior: List[float],
    mode_names: List[str],
    num_particles: int = 200,
) -> BeliefState:
    if len(mode_prior) != len(mode_names):
        raise ValueError("mode_prior and mode_names must have same length")
    if not sensor_init_candidates:
        raise ValueError("Need at least one sensor init candidate")
    if num_particles <= 0:
        raise ValueError("num_particles must be > 0")

    # normalize prior
    s = sum(mode_prior)
    if s <= 0.0:
        mode_prior = [1.0 / len(mode_prior)] * len(mode_prior)
    else:
        mode_prior = [p / s for p in mode_prior]

    def sample_mode() -> int:
        r = random.random()
        c = 0.0
        for i, p in enumerate(mode_prior):
            c += p
            if r <= c:
                return i
        return len(mode_prior) - 1

    particles: List[Particle] = []
    for _ in range(num_particles):
        sp = random.choice(sensor_init_candidates)
        z = sample_mode()
        particles.append(Particle(sensor_pos=sp, mode_idx=z, weight=1.0 / num_particles))

    b = BeliefState(particles=particles, mode_names=mode_names)
    b.normalize()
    return b


@dataclass
class PFStepInfo:
    ess_before: float
    ess_after: float
    resampled: bool


def belief_predict_update(
    belief: BeliefState,
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    obs_model: ObservationModel,
    obs: Tuple[int, int],
    t: int,
    *,
    motion_noise: float = 0.10,
    resample_threshold: float = 0.5,
    log: Optional[Logger] = None,
) -> PFStepInfo:
    if not belief.particles:
        raise ValueError("belief has no particles")

    belief.normalize()
    ess0 = belief.effective_sample_size()

    # Predict + weight update
    for p in belief.particles:
        mi = p.mode_idx
        if 0 <= mi < len(sensor_policies):
            aS = sensor_policies[mi].action(world, p.sensor_pos, robot_pos, t)
        else:
            aS = "S"

        predicted = world.step(p.sensor_pos, aS)
        if random.random() < motion_noise:
            predicted = world.step(predicted, random.choice(ACTIONS))

        p.sensor_pos = predicted
        p.weight *= obs_model.likelihood(obs, p.sensor_pos)

    belief.normalize()
    ess1 = belief.effective_sample_size()

    # Sanity checks
    N = len(belief.particles)
    soft_assert(0.0 <= ess1 <= N + 1e-6, f"ESS out of bounds: {ess1} (N={N})", log=log, t=t)

    resampled = False
    if ess1 < resample_threshold * N:
        belief.particles = systematic_resample(belief.particles)
        belief.normalize()
        resampled = True

    # Controlled prints
    if log is not None:
        if log._should_print(t, log.cfg.pf_every_t, lvl=1) or resampled:
            mp = belief.mode_posterior_named()
            log.log(
                f"[PF] t={t} obs={obs} ESS {ess0:.1f}->{ess1:.1f} resample={resampled} | modes: {topk_str(mp, k=3)}",
                t=t,
                every=log.cfg.pf_every_t,
                lvl=1,
                force=resampled,
            )
            if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
                log.log(f"[PF][detail] full mode posterior: {topk_str(mp, k=10)}", t=t, every=None, lvl=2)

    return PFStepInfo(ess_before=ess0, ess_after=ess1, resampled=resampled)


# =========================
# A* Search with Risk Costs
# =========================

@dataclass
class AStarResult:
    path: List[Tuple[int, int]]
    cost: float
    expanded: int


def astar_path(
    world: GridWorld,
    start: Tuple[int, int],
    goal: Tuple[int, int],
    step_cost: Callable[[Tuple[int, int]], float],
    heuristic: Callable[[Tuple[int, int]], float],
    *,
    max_expansions: int = 50_000,
) -> Optional[AStarResult]:
    if not world.is_free(start) or not world.is_free(goal):
        return None

    frontier: List[Tuple[float, float, Tuple[int, int]]] = []
    heapq.heappush(frontier, (heuristic(start), 0.0, start))

    came_from: Dict[Tuple[int, int], Tuple[int, int]] = {}
    gscore: Dict[Tuple[int, int], float] = {start: 0.0}

    expanded = 0

    # A* with reopen: skip stale pops
    while frontier and expanded < max_expansions:
        f, g, cur = heapq.heappop(frontier)
        if g > gscore.get(cur, float("inf")) + 1e-12:
            continue
        expanded += 1

        if cur == goal:
            path = [cur]
            while cur in came_from:
                cur = came_from[cur]
                path.append(cur)
            path.reverse()
            return AStarResult(path=path, cost=gscore[goal], expanded=expanded)

        for (nxt, _) in world.neighbors(cur):
            if not world.is_free(nxt):
                continue
            sc = step_cost(nxt)
            if not math.isfinite(sc) or sc < 0:
                continue
            ng = gscore[cur] + sc
            if ng + 1e-12 < gscore.get(nxt, float("inf")):
                gscore[nxt] = ng
                came_from[nxt] = cur
                heapq.heappush(frontier, (ng + heuristic(nxt), ng, nxt))

    return None


# =========================
# Multimodal Path Library
# =========================

@dataclass
class PathMode:
    path: List[Tuple[int, int]]
    total_cost: float
    score: float
    hypothesis_sensor_pos: Tuple[int, int]
    hypothesis_mode_idx: int
    hypothesis_weight: float
    debug_info: Dict[str, Any] = field(default_factory=dict)


def build_cost_map_from_hypothesis(
    world: GridWorld,
    detection: "DetectionModel",
    sensor_pos: Tuple[int, int],
    *,
    base_step: float,
    w_detect: float,
    randomize: bool,
    rand_scale: float,
) -> Callable[[Tuple[int, int]], float]:
    def cost_fn(s: Tuple[int, int]) -> float:
        if not world.is_free(s):
            return float("inf")
        p_det = detection.p_detect(robot_pos=s, sensor_pos=sensor_pos)
        c = base_step + w_detect * p_det
        if randomize:
            c *= (1.0 + rand_scale * (random.random() - 0.5))
            c = max(1e-6, c)
        return c

    return cost_fn


def cluster_paths_by_signature(paths: List[List[Tuple[int, int]]], *, max_modes: int) -> List[int]:
    sig_to_cluster: Dict[str, int] = {}
    assignments: List[int] = []

    def signature(path: List[Tuple[int, int]]) -> str:
        if len(path) < 2:
            return "EMPTY"
        dirs = []
        for i in range(1, len(path)):
            dr = path[i][0] - path[i - 1][0]
            dc = path[i][1] - path[i - 1][1]
            dirs.append((dr, dc))

        comp = []
        for d in dirs:
            if not comp or comp[-1] != d:
                comp.append(d)

        m = {(-1, 0): "U", (1, 0): "D", (0, -1): "L", (0, 1): "R", (0, 0): "S"}
        return "".join(m.get(d, "?") for d in comp)[:120]

    for p in paths:
        sig = signature(p)

        if sig in sig_to_cluster:
            cid = sig_to_cluster[sig]
        elif len(sig_to_cluster) < max_modes:
            cid = len(sig_to_cluster)
            sig_to_cluster[sig] = cid
        else:
            cid = stable_bucket(sig, len(sig_to_cluster))

        assignments.append(cid)

    return assignments


@dataclass
class ModeLibraryInfo:
    num_candidates: int
    num_clusters: int


def unique_weighted_sample(particles: List[Particle], weights: List[float], k: int) -> List[Particle]:
    """Approximate weighted sampling without replacement by oversampling then uniquing."""
    if not particles:
        return []
    if k <= 0:
        return []
    oversample = max(k, 3 * k)
    picks = random.choices(particles, weights=weights, k=oversample)
    seen = set()
    out: List[Particle] = []
    for p in picks:
        key = (p.sensor_pos, p.mode_idx)
        if key in seen:
            continue
        seen.add(key)
        out.append(p)
        if len(out) >= k:
            break
    # If we still don't have k, allow duplicates as a fallback
    while len(out) < k:
        out.append(random.choices(particles, weights=weights, k=1)[0])
    return out


def build_multimodal_path_library(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    *,
    num_hypotheses: int = 30,
    per_hypothesis_attempts: int = 2,
    base_step_cost: float = 1.0,
    w_detect: float = 8.0,
    rand_scale: float = 0.40,
    max_modes: int = 6,
    plausibility_lambda: float = 3.0,
    t: int = 0,
    log: Optional[Logger] = None,
) -> Tuple[List[PathMode], ModeLibraryInfo]:
    if not belief.particles:
        return [], ModeLibraryInfo(num_candidates=0, num_clusters=0)

    # Sample particles proportional to weight
    weights = [p.weight for p in belief.particles]
    s = sum(weights)
    if s <= 0.0:
        weights = [1.0 / len(weights)] * len(weights)
    else:
        weights = [w / s for w in weights]

    # NEW: more diverse sampling
    sampled_particles = unique_weighted_sample(belief.particles, weights, k=num_hypotheses)

    candidates: List[PathMode] = []

    hyp_print_budget = log.cfg.max_hyp_print if (log and log.cfg.level >= 2 and t <= log.cfg.detail_until_t) else 0

    for hi, hp in enumerate(sampled_particles):
        hyp_sensor = hp.sensor_pos
        hyp_mode = hp.mode_idx
        hyp_w = float(getattr(hp, "weight", 0.0))

        if log is not None and hyp_print_budget > 0:
            log.log(
                f"[Modes][detail] t={t} hyp#{hi+1}/{num_hypotheses} sensor={hyp_sensor} mode={belief.mode_names[hyp_mode] if 0<=hyp_mode<len(belief.mode_names) else hyp_mode} w={hyp_w:.4f}",
                t=t,
                every=None,
                lvl=2,
            )
            hyp_print_budget -= 1

        for attempt in range(per_hypothesis_attempts):
            cost_fn = build_cost_map_from_hypothesis(
                world,
                detection,
                hyp_sensor,
                base_step=base_step_cost,
                w_detect=w_detect,
                randomize=True,
                rand_scale=rand_scale,
            )
            heur = lambda s: float(manhattan(s, goal))

            res = astar_path(world, robot_pos, goal, step_cost=cost_fn, heuristic=heur)
            if res is None:
                continue

            # NEW: plausibility-regularized score
            score = float(res.cost) + plausibility_lambda * (-math.log(max(1e-12, hyp_w)))

            candidates.append(
                PathMode(
                    path=res.path,
                    total_cost=float(res.cost),
                    score=score,
                    hypothesis_sensor_pos=hyp_sensor,
                    hypothesis_mode_idx=hyp_mode,
                    hypothesis_weight=hyp_w,
                    debug_info={"expanded": res.expanded, "attempt": attempt + 1},
                )
            )

    if not candidates:
        if log is not None and log._should_print(t, log.cfg.modes_every_t, lvl=1):
            log.log(f"[Modes] t={t} candidates=0 (no paths found)", t=t, every=log.cfg.modes_every_t, lvl=1)
        return [], ModeLibraryInfo(num_candidates=0, num_clusters=0)

    cluster_ids = cluster_paths_by_signature([c.path for c in candidates], max_modes=max_modes)

    # NEW: pick best by *score* per cluster
    best_by_cluster: Dict[int, PathMode] = {}
    for c, cid in zip(candidates, cluster_ids):
        if cid not in best_by_cluster or c.score < best_by_cluster[cid].score:
            best_by_cluster[cid] = c

    # NEW: sort by score, not raw cost
    library = sorted(best_by_cluster.values(), key=lambda pm: pm.score)[:max_modes]

    if log is not None and log._should_print(t, log.cfg.modes_every_t, lvl=1):
        best = ", ".join(
            f"{belief.mode_names[pm.hypothesis_mode_idx] if 0<=pm.hypothesis_mode_idx<len(belief.mode_names) else pm.hypothesis_mode_idx}:"
            f"w={pm.hypothesis_weight:.3f},cost={pm.total_cost:.1f},score={pm.score:.1f}"
            for pm in library[:min(3, len(library))]
        )
        log.log(
            f"[Modes] t={t} candidates={len(candidates)} clusters={len(best_by_cluster)} kept={len(library)} | top3: {best}",
            t=t,
            every=log.cfg.modes_every_t,
            lvl=1,
        )

        if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
            for i, pm in enumerate(library):
                a0 = first_action_from_path(world, robot_pos, pm.path)
                mname = belief.mode_names[pm.hypothesis_mode_idx] if 0 <= pm.hypothesis_mode_idx < len(belief.mode_names) else str(pm.hypothesis_mode_idx)
                log.log(
                    f"[ModesKept] t={t} #{i+1} mode={mname} hypS={pm.hypothesis_sensor_pos} a0={a0} w={pm.hypothesis_weight:.4f} cost={pm.total_cost:.2f} score={pm.score:.2f}",
                    t=t,
                    every=None,
                    lvl=2,
                )

    return library, ModeLibraryInfo(num_candidates=len(candidates), num_clusters=len(best_by_cluster))


def first_action_from_path(world: GridWorld, start: Tuple[int, int], path: List[Tuple[int, int]]) -> Action:
    if len(path) < 2:
        return "S"
    nxt = path[1]
    dr = nxt[0] - start[0]
    dc = nxt[1] - start[1]
    for a, (adr, adc) in ACTION_DELTAS.items():
        if (dr, dc) == (adr, adc):
            return a
    return "S"


def build_action_prior_from_library(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    library: List[PathMode],
    *,
    temp: float = 25.0,
) -> Dict[Action, float]:
    """Mixture prior over kept modes: weight = hyp_weight * exp(-cost/temp)."""
    if not library:
        return {a: 1.0 / len(ACTIONS) for a in ACTIONS}

    pri_raw: Dict[Action, float] = defaultdict(float)
    for pm in library:
        a0 = first_action_from_path(world, robot_pos, pm.path)
        w = max(1e-12, pm.hypothesis_weight)
        w *= math.exp(-pm.total_cost / max(1e-6, temp))
        pri_raw[a0] += w

    return normalize_probs_dict(pri_raw, ACTIONS)


def prior_diagnostics(prior: Dict[Action, float]) -> str:
    ent = 0.0
    for a in ACTIONS:
        p = float(prior.get(a, 0.0))
        if p > 1e-12:
            ent -= p * math.log(p)
    return f"H={ent:.3f} top={topk_str(prior, k=3)}"


# =========================
# GenBR-lite: Belief-space MCTS with PUCT
# =========================

@dataclass
class MCTSNode:
    robot_pos: Tuple[int, int]
    depth: int

    N: int = 0
    N_a: Dict[Action, int] = field(default_factory=lambda: {a: 0 for a in ACTIONS})
    W_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})
    Q_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})

    P_a: Dict[Action, float] = field(default_factory=lambda: {a: 1.0 / len(ACTIONS) for a in ACTIONS})

    children: Dict[Action, "MCTSNode"] = field(default_factory=dict)


def select_action_puct(node: MCTSNode, *, c_puct: float) -> Action:
    """PUCT: argmax_a Q + c*P*sqrt(N)/(1+N_a). Uses node.P_a."""
    sqrtN = math.sqrt(max(1, node.N))

    best_a: Optional[Action] = None
    best_score = -1e18

    for a in ACTIONS:
        q = node.Q_a[a]
        u = c_puct * node.P_a[a] * sqrtN / (1.0 + node.N_a[a])
        score = q + u + random.uniform(-1e-9, 1e-9)
        if score > best_score:
            best_score = score
            best_a = a

    assert best_a is not None
    return best_a


def backprop_path(path: List[Tuple[MCTSNode, Action]], value: float) -> None:
    for node, a in reversed(path):
        node.N += 1
        node.N_a[a] += 1
        node.W_a[a] += value
        node.Q_a[a] = node.W_a[a] / node.N_a[a]


def choose_final_action(root: MCTSNode) -> Action:
    maxN = max(root.N_a.values())
    cand = [a for a in ACTIONS if root.N_a[a] == maxN]
    if len(cand) == 1:
        return cand[0]

    maxQ = max(root.Q_a[a] for a in cand)
    cand2 = [a for a in cand if abs(root.Q_a[a] - maxQ) < 1e-12]
    if len(cand2) == 1:
        return cand2[0]

    maxP = max(root.P_a[a] for a in cand2)
    cand3 = [a for a in cand2 if abs(root.P_a[a] - maxP) < 1e-12]
    return random.choice(cand3)


def rollout_value(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    *,
    max_steps: int,
    w_step: float,
    w_detect: float,
) -> float:
    total_cost = 0.0
    cur = robot_pos
    for _ in range(max_steps):
        if cur == goal:
            break
        a = deterministic_greedy_step_toward(world, cur, goal)
        cur = world.step(cur, a)
        total_cost += w_step
        total_cost += w_detect * detection.p_detect(cur, sensor_pos)
    if cur == goal:
        total_cost -= 20.0
    return -total_cost


def simulate_sensor_step(
    world: GridWorld,
    sensor_pos: Tuple[int, int],
    robot_pos: Tuple[int, int],
    t: int,
    mode_idx: int,
    policies: List[SensorModePolicy],
) -> Tuple[int, int]:
    if 0 <= mode_idx < len(policies):
        aS = policies[mode_idx].action(world, sensor_pos, robot_pos, t)
        return world.step(sensor_pos, aS)
    return sensor_pos


def mcts_search_action(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    detection: DetectionModel,
    action_prior: Dict[Action, float],
    *,
    t: int,
    num_sims: int = 250,
    max_depth: int = 10,
    c_puct: float = 1.4,
    w_step: float = 1.0,
    w_detect: float = 7.0,
    gamma: float = 0.95,
    log: Optional[Logger] = None,
) -> Action:
    prior = normalize_probs_dict(action_prior, ACTIONS, eps=0.10)

    root = MCTSNode(robot_pos=robot_pos, depth=0)
    root.P_a = prior

    # Prepare particle sampling distribution
    particles = belief.particles if belief.particles else [Particle(sensor_pos=robot_pos, mode_idx=0, weight=1.0)]
    weights = [float(getattr(p, "weight", 1.0)) for p in particles]
    s = sum(weights)
    if s <= 0.0:
        weights = [1.0 / len(weights)] * len(weights)
    else:
        weights = [w / s for w in weights]

    if log is not None and (log._should_print(t, log.cfg.mcts_every_t, lvl=1)):
        log.log(f"[MCTS] t={t} sims={num_sims} depth={max_depth} prior: {prior_diagnostics(prior)}", t=t, every=log.cfg.mcts_every_t, lvl=1)

    for sim in range(num_sims):
        hp = random.choices(particles, weights=weights, k=1)[0]
        sim_sensor = hp.sensor_pos
        sim_mode = hp.mode_idx

        node = root
        cur_robot = robot_pos
        cur_sensor = sim_sensor

        path: List[Tuple[MCTSNode, Action]] = []
        total_return = 0.0
        disc = 1.0

        if cur_robot == goal:
            root.N += 1
            continue

        for depth in range(max_depth):
            if cur_robot == goal:
                total_return += disc * 50.0
                break

            a = select_action_puct(node, c_puct=c_puct)
            path.append((node, a))

            nxt_robot = world.step(cur_robot, a)
            nxt_sensor = simulate_sensor_step(world, cur_sensor, nxt_robot, t + depth, sim_mode, sensor_policies)

            step_cost = w_step + w_detect * detection.p_detect(nxt_robot, nxt_sensor)
            total_return += disc * (-step_cost)
            disc *= gamma

            if a not in node.children:
                child = MCTSNode(robot_pos=nxt_robot, depth=node.depth + 1)
                child.P_a = prior
                node.children[a] = child

                leaf_v = rollout_value(
                    world,
                    nxt_robot,
                    nxt_sensor,
                    goal,
                    detection,
                    max_steps=max_depth - depth - 1,
                    w_step=w_step,
                    w_detect=w_detect,
                )
                total_return += disc * leaf_v
                break

            node = node.children[a]
            cur_robot, cur_sensor = nxt_robot, nxt_sensor

        if path:
            backprop_path(path, total_return)
        else:
            root.N += 1

        if log is not None and log.cfg.level >= 2 and (sim < 2 or ((sim + 1) % log.cfg.mcts_progress_every_sims == 0 and t <= log.cfg.detail_until_t)):
            log.log(f"[MCTS][detail] t={t} sim={sim+1}/{num_sims} root.N={root.N}", t=t, every=None, lvl=2)

    total_edge_visits = sum(root.N_a.values())
    if log is not None and log._should_print(t, log.cfg.mcts_every_t, lvl=1):
        bestN = max(root.N_a, key=lambda a: root.N_a[a])
        log.log(
            f"[MCTS] t={t} root.N={root.N} sum(N_a)={total_edge_visits} best_by_N={bestN}",
            t=t,
            every=log.cfg.mcts_every_t,
            lvl=1,
        )
        if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
            for a in ACTIONS:
                log.log(f"[MCTS][detail] a={a} N={root.N_a[a]:4d} Q={root.Q_a[a]:+8.3f} P={root.P_a[a]:.3f}", t=t, every=None, lvl=2)

    return choose_final_action(root)


# =========================
# Main Simulation
# =========================

@dataclass
class Config:
    seed: int = 7

    H: int = 12
    W: int = 18
    T: int = 40

    num_particles: int = 250

    # PF
    motion_noise: float = 0.15
    resample_threshold: float = 0.55

    # Mode library
    num_hypotheses: int = 25
    per_hypothesis_attempts: int = 2
    max_modes: int = 6
    w_detect_astar: float = 10.0
    rand_scale: float = 0.45

    # NEW: plausibility-vs-cost tradeoff
    plausibility_lambda: float = 3.0

    # NEW: prior temperature
    prior_temp: float = 25.0

    # MCTS
    num_sims: int = 250
    max_depth: int = 10
    c_puct: float = 1.4
    w_step_mcts: float = 1.0
    w_detect_mcts: float = 7.0

    # NEW: force a specific true mode for reproducibility
    true_mode_name: Optional[str] = "chase_robot"  # set None for random

    # Debug
    debug: DebugConfig = field(default_factory=DebugConfig)


def make_demo_world(cfg: Config) -> Tuple[GridWorld, Tuple[int, int], Tuple[int, int], Tuple[int, int]]:
    obs = set()
    for c in range(3, 15):
        obs.add((5, c))
    for r in range(1, 9):
        obs.add((r, 9))
    obs.discard((5, 7))
    obs.discard((6, 9))

    world = GridWorld(H=cfg.H, W=cfg.W, obstacles=obs)
    robot_start = (10, 2)
    goal = (1, 16)
    sensor_start = (2, 2)
    return world, robot_start, sensor_start, goal


def main() -> None:
    cfg = Config()
    seed_all(cfg.seed)

    log = Logger(cfg.debug)

    world, robot_pos, sensor_pos, goal = make_demo_world(cfg)

    sensor_modes: List[SensorModePolicy] = [
        PatrolLoopPolicy(name="patrol_left", loop=[(2, 2), (2, 6), (4, 6), (4, 2)]),
        PatrolLoopPolicy(name="patrol_mid", loop=[(2, 10), (2, 13), (4, 13), (4, 10)]),
        ChasePolicy(name="chase_robot"),
        RandomWalkPolicy(name="random_walk"),
    ]
    mode_names = [m.name for m in sensor_modes]

    # NEW: deterministic true mode option
    if cfg.true_mode_name is None:
        true_mode_idx = random.randrange(len(sensor_modes))
    else:
        try:
            true_mode_idx = mode_names.index(cfg.true_mode_name)
        except ValueError:
            raise ValueError(f"true_mode_name={cfg.true_mode_name!r} not in modes {mode_names}")

    detection = DetectionModel()
    obs_model = ObservationModel(sigma=1.3)

    sensor_init_candidates = [(2, 2), (2, 10), (3, 3), (3, 9), (1, 1)]
    mode_prior = [1.0] * len(sensor_modes)
    belief = initialize_belief(world, sensor_init_candidates, mode_prior, mode_names, cfg.num_particles)

    log.log(f"[INIT] true_mode={mode_names[true_mode_idx]} robot={robot_pos} sensor={sensor_pos} goal={goal}", force=True)
    log.log(f"[INIT] belief modes: {topk_str(belief.mode_posterior_named(), k=4)}", force=True)

    for t in range(cfg.T):
        show_header = (t <= cfg.debug.detail_until_t) or (cfg.debug.enabled and cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)
        if show_header:
            log.log("=" * 80, force=True)
            log.log(f"[TIME] t={t}", force=True)

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.render_every_t > 0 and t % cfg.debug.render_every_t == 0)):
            log.log("[WORLD]", t=t, every=cfg.debug.render_every_t, lvl=1, force=(t <= cfg.debug.detail_until_t))
            world.render(robot=robot_pos, sensor=sensor_pos, goal=goal)

        obs = obs_model.sample_observation(sensor_pos, world)
        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[OBS] obs={obs} (true sensor={sensor_pos})", force=True)

        # PF update
        _ = belief_predict_update(
            belief,
            world,
            robot_pos,
            sensor_modes,
            obs_model,
            obs,
            t,
            motion_noise=cfg.motion_noise,
            resample_threshold=cfg.resample_threshold,
            log=log,
        )

        # Mode library -> prior
        library, _ = build_multimodal_path_library(
            world,
            belief,
            robot_pos,
            goal,
            detection,
            num_hypotheses=cfg.num_hypotheses,
            per_hypothesis_attempts=cfg.per_hypothesis_attempts,
            w_detect=cfg.w_detect_astar,
            rand_scale=cfg.rand_scale,
            max_modes=cfg.max_modes,
            plausibility_lambda=cfg.plausibility_lambda,
            t=t,
            log=log,
        )

        action_prior = build_action_prior_from_library(world, robot_pos, library, temp=cfg.prior_temp)

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[PriorDiag] {prior_diagnostics(action_prior)}", force=True)

        # MCTS choose action
        aR = mcts_search_action(
            world,
            belief,
            robot_pos,
            goal,
            sensor_modes,
            detection,
            action_prior,
            t=t,
            num_sims=cfg.num_sims,
            max_depth=cfg.max_depth,
            c_puct=cfg.c_puct,
            w_step=cfg.w_step_mcts,
            w_detect=cfg.w_detect_mcts,
            log=log,
        )

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[ACT] robot aR={aR}", force=True)

        # Step robot and true sensor
        new_robot = world.step(robot_pos, aR)
        aS_true = sensor_modes[true_mode_idx].action(world, sensor_pos, new_robot, t)

        # NEW: chase sanity check on TRUE sensor if chase mode
        if mode_names[true_mode_idx] == "chase_robot":
            chase_sanity_check(world, sensor_pos, new_robot, aS_true, log=log, t=t, tag="true")

        new_sensor = world.step(sensor_pos, aS_true)

        robot_pos, sensor_pos = new_robot, new_sensor

        # Detection
        p_det = detection.p_detect(robot_pos, sensor_pos)
        detected = (random.random() < p_det)

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[STEP] robot={robot_pos} sensor={sensor_pos} aS={aS_true} p_det={p_det:.3f} detected={detected}", force=True)

        # Terminate
        if robot_pos == goal:
            log.log("[TERMINAL] reached goal ✅", force=True)
            break
        if detected and manhattan(robot_pos, sensor_pos) <= detection.d0:
            log.log("[TERMINAL] caught ❌", force=True)
            break

    log.log("[DONE]", force=True)


if __name__ == "__main__":
    main()


[INIT] true_mode=chase_robot robot=(10, 2) sensor=(2, 2) goal=(1, 16)
[INIT] belief modes: random_walk:0.272 | patrol_left:0.260 | patrol_mid:0.240 | chase_robot:0.228
[TIME] t=0
[WORLD]
..................
.........#......G.
..S......#........
.........#........
.........#........
...####.#######...
..................
.........#........
.........#........
..................
..R...............
..................
[OBS] obs=(4, 3) (true sensor=(2, 2))
[PF] t=0 obs=(4, 3) ESS 250.0->61.1 resample=True | modes: chase_robot:0.508 | random_walk:0.192 | patrol_left:0.152
[Modes] t=0 candidates=50 clusters=6 kept=6 | top3: random_walk:w=0.004,cost=37.1,score=53.7, random_walk:w=0.004,cost=37.2,score=53.8, random_walk:w=0.004,cost=37.7,score=54.3
[PriorDiag] H=0.000 top=R:1.000 | U:0.000 | D:0.000
[MCTS] t=0 sims=250 depth=10 prior: H=0.390 top=R:0.920 | U:0.020 | D:0.020
[MCTS] t=0 root.N=250 sum(N_a)=250 best_by_N=R
[ACT] robot aR=R
[CHASE] true t=0 src=(2, 2) dst=(10, 3) a=D nxt=(3, 2) d:9->8

In [19]:
#!/usr/bin/env python3
"""GenBR-lite + PF Belief + Multimodal Path Library (debug-robust)

This version includes the specific fixes suggested from your logs:
1) **Chase policy sanity + determinism**:
   - Chase now uses a deterministic tie-break (no random sideways moves on ties).
   - We print (and warn on) any chase step that *increases* Manhattan distance when a decreasing move exists.

2) **Mode-library selection uses plausibility + cost** (not cost-only optimism):
   - Each candidate path is scored as:  score = cost + lambda_plaus * (-log(hyp_weight)).
   - Best-per-cluster and final sorting use this score.

3) **Action prior is a mixture over modes** (not single-best-path):
   - Prior weights each kept mode by: hyp_weight * exp(-cost/temp).

4) **Detection is actually terminal**:
   - If detected and within d0, episode terminates as "caught".

What to send me to verify correctness:
- Run with DebugConfig(level=2, detail_until_t=6, summary_every_t=1, render_every_t=1) for ~10 steps.
- Paste the log lines that start with: [CHASE], [CHASE][WARN], [ModesKept], [PriorDiag], [MCTS]

Run:
  python3 this_file.py
"""

from __future__ import annotations

import math
import random
import heapq
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Iterable, Callable, Any, Set
from collections import defaultdict
import hashlib


# =========================
# Debug / Logging utilities
# =========================

@dataclass
class DebugConfig:
    enabled: bool = True
    level: int = 1
    # 0 = silent
    # 1 = periodic summaries
    # 2 = detailed (still gated)

    # Print detail for the first few timesteps
    detail_until_t: int = 2

    # Periodic summary controls
    summary_every_t: int = 5
    render_every_t: int = 10

    # Stage-specific periodic prints
    pf_every_t: int = 5
    modes_every_t: int = 5
    mcts_every_t: int = 5

    # Within MCTS
    mcts_progress_every_sims: int = 50

    # Safety: cap printing of mode hypotheses even at detail level
    max_hyp_print: int = 3


class Logger:
    def __init__(self, cfg: DebugConfig):
        self.cfg = cfg

    def _should_print(self, t: Optional[int], every: Optional[int], lvl: int) -> bool:
        if not self.cfg.enabled:
            return False
        if lvl > self.cfg.level:
            return False
        if t is None:
            return True
        if t <= self.cfg.detail_until_t:
            return True
        if every is None:
            return False
        return (every > 0) and (t % every == 0)

    def log(self, msg: str, *, t: Optional[int] = None, every: Optional[int] = None, lvl: int = 1, force: bool = False) -> None:
        if force or self._should_print(t, every, lvl):
            print(msg)


def soft_assert(cond: bool, msg: str, *, fatal: bool = False, log: Optional[Logger] = None, t: Optional[int] = None) -> None:
    if cond:
        return
    if log is not None:
        log.log(f"[WARN] {msg}", t=t, every=None, lvl=1, force=True)
    if fatal:
        raise AssertionError(msg)


def clamp(x: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, x))


def manhattan(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def seed_all(seed: int) -> None:
    random.seed(seed)


def stable_bucket(sig: str, m: int) -> int:
    """Stable mapping sig -> [0..m-1] independent of Python's randomized hash."""
    if m <= 0:
        return 0
    h = hashlib.md5(sig.encode("utf-8")).hexdigest()
    return int(h[:8], 16) % m


def normalize_probs_dict(d: Dict[Any, float], keys: Iterable[Any], *, eps: float = 0.0) -> Dict[Any, float]:
    """Return a normalized distribution over `keys` from possibly-missing/negative inputs.

    - Clips negatives to 0
    - Adds optional epsilon smoothing to avoid degeneracy
    - Falls back to uniform if total mass is 0
    """
    klist = list(keys)
    out = {k: max(0.0, float(d.get(k, 0.0))) for k in klist}
    s = sum(out.values())

    if s <= 0.0:
        u = 1.0 / max(1, len(klist))
        return {k: u for k in klist}

    out = {k: v / s for k, v in out.items()}

    if eps > 0.0:
        u = 1.0 / max(1, len(klist))
        out = {k: (1.0 - eps) * out[k] + eps * u for k in klist}
        s2 = sum(out.values())
        out = {k: v / s2 for k, v in out.items()}

    return out


def topk_str(d: Dict[Any, float], k: int = 3, fmt: str = "{k}:{v:.3f}") -> str:
    items = sorted(d.items(), key=lambda kv: kv[1], reverse=True)[:k]
    return " | ".join(fmt.format(k=kk, v=vv) for kk, vv in items)


# =========================
# Gridworld Environment
# =========================

Action = str  # 'U','D','L','R','S'

ACTIONS: List[Action] = ["U", "D", "L", "R", "S"]
ACTION_DELTAS: Dict[Action, Tuple[int, int]] = {
    "U": (-1, 0),
    "D": (1, 0),
    "L": (0, -1),
    "R": (0, 1),
    "S": (0, 0),
}


@dataclass
class GridWorld:
    H: int
    W: int
    obstacles: Set[Tuple[int, int]] = field(default_factory=set)

    def in_bounds(self, p: Tuple[int, int]) -> bool:
        r, c = p
        return 0 <= r < self.H and 0 <= c < self.W

    def is_free(self, p: Tuple[int, int]) -> bool:
        return self.in_bounds(p) and (p not in self.obstacles)

    def step(self, p: Tuple[int, int], a: Action) -> Tuple[int, int]:
        if a not in ACTION_DELTAS:
            raise ValueError(f"Unknown action: {a}")
        dr, dc = ACTION_DELTAS[a]
        np = (p[0] + dr, p[1] + dc)
        return np if self.is_free(np) else p

    def neighbors(self, p: Tuple[int, int]) -> List[Tuple[Tuple[int, int], Action]]:
        return [(self.step(p, a), a) for a in ACTIONS]

    def render(self, robot: Tuple[int, int], sensor: Tuple[int, int], goal: Tuple[int, int]) -> None:
        grid = [["." for _ in range(self.W)] for _ in range(self.H)]
        for (r, c) in self.obstacles:
            grid[r][c] = "#"
        rr, rc = robot
        sr, sc = sensor
        gr, gc = goal
        grid[gr][gc] = "G"
        grid[sr][sc] = "S"
        grid[rr][rc] = "R"
        for r in range(self.H):
            print("".join(grid[r]))


# =========================
# Sensor Models (Modes)
# =========================

@dataclass
class SensorModePolicy:
    name: str

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        raise NotImplementedError


def best_actions_toward(world: GridWorld, src: Tuple[int, int], dst: Tuple[int, int]) -> Tuple[List[Action], int]:
    """Return (best_actions, best_dist) minimizing manhattan(next(src,a), dst)."""
    best_actions: List[Action] = []
    best_dist = 10**9
    for a in ACTIONS:
        np = world.step(src, a)
        d = manhattan(np, dst)
        if d < best_dist:
            best_dist = d
            best_actions = [a]
        elif d == best_dist:
            best_actions.append(a)
    return best_actions, best_dist


def deterministic_greedy_step_toward(world: GridWorld, src: Tuple[int, int], dst: Tuple[int, int]) -> Action:
    """Greedy toward dst with deterministic tie-break (easier to debug than random ties)."""
    best_as, _ = best_actions_toward(world, src, dst)
    if not best_as:
        return "S"

    # Preferred axis: move along the larger |delta| if possible
    dr = dst[0] - src[0]
    dc = dst[1] - src[1]

    preferred: List[Action] = []
    if abs(dr) >= abs(dc):
        if dr > 0:
            preferred.append("D")
        elif dr < 0:
            preferred.append("U")
    if abs(dc) > abs(dr):
        if dc > 0:
            preferred.append("R")
        elif dc < 0:
            preferred.append("L")

    # If the preferred move is among best actions, take it
    for a in preferred:
        if a in best_as:
            return a

    # Otherwise deterministic order
    for a in ["U", "D", "L", "R", "S"]:
        if a in best_as:
            return a

    return best_as[0]


def chase_sanity_check(
    world: GridWorld,
    src: Tuple[int, int],
    dst: Tuple[int, int],
    chosen: Action,
    *,
    log: Optional[Logger],
    t: int,
    tag: str,
) -> None:
    """Warn if chosen action increases distance when a non-increasing action exists."""
    d0 = manhattan(src, dst)
    nxt = world.step(src, chosen)
    d1 = manhattan(nxt, dst)

    best_as, best_d = best_actions_toward(world, src, dst)

    if log is not None and log.cfg.enabled and (t <= log.cfg.detail_until_t or (log.cfg.summary_every_t > 0 and t % log.cfg.summary_every_t == 0)):
        log.log(
            f"[CHASE] {tag} t={t} src={src} dst={dst} a={chosen} nxt={nxt} d:{d0}->{d1} best_d={best_d} best={best_as}",
            t=t,
            every=log.cfg.summary_every_t,
            lvl=1,
        )

    # If chosen isn't among best actions, it's suspicious
    if chosen not in best_as:
        if log is not None:
            log.log(
                f"[CHASE][WARN] {tag} t={t} chose {chosen} but best={best_as} (d:{d0}->{d1}, best_d={best_d})",
                t=t,
                every=None,
                lvl=1,
                force=True,
            )


@dataclass
class PatrolLoopPolicy(SensorModePolicy):
    loop: List[Tuple[int, int]] = field(default_factory=list)

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        if not self.loop:
            return "S"
        idx = t % len(self.loop)
        return deterministic_greedy_step_toward(world, sensor_pos, self.loop[idx])


@dataclass
class ChasePolicy(SensorModePolicy):
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        return deterministic_greedy_step_toward(world, sensor_pos, robot_pos)


@dataclass
class RandomWalkPolicy(SensorModePolicy):
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        return random.choice(ACTIONS)


# =========================
# Detection / Observation
# =========================

@dataclass
class DetectionModel:
    d0: int = 2
    p_close: float = 0.9
    p_far: float = 0.05
    decay: float = 0.3

    def p_detect(self, robot_pos: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        dist = manhattan(robot_pos, sensor_pos)
        if dist <= self.d0:
            return self.p_close
        return self.p_far + (self.p_close - self.p_far) * math.exp(-self.decay * (dist - self.d0))


@dataclass
class ObservationModel:
    sigma: float = 1.2

    def sample_observation(self, sensor_pos: Tuple[int, int], world: GridWorld) -> Tuple[int, int]:
        max_jump = 2
        dr = random.randint(-max_jump, max_jump)
        dc = random.randint(-max_jump, max_jump)
        o = (sensor_pos[0] + dr, sensor_pos[1] + dc)
        o = (int(clamp(o[0], 0, world.H - 1)), int(clamp(o[1], 0, world.W - 1)))
        return o

    def likelihood(self, obs: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        d = manhattan(obs, sensor_pos)
        return max(1e-12, math.exp(-d / max(1e-6, self.sigma)))


# =========================
# Particle Filter over (sensor_pos, mode)
# =========================

@dataclass
class Particle:
    sensor_pos: Tuple[int, int]
    mode_idx: int
    weight: float = 1.0


@dataclass
class BeliefState:
    particles: List[Particle]
    mode_names: List[str]

    def normalize(self) -> None:
        s = sum(p.weight for p in self.particles)
        if s <= 0.0:
            w = 1.0 / max(1, len(self.particles))
            for p in self.particles:
                p.weight = w
            return
        for p in self.particles:
            p.weight /= s

    def effective_sample_size(self) -> float:
        s = sum(p.weight * p.weight for p in self.particles)
        return 0.0 if s <= 0.0 else 1.0 / s

    def mode_posterior(self) -> Dict[int, float]:
        post: Dict[int, float] = defaultdict(float)
        for p in self.particles:
            post[p.mode_idx] += p.weight
        return dict(post)

    def mode_posterior_named(self) -> Dict[str, float]:
        post_i = self.mode_posterior()
        out: Dict[str, float] = {}
        for i, v in post_i.items():
            name = self.mode_names[i] if 0 <= i < len(self.mode_names) else f"mode{i}"
            out[name] = out.get(name, 0.0) + v
        return out


def systematic_resample(particles: List[Particle]) -> List[Particle]:
    n = len(particles)
    if n == 0:
        return []
    positions = [(random.random() + i) / n for i in range(n)]
    cumulative = []
    csum = 0.0
    for p in particles:
        csum += p.weight
        cumulative.append(csum)

    out: List[Particle] = []
    i = 0
    for pos in positions:
        while i < n - 1 and pos > cumulative[i]:
            i += 1
        chosen = particles[i]
        out.append(Particle(sensor_pos=chosen.sensor_pos, mode_idx=chosen.mode_idx, weight=1.0 / n))
    return out


def initialize_belief(
    world: GridWorld,
    sensor_init_candidates: List[Tuple[int, int]],
    mode_prior: List[float],
    mode_names: List[str],
    num_particles: int = 200,
) -> BeliefState:
    if len(mode_prior) != len(mode_names):
        raise ValueError("mode_prior and mode_names must have same length")
    if not sensor_init_candidates:
        raise ValueError("Need at least one sensor init candidate")
    if num_particles <= 0:
        raise ValueError("num_particles must be > 0")

    # normalize prior
    s = sum(mode_prior)
    if s <= 0.0:
        mode_prior = [1.0 / len(mode_prior)] * len(mode_prior)
    else:
        mode_prior = [p / s for p in mode_prior]

    def sample_mode() -> int:
        r = random.random()
        c = 0.0
        for i, p in enumerate(mode_prior):
            c += p
            if r <= c:
                return i
        return len(mode_prior) - 1

    particles: List[Particle] = []
    for _ in range(num_particles):
        sp = random.choice(sensor_init_candidates)
        z = sample_mode()
        particles.append(Particle(sensor_pos=sp, mode_idx=z, weight=1.0 / num_particles))

    b = BeliefState(particles=particles, mode_names=mode_names)
    b.normalize()
    return b


@dataclass
class PFStepInfo:
    ess_before: float
    ess_after: float
    resampled: bool


def belief_predict_update(
    belief: BeliefState,
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    obs_model: ObservationModel,
    obs: Tuple[int, int],
    t: int,
    *,
    motion_noise: float = 0.10,
    resample_threshold: float = 0.5,
    log: Optional[Logger] = None,
) -> PFStepInfo:
    if not belief.particles:
        raise ValueError("belief has no particles")

    belief.normalize()
    ess0 = belief.effective_sample_size()

    # Predict + weight update
    for p in belief.particles:
        mi = p.mode_idx
        if 0 <= mi < len(sensor_policies):
            aS = sensor_policies[mi].action(world, p.sensor_pos, robot_pos, t)
        else:
            aS = "S"

        predicted = world.step(p.sensor_pos, aS)
        if random.random() < motion_noise:
            predicted = world.step(predicted, random.choice(ACTIONS))

        p.sensor_pos = predicted
        p.weight *= obs_model.likelihood(obs, p.sensor_pos)

    belief.normalize()
    ess1 = belief.effective_sample_size()

    # Sanity checks
    N = len(belief.particles)
    soft_assert(0.0 <= ess1 <= N + 1e-6, f"ESS out of bounds: {ess1} (N={N})", log=log, t=t)

    resampled = False
    if ess1 < resample_threshold * N:
        belief.particles = systematic_resample(belief.particles)
        belief.normalize()
        resampled = True

    # Controlled prints
    if log is not None:
        if log._should_print(t, log.cfg.pf_every_t, lvl=1) or resampled:
            mp = belief.mode_posterior_named()
            log.log(
                f"[PF] t={t} obs={obs} ESS {ess0:.1f}->{ess1:.1f} resample={resampled} | modes: {topk_str(mp, k=3)}",
                t=t,
                every=log.cfg.pf_every_t,
                lvl=1,
                force=resampled,
            )
            if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
                log.log(f"[PF][detail] full mode posterior: {topk_str(mp, k=10)}", t=t, every=None, lvl=2)

    return PFStepInfo(ess_before=ess0, ess_after=ess1, resampled=resampled)


# =========================
# A* Search with Risk Costs
# =========================

@dataclass
class AStarResult:
    path: List[Tuple[int, int]]
    cost: float
    expanded: int


def astar_path(
    world: GridWorld,
    start: Tuple[int, int],
    goal: Tuple[int, int],
    step_cost: Callable[[Tuple[int, int]], float],
    heuristic: Callable[[Tuple[int, int]], float],
    *,
    max_expansions: int = 50_000,
) -> Optional[AStarResult]:
    if not world.is_free(start) or not world.is_free(goal):
        return None

    frontier: List[Tuple[float, float, Tuple[int, int]]] = []
    heapq.heappush(frontier, (heuristic(start), 0.0, start))

    came_from: Dict[Tuple[int, int], Tuple[int, int]] = {}
    gscore: Dict[Tuple[int, int], float] = {start: 0.0}

    expanded = 0

    # A* with reopen: skip stale pops
    while frontier and expanded < max_expansions:
        f, g, cur = heapq.heappop(frontier)
        if g > gscore.get(cur, float("inf")) + 1e-12:
            continue
        expanded += 1

        if cur == goal:
            path = [cur]
            while cur in came_from:
                cur = came_from[cur]
                path.append(cur)
            path.reverse()
            return AStarResult(path=path, cost=gscore[goal], expanded=expanded)

        for (nxt, _) in world.neighbors(cur):
            if not world.is_free(nxt):
                continue
            sc = step_cost(nxt)
            if not math.isfinite(sc) or sc < 0:
                continue
            ng = gscore[cur] + sc
            if ng + 1e-12 < gscore.get(nxt, float("inf")):
                gscore[nxt] = ng
                came_from[nxt] = cur
                heapq.heappush(frontier, (ng + heuristic(nxt), ng, nxt))

    return None


# =========================
# Multimodal Path Library
# =========================

@dataclass
class PathMode:
    path: List[Tuple[int, int]]
    total_cost: float
    score: float
    hypothesis_sensor_pos: Tuple[int, int]
    hypothesis_mode_idx: int
    hypothesis_weight: float
    debug_info: Dict[str, Any] = field(default_factory=dict)


def build_cost_map_from_hypothesis(
    world: GridWorld,
    detection: "DetectionModel",
    sensor_pos: Tuple[int, int],
    *,
    base_step: float,
    w_detect: float,
    randomize: bool,
    rand_scale: float,
) -> Callable[[Tuple[int, int]], float]:
    def cost_fn(s: Tuple[int, int]) -> float:
        if not world.is_free(s):
            return float("inf")
        p_det = detection.p_detect(robot_pos=s, sensor_pos=sensor_pos)
        c = base_step + w_detect * p_det
        if randomize:
            c *= (1.0 + rand_scale * (random.random() - 0.5))
            c = max(1e-6, c)
        return c

    return cost_fn


def cluster_paths_by_signature(paths: List[List[Tuple[int, int]]], *, max_modes: int) -> List[int]:
    sig_to_cluster: Dict[str, int] = {}
    assignments: List[int] = []

    def signature(path: List[Tuple[int, int]]) -> str:
        if len(path) < 2:
            return "EMPTY"
        dirs = []
        for i in range(1, len(path)):
            dr = path[i][0] - path[i - 1][0]
            dc = path[i][1] - path[i - 1][1]
            dirs.append((dr, dc))

        comp = []
        for d in dirs:
            if not comp or comp[-1] != d:
                comp.append(d)

        m = {(-1, 0): "U", (1, 0): "D", (0, -1): "L", (0, 1): "R", (0, 0): "S"}
        return "".join(m.get(d, "?") for d in comp)[:120]

    for p in paths:
        sig = signature(p)

        if sig in sig_to_cluster:
            cid = sig_to_cluster[sig]
        elif len(sig_to_cluster) < max_modes:
            cid = len(sig_to_cluster)
            sig_to_cluster[sig] = cid
        else:
            cid = stable_bucket(sig, len(sig_to_cluster))

        assignments.append(cid)

    return assignments


@dataclass
class ModeLibraryInfo:
    num_candidates: int
    num_clusters: int


def unique_weighted_sample(particles: List[Particle], weights: List[float], k: int) -> List[Particle]:
    """Approximate weighted sampling without replacement by oversampling then uniquing."""
    if not particles:
        return []
    if k <= 0:
        return []
    oversample = max(k, 3 * k)
    picks = random.choices(particles, weights=weights, k=oversample)
    seen = set()
    out: List[Particle] = []
    for p in picks:
        key = (p.sensor_pos, p.mode_idx)
        if key in seen:
            continue
        seen.add(key)
        out.append(p)
        if len(out) >= k:
            break
    # If we still don't have k, allow duplicates as a fallback
    while len(out) < k:
        out.append(random.choices(particles, weights=weights, k=1)[0])
    return out


def build_multimodal_path_library(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    *,
    num_hypotheses: int = 30,
    per_hypothesis_attempts: int = 2,
    base_step_cost: float = 1.0,
    w_detect: float = 8.0,
    rand_scale: float = 0.40,
    max_modes: int = 6,
    plausibility_lambda: float = 3.0,
    t: int = 0,
    log: Optional[Logger] = None,
) -> Tuple[List[PathMode], ModeLibraryInfo]:
    if not belief.particles:
        return [], ModeLibraryInfo(num_candidates=0, num_clusters=0)

    # Sample particles proportional to weight
    weights = [p.weight for p in belief.particles]
    s = sum(weights)
    if s <= 0.0:
        weights = [1.0 / len(weights)] * len(weights)
    else:
        weights = [w / s for w in weights]

    # NEW: more diverse sampling
    sampled_particles = unique_weighted_sample(belief.particles, weights, k=num_hypotheses)

    candidates: List[PathMode] = []

    hyp_print_budget = log.cfg.max_hyp_print if (log and log.cfg.level >= 2 and t <= log.cfg.detail_until_t) else 0

    for hi, hp in enumerate(sampled_particles):
        hyp_sensor = hp.sensor_pos
        hyp_mode = hp.mode_idx
        hyp_w = float(getattr(hp, "weight", 0.0))

        if log is not None and hyp_print_budget > 0:
            log.log(
                f"[Modes][detail] t={t} hyp#{hi+1}/{num_hypotheses} sensor={hyp_sensor} mode={belief.mode_names[hyp_mode] if 0<=hyp_mode<len(belief.mode_names) else hyp_mode} w={hyp_w:.4f}",
                t=t,
                every=None,
                lvl=2,
            )
            hyp_print_budget -= 1

        for attempt in range(per_hypothesis_attempts):
            cost_fn = build_cost_map_from_hypothesis(
                world,
                detection,
                hyp_sensor,
                base_step=base_step_cost,
                w_detect=w_detect,
                randomize=True,
                rand_scale=rand_scale,
            )
            heur = lambda s: float(manhattan(s, goal))

            res = astar_path(world, robot_pos, goal, step_cost=cost_fn, heuristic=heur)
            if res is None:
                continue

            # NEW: plausibility-regularized score
            score = float(res.cost) + plausibility_lambda * (-math.log(max(1e-12, hyp_w)))

            candidates.append(
                PathMode(
                    path=res.path,
                    total_cost=float(res.cost),
                    score=score,
                    hypothesis_sensor_pos=hyp_sensor,
                    hypothesis_mode_idx=hyp_mode,
                    hypothesis_weight=hyp_w,
                    debug_info={"expanded": res.expanded, "attempt": attempt + 1},
                )
            )

    if not candidates:
        if log is not None and log._should_print(t, log.cfg.modes_every_t, lvl=1):
            log.log(f"[Modes] t={t} candidates=0 (no paths found)", t=t, every=log.cfg.modes_every_t, lvl=1)
        return [], ModeLibraryInfo(num_candidates=0, num_clusters=0)

    cluster_ids = cluster_paths_by_signature([c.path for c in candidates], max_modes=max_modes)

    # NEW: pick best by *score* per cluster
    best_by_cluster: Dict[int, PathMode] = {}
    for c, cid in zip(candidates, cluster_ids):
        if cid not in best_by_cluster or c.score < best_by_cluster[cid].score:
            best_by_cluster[cid] = c

    # NEW: sort by score, not raw cost
    library = sorted(best_by_cluster.values(), key=lambda pm: pm.score)[:max_modes]

    if log is not None and log._should_print(t, log.cfg.modes_every_t, lvl=1):
        best = ", ".join(
            f"{belief.mode_names[pm.hypothesis_mode_idx] if 0<=pm.hypothesis_mode_idx<len(belief.mode_names) else pm.hypothesis_mode_idx}:"
            f"w={pm.hypothesis_weight:.3f},cost={pm.total_cost:.1f},score={pm.score:.1f}"
            for pm in library[:min(3, len(library))]
        )
        log.log(
            f"[Modes] t={t} candidates={len(candidates)} clusters={len(best_by_cluster)} kept={len(library)} | top3: {best}",
            t=t,
            every=log.cfg.modes_every_t,
            lvl=1,
        )

        if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
            for i, pm in enumerate(library):
                a0 = first_action_from_path(world, robot_pos, pm.path)
                mname = belief.mode_names[pm.hypothesis_mode_idx] if 0 <= pm.hypothesis_mode_idx < len(belief.mode_names) else str(pm.hypothesis_mode_idx)
                log.log(
                    f"[ModesKept] t={t} #{i+1} mode={mname} hypS={pm.hypothesis_sensor_pos} a0={a0} w={pm.hypothesis_weight:.4f} cost={pm.total_cost:.2f} score={pm.score:.2f}",
                    t=t,
                    every=None,
                    lvl=2,
                )

    return library, ModeLibraryInfo(num_candidates=len(candidates), num_clusters=len(best_by_cluster))


def first_action_from_path(world: GridWorld, start: Tuple[int, int], path: List[Tuple[int, int]]) -> Action:
    if len(path) < 2:
        return "S"
    nxt = path[1]
    dr = nxt[0] - start[0]
    dc = nxt[1] - start[1]
    for a, (adr, adc) in ACTION_DELTAS.items():
        if (dr, dc) == (adr, adc):
            return a
    return "S"


def build_action_prior_from_library(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    library: List[PathMode],
    *,
    temp: float = 25.0,
) -> Dict[Action, float]:
    """Mixture prior over kept modes: weight = hyp_weight * exp(-cost/temp)."""
    if not library:
        return {a: 1.0 / len(ACTIONS) for a in ACTIONS}

    pri_raw: Dict[Action, float] = defaultdict(float)
    for pm in library:
        a0 = first_action_from_path(world, robot_pos, pm.path)
        w = max(1e-12, pm.hypothesis_weight)
        w *= math.exp(-pm.total_cost / max(1e-6, temp))
        pri_raw[a0] += w

    return normalize_probs_dict(pri_raw, ACTIONS)


def prior_entropy(prior: Dict[Action, float]) -> float:
    ent = 0.0
    for a in ACTIONS:
        p = float(prior.get(a, 0.0))
        if p > 1e-12:
            ent -= p * math.log(p)
    return ent


def prior_diagnostics(prior: Dict[Action, float]) -> str:
    ent = prior_entropy(prior)
    return f"H={ent:.3f} top={topk_str(prior, k=3)}"


def prior_check_and_log(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    library: List[PathMode],
    raw_prior: Dict[Action, float],
    smoothed_prior: Dict[Action, float],
    *,
    temp: float,
    eps: float,
    mode_names: List[str],
    warn_maxp: float,
    warn_minH: float,
    t: int,
    log: Optional[Logger],
) -> None:
    """Warn when the library induces a near-delta prior.

    When it triggers, we print *why* (first-action mass + kept-mode breakdown), so you can
    tell whether it's a real collapse or just a legitimately one-sided situation.
    """
    if log is None or not log.cfg.enabled:
        return
    if not library:
        return

    kept_a0: List[Action] = [first_action_from_path(world, robot_pos, pm.path) for pm in library]
    distinct_a0 = sorted(set(kept_a0))

    rawH = prior_entropy(raw_prior)
    rawMax = max(float(raw_prior.get(a, 0.0)) for a in ACTIONS)
    smH = prior_entropy(smoothed_prior)
    smMax = max(float(smoothed_prior.get(a, 0.0)) for a in ACTIONS)

    warn = False
    if len(library) >= 2 and len(distinct_a0) <= 1:
        warn = True
    if rawMax >= warn_maxp:
        warn = True
    if rawH <= warn_minH:
        warn = True

    if not warn:
        return

    log.log(
        f"[PriorCheck][WARN] t={t} kept={len(library)} distinct_a0={len(distinct_a0)}({','.join(distinct_a0) if distinct_a0 else '-'}) "
        f"rawH={rawH:.3f} rawMax={rawMax:.3f} | eps={eps:.2f} smH={smH:.3f} smMax={smMax:.3f}",
        t=t,
        every=None,
        lvl=1,
        force=True,
    )
    log.log(f"[PriorDiagRaw] {prior_diagnostics(raw_prior)}", t=t, every=None, lvl=1, force=True)
    log.log(f"[PriorDiagSm]  eps={eps:.2f} {prior_diagnostics(smoothed_prior)}", t=t, every=None, lvl=1, force=True)

    mix_by_a: Dict[Action, float] = defaultdict(float)
    for pm in library:
        a0 = first_action_from_path(world, robot_pos, pm.path)
        mix_w = max(1e-12, pm.hypothesis_weight) * math.exp(-pm.total_cost / max(1e-6, temp))
        mix_by_a[a0] += mix_w

    mix_norm = normalize_probs_dict(mix_by_a, ACTIONS)
    log.log(f"[PriorCheck] mix_mass(raw): {topk_str(mix_by_a, k=5, fmt='{k}:{v:.3g}')}", t=t, every=None, lvl=1, force=True)
    log.log(f"[PriorCheck] mix_mass(nrm): {topk_str(mix_norm, k=5)}", t=t, every=None, lvl=1, force=True)

    for i, pm in enumerate(library, start=1):
        a0 = first_action_from_path(world, robot_pos, pm.path)
        mname = mode_names[pm.hypothesis_mode_idx] if 0 <= pm.hypothesis_mode_idx < len(mode_names) else str(pm.hypothesis_mode_idx)
        mix_w = max(1e-12, pm.hypothesis_weight) * math.exp(-pm.total_cost / max(1e-6, temp))
        log.log(
            f"[PriorCheck][Kept] t={t} #{i} a0={a0} mix_w={mix_w:.3g} mode={mname} hypS={pm.hypothesis_sensor_pos} "
            f"hyp_w={pm.hypothesis_weight:.4f} cost={pm.total_cost:.2f} score={pm.score:.2f}",
            t=t,
            every=None,
            lvl=1,
            force=True,
        )


# =========================
# GenBR-lite: Belief-space MCTS with PUCT
# =========================

@dataclass
class MCTSNode:
    robot_pos: Tuple[int, int]
    depth: int

    N: int = 0
    N_a: Dict[Action, int] = field(default_factory=lambda: {a: 0 for a in ACTIONS})
    W_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})
    Q_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})

    P_a: Dict[Action, float] = field(default_factory=lambda: {a: 1.0 / len(ACTIONS) for a in ACTIONS})

    children: Dict[Action, "MCTSNode"] = field(default_factory=dict)


def select_action_puct(node: MCTSNode, *, c_puct: float) -> Action:
    """PUCT: argmax_a Q + c*P*sqrt(N)/(1+N_a). Uses node.P_a."""
    sqrtN = math.sqrt(max(1, node.N))

    best_a: Optional[Action] = None
    best_score = -1e18

    for a in ACTIONS:
        q = node.Q_a[a]
        u = c_puct * node.P_a[a] * sqrtN / (1.0 + node.N_a[a])
        score = q + u + random.uniform(-1e-9, 1e-9)
        if score > best_score:
            best_score = score
            best_a = a

    assert best_a is not None
    return best_a


def backprop_path(path: List[Tuple[MCTSNode, Action]], value: float) -> None:
    for node, a in reversed(path):
        node.N += 1
        node.N_a[a] += 1
        node.W_a[a] += value
        node.Q_a[a] = node.W_a[a] / node.N_a[a]


def choose_final_action(root: MCTSNode) -> Action:
    maxN = max(root.N_a.values())
    cand = [a for a in ACTIONS if root.N_a[a] == maxN]
    if len(cand) == 1:
        return cand[0]

    maxQ = max(root.Q_a[a] for a in cand)
    cand2 = [a for a in cand if abs(root.Q_a[a] - maxQ) < 1e-12]
    if len(cand2) == 1:
        return cand2[0]

    maxP = max(root.P_a[a] for a in cand2)
    cand3 = [a for a in cand2 if abs(root.P_a[a] - maxP) < 1e-12]
    return random.choice(cand3)


def rollout_value(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    *,
    max_steps: int,
    w_step: float,
    w_detect: float,
) -> float:
    total_cost = 0.0
    cur = robot_pos
    for _ in range(max_steps):
        if cur == goal:
            break
        a = deterministic_greedy_step_toward(world, cur, goal)
        cur = world.step(cur, a)
        total_cost += w_step
        total_cost += w_detect * detection.p_detect(cur, sensor_pos)
    if cur == goal:
        total_cost -= 20.0
    return -total_cost


def simulate_sensor_step(
    world: GridWorld,
    sensor_pos: Tuple[int, int],
    robot_pos: Tuple[int, int],
    t: int,
    mode_idx: int,
    policies: List[SensorModePolicy],
) -> Tuple[int, int]:
    if 0 <= mode_idx < len(policies):
        aS = policies[mode_idx].action(world, sensor_pos, robot_pos, t)
        return world.step(sensor_pos, aS)
    return sensor_pos


def mcts_search_action(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    detection: DetectionModel,
    action_prior: Dict[Action, float],
    *,
    t: int,
    num_sims: int = 250,
    max_depth: int = 10,
    c_puct: float = 1.4,
    w_step: float = 1.0,
    w_detect: float = 7.0,
    gamma: float = 0.95,
    prior_eps: float = 0.10,
    log: Optional[Logger] = None,
) -> Action:
    prior = normalize_probs_dict(action_prior, ACTIONS, eps=prior_eps)

    root = MCTSNode(robot_pos=robot_pos, depth=0)
    root.P_a = prior

    # Prepare particle sampling distribution
    particles = belief.particles if belief.particles else [Particle(sensor_pos=robot_pos, mode_idx=0, weight=1.0)]
    weights = [float(getattr(p, "weight", 1.0)) for p in particles]
    s = sum(weights)
    if s <= 0.0:
        weights = [1.0 / len(weights)] * len(weights)
    else:
        weights = [w / s for w in weights]

    if log is not None and (log._should_print(t, log.cfg.mcts_every_t, lvl=1)):
        log.log(f"[MCTS] t={t} sims={num_sims} depth={max_depth} prior: {prior_diagnostics(prior)}", t=t, every=log.cfg.mcts_every_t, lvl=1)

    for sim in range(num_sims):
        hp = random.choices(particles, weights=weights, k=1)[0]
        sim_sensor = hp.sensor_pos
        sim_mode = hp.mode_idx

        node = root
        cur_robot = robot_pos
        cur_sensor = sim_sensor

        path: List[Tuple[MCTSNode, Action]] = []
        total_return = 0.0
        disc = 1.0

        if cur_robot == goal:
            root.N += 1
            continue

        for depth in range(max_depth):
            if cur_robot == goal:
                total_return += disc * 50.0
                break

            a = select_action_puct(node, c_puct=c_puct)
            path.append((node, a))

            nxt_robot = world.step(cur_robot, a)
            nxt_sensor = simulate_sensor_step(world, cur_sensor, nxt_robot, t + depth, sim_mode, sensor_policies)

            step_cost = w_step + w_detect * detection.p_detect(nxt_robot, nxt_sensor)
            total_return += disc * (-step_cost)
            disc *= gamma

            if a not in node.children:
                child = MCTSNode(robot_pos=nxt_robot, depth=node.depth + 1)
                child.P_a = prior
                node.children[a] = child

                leaf_v = rollout_value(
                    world,
                    nxt_robot,
                    nxt_sensor,
                    goal,
                    detection,
                    max_steps=max_depth - depth - 1,
                    w_step=w_step,
                    w_detect=w_detect,
                )
                total_return += disc * leaf_v
                break

            node = node.children[a]
            cur_robot, cur_sensor = nxt_robot, nxt_sensor

        if path:
            backprop_path(path, total_return)
        else:
            root.N += 1

        if log is not None and log.cfg.level >= 2 and (sim < 2 or ((sim + 1) % log.cfg.mcts_progress_every_sims == 0 and t <= log.cfg.detail_until_t)):
            log.log(f"[MCTS][detail] t={t} sim={sim+1}/{num_sims} root.N={root.N}", t=t, every=None, lvl=2)

    total_edge_visits = sum(root.N_a.values())
    if log is not None and log._should_print(t, log.cfg.mcts_every_t, lvl=1):
        bestN = max(root.N_a, key=lambda a: root.N_a[a])
        log.log(
            f"[MCTS] t={t} root.N={root.N} sum(N_a)={total_edge_visits} best_by_N={bestN}",
            t=t,
            every=log.cfg.mcts_every_t,
            lvl=1,
        )
        if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
            for a in ACTIONS:
                log.log(f"[MCTS][detail] a={a} N={root.N_a[a]:4d} Q={root.Q_a[a]:+8.3f} P={root.P_a[a]:.3f}", t=t, every=None, lvl=2)

    return choose_final_action(root)


# =========================
# Main Simulation
# =========================

@dataclass
class Config:
    seed: int = 7

    H: int = 12
    W: int = 18
    T: int = 40

    num_particles: int = 250

    # PF
    motion_noise: float = 0.15
    resample_threshold: float = 0.55

    # Mode library
    num_hypotheses: int = 25
    per_hypothesis_attempts: int = 2
    max_modes: int = 6
    w_detect_astar: float = 10.0
    rand_scale: float = 0.45

    # NEW: plausibility-vs-cost tradeoff
    plausibility_lambda: float = 3.0

    # NEW: prior temperature
    prior_temp: float = 25.0

    # Prior smoothing/diagnostics (helps catch "prior collapse" from the mode library)
    prior_eps_main: float = 0.15
    prior_warn_maxp: float = 0.97
    prior_warn_minH: float = 0.05

    # MCTS
    num_sims: int = 250
    max_depth: int = 10
    c_puct: float = 1.4
    w_step_mcts: float = 1.0
    w_detect_mcts: float = 7.0

    # NEW: force a specific true mode for reproducibility
    true_mode_name: Optional[str] = "chase_robot"  # set None for random

    # Debug
    debug: DebugConfig = field(default_factory=DebugConfig)


def make_demo_world(cfg: Config) -> Tuple[GridWorld, Tuple[int, int], Tuple[int, int], Tuple[int, int]]:
    obs = set()
    for c in range(3, 15):
        obs.add((5, c))
    for r in range(1, 9):
        obs.add((r, 9))
    obs.discard((5, 7))
    obs.discard((6, 9))

    world = GridWorld(H=cfg.H, W=cfg.W, obstacles=obs)
    robot_start = (10, 2)
    goal = (1, 16)
    sensor_start = (2, 2)
    return world, robot_start, sensor_start, goal


def main() -> None:
    cfg = Config()
    seed_all(cfg.seed)

    log = Logger(cfg.debug)

    world, robot_pos, sensor_pos, goal = make_demo_world(cfg)

    sensor_modes: List[SensorModePolicy] = [
        PatrolLoopPolicy(name="patrol_left", loop=[(2, 2), (2, 6), (4, 6), (4, 2)]),
        PatrolLoopPolicy(name="patrol_mid", loop=[(2, 10), (2, 13), (4, 13), (4, 10)]),
        ChasePolicy(name="chase_robot"),
        RandomWalkPolicy(name="random_walk"),
    ]
    mode_names = [m.name for m in sensor_modes]

    # NEW: deterministic true mode option
    if cfg.true_mode_name is None:
        true_mode_idx = random.randrange(len(sensor_modes))
    else:
        try:
            true_mode_idx = mode_names.index(cfg.true_mode_name)
        except ValueError:
            raise ValueError(f"true_mode_name={cfg.true_mode_name!r} not in modes {mode_names}")

    detection = DetectionModel()
    obs_model = ObservationModel(sigma=1.3)

    sensor_init_candidates = [(2, 2), (2, 10), (3, 3), (3, 9), (1, 1)]
    mode_prior = [1.0] * len(sensor_modes)
    belief = initialize_belief(world, sensor_init_candidates, mode_prior, mode_names, cfg.num_particles)

    log.log(f"[INIT] true_mode={mode_names[true_mode_idx]} robot={robot_pos} sensor={sensor_pos} goal={goal}", force=True)
    log.log(f"[INIT] belief modes: {topk_str(belief.mode_posterior_named(), k=4)}", force=True)

    for t in range(cfg.T):
        show_header = (t <= cfg.debug.detail_until_t) or (cfg.debug.enabled and cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)
        if show_header:
            log.log("=" * 80, force=True)
            log.log(f"[TIME] t={t}", force=True)

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.render_every_t > 0 and t % cfg.debug.render_every_t == 0)):
            log.log("[WORLD]", t=t, every=cfg.debug.render_every_t, lvl=1, force=(t <= cfg.debug.detail_until_t))
            world.render(robot=robot_pos, sensor=sensor_pos, goal=goal)

        obs = obs_model.sample_observation(sensor_pos, world)
        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[OBS] obs={obs} (true sensor={sensor_pos})", force=True)

        # PF update
        _ = belief_predict_update(
            belief,
            world,
            robot_pos,
            sensor_modes,
            obs_model,
            obs,
            t,
            motion_noise=cfg.motion_noise,
            resample_threshold=cfg.resample_threshold,
            log=log,
        )

        # Mode library -> prior
        library, _ = build_multimodal_path_library(
            world,
            belief,
            robot_pos,
            goal,
            detection,
            num_hypotheses=cfg.num_hypotheses,
            per_hypothesis_attempts=cfg.per_hypothesis_attempts,
            w_detect=cfg.w_detect_astar,
            rand_scale=cfg.rand_scale,
            max_modes=cfg.max_modes,
            plausibility_lambda=cfg.plausibility_lambda,
            t=t,
            log=log,
        )

        raw_prior = build_action_prior_from_library(world, robot_pos, library, temp=cfg.prior_temp)
        action_prior = normalize_probs_dict(raw_prior, ACTIONS, eps=cfg.prior_eps_main)

        # Catch and explain "prior collapse" (degenerate action prior)
        prior_check_and_log(
            world,
            robot_pos,
            library,
            raw_prior,
            action_prior,
            temp=cfg.prior_temp,
            eps=cfg.prior_eps_main,
            mode_names=belief.mode_names,
            warn_maxp=cfg.prior_warn_maxp,
            warn_minH=cfg.prior_warn_minH,
            t=t,
            log=log,
        )

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[PriorDiag] eps={cfg.prior_eps_main:.2f} {prior_diagnostics(action_prior)}", force=True)

        # MCTS choose action
        aR = mcts_search_action(
            world,
            belief,
            robot_pos,
            goal,
            sensor_modes,
            detection,
            action_prior,
            t=t,
            num_sims=cfg.num_sims,
            max_depth=cfg.max_depth,
            c_puct=cfg.c_puct,
            w_step=cfg.w_step_mcts,
            w_detect=cfg.w_detect_mcts,
            prior_eps=0.0,
            log=log,
        )

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[ACT] robot aR={aR}", force=True)

        # Step robot and true sensor
        new_robot = world.step(robot_pos, aR)
        aS_true = sensor_modes[true_mode_idx].action(world, sensor_pos, new_robot, t)

        # NEW: chase sanity check on TRUE sensor if chase mode
        if mode_names[true_mode_idx] == "chase_robot":
            chase_sanity_check(world, sensor_pos, new_robot, aS_true, log=log, t=t, tag="true")

        new_sensor = world.step(sensor_pos, aS_true)

        robot_pos, sensor_pos = new_robot, new_sensor

        # Detection
        p_det = detection.p_detect(robot_pos, sensor_pos)
        detected = (random.random() < p_det)

        if cfg.debug.enabled and (t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)):
            log.log(f"[STEP] robot={robot_pos} sensor={sensor_pos} aS={aS_true} p_det={p_det:.3f} detected={detected}", force=True)

        # Terminate
        if robot_pos == goal:
            log.log("[TERMINAL] reached goal ✅", force=True)
            break
        if detected and manhattan(robot_pos, sensor_pos) <= detection.d0:
            log.log("[TERMINAL] caught ❌", force=True)
            break

    log.log("[DONE]", force=True)


if __name__ == "__main__":
    main()


[INIT] true_mode=chase_robot robot=(10, 2) sensor=(2, 2) goal=(1, 16)
[INIT] belief modes: random_walk:0.272 | patrol_left:0.260 | patrol_mid:0.240 | chase_robot:0.228
[TIME] t=0
[WORLD]
..................
.........#......G.
..S......#........
.........#........
.........#........
...####.#######...
..................
.........#........
.........#........
..................
..R...............
..................
[OBS] obs=(4, 3) (true sensor=(2, 2))
[PF] t=0 obs=(4, 3) ESS 250.0->61.1 resample=True | modes: chase_robot:0.508 | random_walk:0.192 | patrol_left:0.152
[Modes] t=0 candidates=50 clusters=6 kept=6 | top3: random_walk:w=0.004,cost=37.1,score=53.7, random_walk:w=0.004,cost=37.2,score=53.8, random_walk:w=0.004,cost=37.7,score=54.3
[PriorCheck][WARN] t=0 kept=6 distinct_a0=1(R) rawH=0.000 rawMax=1.000 | eps=0.15 smH=0.533 smMax=0.880
[PriorDiagRaw] H=0.000 top=R:1.000 | U:0.000 | D:0.000
[PriorDiagSm]  eps=0.15 H=0.533 top=R:0.880 | U:0.030 | D:0.030
[PriorCheck] mix_mass(raw): R:

In [2]:
#!/usr/bin/env python3
"""GenBR-lite + PF Belief + Multimodal Path Library (debug-robust)

This version includes the specific fixes suggested from your logs:
1) **Chase policy sanity + determinism**:
   - Chase now uses a deterministic tie-break (no random sideways moves on ties).
   - We print (and warn on) any chase step that *increases* Manhattan distance when a decreasing move exists.

2) **Mode-library selection uses plausibility + cost** (not cost-only optimism):
   - Each candidate path is scored as:  score = cost + lambda_plaus * (-log(hyp_weight)).
   - Best-per-cluster and final sorting use this score.

3) **Action prior is a mixture over modes** (not single-best-path):
   - Prior weights each kept mode by: hyp_weight * exp(-cost/temp).

4) **Detection is actually terminal**:
   - If detected and within d0, episode terminates as "caught".

What to send me to verify correctness:
- Run with DebugConfig(level=2, detail_until_t=6, summary_every_t=1, render_every_t=1) for ~10 steps.
- Paste the log lines that start with: [CHASE], [CHASE][WARN], [ModesKept], [PriorDiag], [MCTS]

Run:
  python3 this_file.py
"""

from __future__ import annotations

import math
import random
import heapq
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Iterable, Callable, Any, Set
from collections import defaultdict
import hashlib


# =========================
# Debug / Logging utilities
# =========================

@dataclass
class DebugConfig:
    enabled: bool = True
    level: int = 1
    # 0 = silent
    # 1 = periodic summaries
    # 2 = detailed (still gated)

    # Print detail for the first few timesteps
    detail_until_t: int = 2

    # Periodic summary controls
    summary_every_t: int = 5
    render_every_t: int = 10

    # Stage-specific periodic prints
    pf_every_t: int = 5
    modes_every_t: int = 5
    mcts_every_t: int = 5

    # Within MCTS
    mcts_progress_every_sims: int = 50

    # Safety: cap printing of mode hypotheses even at detail level
    max_hyp_print: int = 3


class Logger:
    def __init__(self, cfg: DebugConfig):
        self.cfg = cfg

    def _should_print(self, t: Optional[int], every: Optional[int], lvl: int) -> bool:
        if not self.cfg.enabled:
            return False
        if lvl > self.cfg.level:
            return False
        if t is None:
            return True
        if t <= self.cfg.detail_until_t:
            return True
        if every is None:
            return False
        return (every > 0) and (t % every == 0)

    def log(self, msg: str, *, t: Optional[int] = None, every: Optional[int] = None, lvl: int = 1, force: bool = False) -> None:
        if force or self._should_print(t, every, lvl):
            print(msg)


def soft_assert(cond: bool, msg: str, *, fatal: bool = False, log: Optional[Logger] = None, t: Optional[int] = None) -> None:
    if cond:
        return
    if log is not None:
        log.log(f"[WARN] {msg}", t=t, every=None, lvl=1, force=True)
    if fatal:
        raise AssertionError(msg)


def clamp(x: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, x))


def manhattan(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def seed_all(seed: int) -> None:
    random.seed(seed)


def stable_bucket(sig: str, m: int) -> int:
    """Stable mapping sig -> [0..m-1] independent of Python's randomized hash."""
    if m <= 0:
        return 0
    h = hashlib.md5(sig.encode("utf-8")).hexdigest()
    return int(h[:8], 16) % m


def normalize_probs_dict(d: Dict[Any, float], keys: Iterable[Any], *, eps: float = 0.0) -> Dict[Any, float]:
    """Return a normalized distribution over `keys` from possibly-missing/negative inputs.

    - Clips negatives to 0
    - Adds optional epsilon smoothing to avoid degeneracy
    - Falls back to uniform if total mass is 0
    """
    klist = list(keys)
    out = {k: max(0.0, float(d.get(k, 0.0))) for k in klist}
    s = sum(out.values())

    if s <= 0.0:
        u = 1.0 / max(1, len(klist))
        return {k: u for k in klist}

    out = {k: v / s for k, v in out.items()}

    if eps > 0.0:
        u = 1.0 / max(1, len(klist))
        out = {k: (1.0 - eps) * out[k] + eps * u for k in klist}
        s2 = sum(out.values())
        out = {k: v / s2 for k, v in out.items()}

    return out


def topk_str(d: Dict[Any, float], k: int = 3, fmt: str = "{k}:{v:.3f}") -> str:
    items = sorted(d.items(), key=lambda kv: kv[1], reverse=True)[:k]
    return " | ".join(fmt.format(k=kk, v=vv) for kk, vv in items)


# =========================
# Gridworld Environment
# =========================

Action = str  # 'U','D','L','R','S'

ACTIONS: List[Action] = ["U", "D", "L", "R", "S"]
ACTION_DELTAS: Dict[Action, Tuple[int, int]] = {
    "U": (-1, 0),
    "D": (1, 0),
    "L": (0, -1),
    "R": (0, 1),
    "S": (0, 0),
}


@dataclass
class GridWorld:
    H: int
    W: int
    obstacles: Set[Tuple[int, int]] = field(default_factory=set)

    def in_bounds(self, p: Tuple[int, int]) -> bool:
        r, c = p
        return 0 <= r < self.H and 0 <= c < self.W

    def is_free(self, p: Tuple[int, int]) -> bool:
        return self.in_bounds(p) and (p not in self.obstacles)

    def step(self, p: Tuple[int, int], a: Action) -> Tuple[int, int]:
        if a not in ACTION_DELTAS:
            raise ValueError(f"Unknown action: {a}")
        dr, dc = ACTION_DELTAS[a]
        np = (p[0] + dr, p[1] + dc)
        return np if self.is_free(np) else p

    def neighbors(self, p: Tuple[int, int]) -> List[Tuple[Tuple[int, int], Action]]:
        return [(self.step(p, a), a) for a in ACTIONS]

    def render(self, robot: Tuple[int, int], sensor: Tuple[int, int], goal: Tuple[int, int]) -> None:
        grid = [["." for _ in range(self.W)] for _ in range(self.H)]
        for (r, c) in self.obstacles:
            grid[r][c] = "#"
        rr, rc = robot
        sr, sc = sensor
        gr, gc = goal
        grid[gr][gc] = "G"
        grid[sr][sc] = "S"
        grid[rr][rc] = "R"
        for r in range(self.H):
            print("".join(grid[r]))


# =========================
# Sensor Models (Modes)
# =========================

@dataclass
class SensorModePolicy:
    name: str

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        raise NotImplementedError


def best_actions_toward(world: GridWorld, src: Tuple[int, int], dst: Tuple[int, int]) -> Tuple[List[Action], int]:
    """Return (best_actions, best_dist) minimizing manhattan(next(src,a), dst)."""
    best_actions: List[Action] = []
    best_dist = 10**9
    for a in ACTIONS:
        np = world.step(src, a)
        d = manhattan(np, dst)
        if d < best_dist:
            best_dist = d
            best_actions = [a]
        elif d == best_dist:
            best_actions.append(a)
    return best_actions, best_dist


def deterministic_greedy_step_toward(world: GridWorld, src: Tuple[int, int], dst: Tuple[int, int]) -> Action:
    """Greedy toward dst with deterministic tie-break (easier to debug than random ties)."""
    best_as, _ = best_actions_toward(world, src, dst)
    if not best_as:
        return "S"

    # Preferred axis: move along the larger |delta| if possible
    dr = dst[0] - src[0]
    dc = dst[1] - src[1]

    preferred: List[Action] = []
    if abs(dr) >= abs(dc):
        if dr > 0:
            preferred.append("D")
        elif dr < 0:
            preferred.append("U")
    if abs(dc) > abs(dr):
        if dc > 0:
            preferred.append("R")
        elif dc < 0:
            preferred.append("L")

    # If the preferred move is among best actions, take it
    for a in preferred:
        if a in best_as:
            return a

    # Otherwise deterministic order
    for a in ["U", "D", "L", "R", "S"]:
        if a in best_as:
            return a

    return best_as[0]


def chase_sanity_check(
    world: GridWorld,
    src: Tuple[int, int],
    dst: Tuple[int, int],
    chosen: Action,
    *,
    log: Optional[Logger],
    t: int,
    tag: str,
) -> None:
    """Warn if chosen action increases distance when a non-increasing action exists."""
    d0 = manhattan(src, dst)
    nxt = world.step(src, chosen)
    d1 = manhattan(nxt, dst)

    best_as, best_d = best_actions_toward(world, src, dst)

    if log is not None and log.cfg.enabled and (t <= log.cfg.detail_until_t or (log.cfg.summary_every_t > 0 and t % log.cfg.summary_every_t == 0)):
        log.log(
            f"[CHASE] {tag} t={t} src={src} dst={dst} a={chosen} nxt={nxt} d:{d0}->{d1} best_d={best_d} best={best_as}",
            t=t,
            every=log.cfg.summary_every_t,
            lvl=1,
        )

    # If chosen isn't among best actions, it's suspicious
    if chosen not in best_as:
        if log is not None:
            log.log(
                f"[CHASE][WARN] {tag} t={t} chose {chosen} but best={best_as} (d:{d0}->{d1}, best_d={best_d})",
                t=t,
                every=None,
                lvl=1,
                force=True,
            )


@dataclass
class PatrolLoopPolicy(SensorModePolicy):
    loop: List[Tuple[int, int]] = field(default_factory=list)

    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        if not self.loop:
            return "S"
        idx = t % len(self.loop)
        return deterministic_greedy_step_toward(world, sensor_pos, self.loop[idx])


@dataclass
class ChasePolicy(SensorModePolicy):
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        return deterministic_greedy_step_toward(world, sensor_pos, robot_pos)


@dataclass
class RandomWalkPolicy(SensorModePolicy):
    def action(self, world: GridWorld, sensor_pos: Tuple[int, int], robot_pos: Tuple[int, int], t: int) -> Action:
        return random.choice(ACTIONS)


# =========================
# Detection / Observation
# =========================

@dataclass
class DetectionModel:
    d0: int = 2
    p_close: float = 0.9
    p_far: float = 0.05
    decay: float = 0.3

    def p_detect(self, robot_pos: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        dist = manhattan(robot_pos, sensor_pos)
        if dist <= self.d0:
            return self.p_close
        return self.p_far + (self.p_close - self.p_far) * math.exp(-self.decay * (dist - self.d0))


@dataclass
class ObservationModel:
    sigma: float = 1.2

    def sample_observation(self, sensor_pos: Tuple[int, int], world: GridWorld) -> Tuple[int, int]:
        max_jump = 2
        dr = random.randint(-max_jump, max_jump)
        dc = random.randint(-max_jump, max_jump)
        o = (sensor_pos[0] + dr, sensor_pos[1] + dc)
        o = (int(clamp(o[0], 0, world.H - 1)), int(clamp(o[1], 0, world.W - 1)))
        return o

    def likelihood(self, obs: Tuple[int, int], sensor_pos: Tuple[int, int]) -> float:
        d = manhattan(obs, sensor_pos)
        return max(1e-12, math.exp(-d / max(1e-6, self.sigma)))


# =========================
# Particle Filter over (sensor_pos, mode)
# =========================

@dataclass
class Particle:
    sensor_pos: Tuple[int, int]
    mode_idx: int
    weight: float = 1.0


@dataclass
class BeliefState:
    particles: List[Particle]
    mode_names: List[str]

    def normalize(self) -> None:
        s = sum(p.weight for p in self.particles)
        if s <= 0.0:
            w = 1.0 / max(1, len(self.particles))
            for p in self.particles:
                p.weight = w
            return
        for p in self.particles:
            p.weight /= s

    def effective_sample_size(self) -> float:
        s = sum(p.weight * p.weight for p in self.particles)
        return 0.0 if s <= 0.0 else 1.0 / s

    def mode_posterior(self) -> Dict[int, float]:
        post: Dict[int, float] = defaultdict(float)
        for p in self.particles:
            post[p.mode_idx] += p.weight
        return dict(post)

    def mode_posterior_named(self) -> Dict[str, float]:
        post_i = self.mode_posterior()
        out: Dict[str, float] = {}
        for i, v in post_i.items():
            name = self.mode_names[i] if 0 <= i < len(self.mode_names) else f"mode{i}"
            out[name] = out.get(name, 0.0) + v
        return out


def systematic_resample(particles: List[Particle]) -> List[Particle]:
    n = len(particles)
    if n == 0:
        return []
    positions = [(random.random() + i) / n for i in range(n)]
    cumulative = []
    csum = 0.0
    for p in particles:
        csum += p.weight
        cumulative.append(csum)

    out: List[Particle] = []
    i = 0
    for pos in positions:
        while i < n - 1 and pos > cumulative[i]:
            i += 1
        chosen = particles[i]
        out.append(Particle(sensor_pos=chosen.sensor_pos, mode_idx=chosen.mode_idx, weight=1.0 / n))
    return out


def initialize_belief(
    world: GridWorld,
    sensor_init_candidates: List[Tuple[int, int]],
    mode_prior: List[float],
    mode_names: List[str],
    num_particles: int = 200,
) -> BeliefState:
    if len(mode_prior) != len(mode_names):
        raise ValueError("mode_prior and mode_names must have same length")
    if not sensor_init_candidates:
        raise ValueError("Need at least one sensor init candidate")
    if num_particles <= 0:
        raise ValueError("num_particles must be > 0")

    # normalize prior
    s = sum(mode_prior)
    if s <= 0.0:
        mode_prior = [1.0 / len(mode_prior)] * len(mode_prior)
    else:
        mode_prior = [p / s for p in mode_prior]

    def sample_mode() -> int:
        r = random.random()
        c = 0.0
        for i, p in enumerate(mode_prior):
            c += p
            if r <= c:
                return i
        return len(mode_prior) - 1

    particles: List[Particle] = []
    for _ in range(num_particles):
        sp = random.choice(sensor_init_candidates)
        z = sample_mode()
        particles.append(Particle(sensor_pos=sp, mode_idx=z, weight=1.0 / num_particles))

    b = BeliefState(particles=particles, mode_names=mode_names)
    b.normalize()
    return b


@dataclass
class PFStepInfo:
    ess_before: float
    ess_after: float
    resampled: bool


def belief_predict_update(
    belief: BeliefState,
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    obs_model: ObservationModel,
    obs: Tuple[int, int],
    t: int,
    *,
    motion_noise: float = 0.10,
    resample_threshold: float = 0.5,
    log: Optional[Logger] = None,
) -> PFStepInfo:
    if not belief.particles:
        raise ValueError("belief has no particles")

    belief.normalize()
    ess0 = belief.effective_sample_size()

    # Predict + weight update
    for p in belief.particles:
        mi = p.mode_idx
        if 0 <= mi < len(sensor_policies):
            aS = sensor_policies[mi].action(world, p.sensor_pos, robot_pos, t)
        else:
            aS = "S"

        predicted = world.step(p.sensor_pos, aS)
        if random.random() < motion_noise:
            predicted = world.step(predicted, random.choice(ACTIONS))

        p.sensor_pos = predicted
        p.weight *= obs_model.likelihood(obs, p.sensor_pos)

    belief.normalize()
    ess1 = belief.effective_sample_size()

    # Sanity checks
    N = len(belief.particles)
    soft_assert(0.0 <= ess1 <= N + 1e-6, f"ESS out of bounds: {ess1} (N={N})", log=log, t=t)

    resampled = False
    if ess1 < resample_threshold * N:
        belief.particles = systematic_resample(belief.particles)
        belief.normalize()
        resampled = True

    # Controlled prints
    if log is not None:
        if log._should_print(t, log.cfg.pf_every_t, lvl=1) or resampled:
            mp = belief.mode_posterior_named()
            log.log(
                f"[PF] t={t} obs={obs} ESS {ess0:.1f}->{ess1:.1f} resample={resampled} | modes: {topk_str(mp, k=3)}",
                t=t,
                every=log.cfg.pf_every_t,
                lvl=1,
                force=resampled,
            )
            if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
                log.log(f"[PF][detail] full mode posterior: {topk_str(mp, k=10)}", t=t, every=None, lvl=2)

    return PFStepInfo(ess_before=ess0, ess_after=ess1, resampled=resampled)


# =========================
# A* Search with Risk Costs
# =========================

@dataclass
class AStarResult:
    path: List[Tuple[int, int]]
    cost: float
    expanded: int


def astar_path(
    world: GridWorld,
    start: Tuple[int, int],
    goal: Tuple[int, int],
    step_cost: Callable[[Tuple[int, int]], float],
    heuristic: Callable[[Tuple[int, int]], float],
    *,
    max_expansions: int = 50_000,
) -> Optional[AStarResult]:
    if not world.is_free(start) or not world.is_free(goal):
        return None

    frontier: List[Tuple[float, float, Tuple[int, int]]] = []
    heapq.heappush(frontier, (heuristic(start), 0.0, start))

    came_from: Dict[Tuple[int, int], Tuple[int, int]] = {}
    gscore: Dict[Tuple[int, int], float] = {start: 0.0}

    expanded = 0

    # A* with reopen: skip stale pops
    while frontier and expanded < max_expansions:
        f, g, cur = heapq.heappop(frontier)
        if g > gscore.get(cur, float("inf")) + 1e-12:
            continue
        expanded += 1

        if cur == goal:
            path = [cur]
            while cur in came_from:
                cur = came_from[cur]
                path.append(cur)
            path.reverse()
            return AStarResult(path=path, cost=gscore[goal], expanded=expanded)

        for (nxt, _) in world.neighbors(cur):
            if not world.is_free(nxt):
                continue
            sc = step_cost(nxt)
            if not math.isfinite(sc) or sc < 0:
                continue
            ng = gscore[cur] + sc
            if ng + 1e-12 < gscore.get(nxt, float("inf")):
                gscore[nxt] = ng
                came_from[nxt] = cur
                heapq.heappush(frontier, (ng + heuristic(nxt), ng, nxt))

    return None


# =========================
# Multimodal Path Library
# =========================

@dataclass
class PathMode:
    path: List[Tuple[int, int]]
    total_cost: float
    score: float
    hypothesis_sensor_pos: Tuple[int, int]
    hypothesis_mode_idx: int
    hypothesis_weight: float
    debug_info: Dict[str, Any] = field(default_factory=dict)


def build_cost_map_from_hypothesis(
    world: GridWorld,
    detection: "DetectionModel",
    sensor_pos: Tuple[int, int],
    *,
    base_step: float,
    w_detect: float,
    randomize: bool,
    rand_scale: float,
) -> Callable[[Tuple[int, int]], float]:
    def cost_fn(s: Tuple[int, int]) -> float:
        if not world.is_free(s):
            return float("inf")
        p_det = detection.p_detect(robot_pos=s, sensor_pos=sensor_pos)
        c = base_step + w_detect * p_det
        if randomize:
            c *= (1.0 + rand_scale * (random.random() - 0.5))
            c = max(1e-6, c)
        return c

    return cost_fn


def cluster_paths_by_signature(paths: List[List[Tuple[int, int]]], *, max_modes: int) -> List[int]:
    sig_to_cluster: Dict[str, int] = {}
    assignments: List[int] = []

    def signature(path: List[Tuple[int, int]]) -> str:
        if len(path) < 2:
            return "EMPTY"
        dirs = []
        for i in range(1, len(path)):
            dr = path[i][0] - path[i - 1][0]
            dc = path[i][1] - path[i - 1][1]
            dirs.append((dr, dc))

        comp = []
        for d in dirs:
            if not comp or comp[-1] != d:
                comp.append(d)

        m = {(-1, 0): "U", (1, 0): "D", (0, -1): "L", (0, 1): "R", (0, 0): "S"}
        return "".join(m.get(d, "?") for d in comp)[:120]

    for p in paths:
        sig = signature(p)

        if sig in sig_to_cluster:
            cid = sig_to_cluster[sig]
        elif len(sig_to_cluster) < max_modes:
            cid = len(sig_to_cluster)
            sig_to_cluster[sig] = cid
        else:
            cid = stable_bucket(sig, len(sig_to_cluster))

        assignments.append(cid)

    return assignments


@dataclass
class ModeLibraryInfo:
    num_candidates: int
    num_clusters: int


def unique_weighted_sample(particles: List[Particle], weights: List[float], k: int) -> List[Particle]:
    """Approximate weighted sampling without replacement by oversampling then uniquing."""
    if not particles:
        return []
    if k <= 0:
        return []
    oversample = max(k, 3 * k)
    picks = random.choices(particles, weights=weights, k=oversample)
    seen = set()
    out: List[Particle] = []
    for p in picks:
        key = (p.sensor_pos, p.mode_idx)
        if key in seen:
            continue
        seen.add(key)
        out.append(p)
        if len(out) >= k:
            break
    # If we still don't have k, allow duplicates as a fallback
    while len(out) < k:
        out.append(random.choices(particles, weights=weights, k=1)[0])
    return out


def build_multimodal_path_library(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    *,
    num_hypotheses: int = 30,
    per_hypothesis_attempts: int = 2,
    base_step_cost: float = 1.0,
    w_detect: float = 8.0,
    rand_scale: float = 0.40,
    max_modes: int = 6,
    plausibility_lambda: float = 3.0,
    t: int = 0,
    log: Optional[Logger] = None,
) -> Tuple[List[PathMode], ModeLibraryInfo]:
    if not belief.particles:
        return [], ModeLibraryInfo(num_candidates=0, num_clusters=0)

    # Sample particles proportional to weight
    weights = [p.weight for p in belief.particles]
    s = sum(weights)
    if s <= 0.0:
        weights = [1.0 / len(weights)] * len(weights)
    else:
        weights = [w / s for w in weights]

    # NEW: more diverse sampling
    sampled_particles = unique_weighted_sample(belief.particles, weights, k=num_hypotheses)

    candidates: List[PathMode] = []

    hyp_print_budget = log.cfg.max_hyp_print if (log and log.cfg.level >= 2 and t <= log.cfg.detail_until_t) else 0

    for hi, hp in enumerate(sampled_particles):
        hyp_sensor = hp.sensor_pos
        hyp_mode = hp.mode_idx
        hyp_w = float(getattr(hp, "weight", 0.0))

        if log is not None and hyp_print_budget > 0:
            log.log(
                f"[Modes][detail] t={t} hyp#{hi+1}/{num_hypotheses} sensor={hyp_sensor} mode={belief.mode_names[hyp_mode] if 0<=hyp_mode<len(belief.mode_names) else hyp_mode} w={hyp_w:.4f}",
                t=t,
                every=None,
                lvl=2,
            )
            hyp_print_budget -= 1

        for attempt in range(per_hypothesis_attempts):
            cost_fn = build_cost_map_from_hypothesis(
                world,
                detection,
                hyp_sensor,
                base_step=base_step_cost,
                w_detect=w_detect,
                randomize=True,
                rand_scale=rand_scale,
            )
            heur = lambda s: float(manhattan(s, goal))

            res = astar_path(world, robot_pos, goal, step_cost=cost_fn, heuristic=heur)
            if res is None:
                continue

            # NEW: plausibility-regularized score
            score = float(res.cost) + plausibility_lambda * (-math.log(max(1e-12, hyp_w)))

            candidates.append(
                PathMode(
                    path=res.path,
                    total_cost=float(res.cost),
                    score=score,
                    hypothesis_sensor_pos=hyp_sensor,
                    hypothesis_mode_idx=hyp_mode,
                    hypothesis_weight=hyp_w,
                    debug_info={"expanded": res.expanded, "attempt": attempt + 1},
                )
            )

    if not candidates:
        if log is not None and log._should_print(t, log.cfg.modes_every_t, lvl=1):
            log.log(f"[Modes] t={t} candidates=0 (no paths found)", t=t, every=log.cfg.modes_every_t, lvl=1)
        return [], ModeLibraryInfo(num_candidates=0, num_clusters=0)

    def _select_library(cands: List[PathMode]) -> Tuple[List[PathMode], Dict[int, PathMode]]:
        cluster_ids = cluster_paths_by_signature([c.path for c in cands], max_modes=max_modes)

        # pick best by *score* per cluster
        best_by_cluster: Dict[int, PathMode] = {}
        for c, cid in zip(cands, cluster_ids):
            if cid not in best_by_cluster or c.score < best_by_cluster[cid].score:
                best_by_cluster[cid] = c

        # sort by score, not raw cost
        library = sorted(best_by_cluster.values(), key=lambda pm: pm.score)[:max_modes]
        return library, best_by_cluster

    library, best_by_cluster = _select_library(candidates)

    # DIVERSITY RESCUE: if all kept modes agree on the same first robot action,
    # inject additional candidates by forcing alternative first actions, then re-cluster.
    kept_a0 = [first_action_from_path(world, robot_pos, pm.path) for pm in library]
    distinct_a0 = sorted(set(kept_a0))

    if len(library) >= 2 and len(distinct_a0) <= 1:
        dom = distinct_a0[0] if distinct_a0 else "?"
        if log is not None:
            log.log(
                f"[ModesRescue][WARN] t={t} all kept paths start with {dom}; attempting forced-first-action rescue",
                t=t,
                every=None,
                lvl=1,
                force=True,
            )

        # Pick a few high-weight hypotheses (unique (sensor_pos, mode_idx))
        uniq: Dict[Tuple[Tuple[int, int], int], Particle] = {}
        for hp in sampled_particles:
            uniq[(hp.sensor_pos, hp.mode_idx)] = hp
        top_hyps = sorted(uniq.values(), key=lambda p: float(getattr(p, "weight", 0.0)), reverse=True)
        top_hyps = top_hyps[: min(3, len(top_hyps))]

        added = 0
        for hp in top_hyps:
            hyp_sensor = hp.sensor_pos
            hyp_mode = hp.mode_idx
            hyp_w = float(getattr(hp, "weight", 0.0))

            # deterministic cost map for rescue (no randomization)
            cost_fn = build_cost_map_from_hypothesis(
                world,
                detection,
                hyp_sensor,
                base_step=base_step_cost,
                w_detect=w_detect,
                randomize=False,
                rand_scale=0.0,
            )
            heur = lambda s: float(manhattan(s, goal))

            for a0 in ACTIONS:
                if a0 == dom:
                    continue

                nxt0 = world.step(robot_pos, a0)

                if a0 == "S":
                    res2 = astar_path(world, robot_pos, goal, step_cost=cost_fn, heuristic=heur)
                    if res2 is None:
                        continue
                    forced_path = [robot_pos, robot_pos] + res2.path[1:]
                    forced_cost = float(cost_fn(robot_pos)) + float(res2.cost)
                    expanded = res2.expanded
                else:
                    res2 = astar_path(world, nxt0, goal, step_cost=cost_fn, heuristic=heur)
                    if res2 is None:
                        continue
                    forced_path = [robot_pos] + res2.path
                    forced_cost = float(cost_fn(nxt0)) + float(res2.cost)
                    expanded = res2.expanded

                score = float(forced_cost) + plausibility_lambda * (-math.log(max(1e-12, hyp_w)))
                candidates.append(
                    PathMode(
                        path=forced_path,
                        total_cost=float(forced_cost),
                        score=float(score),
                        hypothesis_sensor_pos=hyp_sensor,
                        hypothesis_mode_idx=hyp_mode,
                        hypothesis_weight=hyp_w,
                        debug_info={"expanded": expanded, "attempt": "rescue", "forced_a0": a0},
                    )
                )
                added += 1

        if log is not None:
            log.log(
                f"[ModesRescue] t={t} added={added} forced-a0 candidates (excluding {dom})",
                t=t,
                every=None,
                lvl=1,
                force=True,
            )

        # Re-select after rescue
        library, best_by_cluster = _select_library(candidates)
        kept2 = sorted(set(first_action_from_path(world, robot_pos, pm.path) for pm in library))
        if log is not None:
            log.log(
                f"[ModesRescue] t={t} new kept distinct_a0={len(kept2)}({','.join(kept2) if kept2 else '-'})",
                t=t,
                every=None,
                lvl=1,
                force=True,
            )

    # FINAL DIVERSITY GUARANTEE:
    # Even after rescue, clustering/score-sorting can still throw away all non-dominant a0.
    # Here we *force* at least 2 distinct first-actions in the kept library whenever possible.
    min_diverse_a0 = 2
    kept_a0 = [first_action_from_path(world, robot_pos, pm.path) for pm in library]
    distinct_a0 = sorted(set(kept_a0))

    if library and len(distinct_a0) < min_diverse_a0:
        dom = distinct_a0[0] if distinct_a0 else "S"
        missing = [a for a in ACTIONS if a not in distinct_a0]

        # Best candidate for each missing first-action
        best_by_missing: Dict[Action, PathMode] = {}
        for pm in candidates:
            a0 = first_action_from_path(world, robot_pos, pm.path)
            if a0 in missing:
                if (a0 not in best_by_missing) or (pm.score < best_by_missing[a0].score):
                    best_by_missing[a0] = pm

        if best_by_missing:
            inject = min(best_by_missing.values(), key=lambda pm: pm.score)
            inject_a0 = first_action_from_path(world, robot_pos, inject.path)

            replaced: Optional[PathMode] = None
            if len(library) < max_modes:
                library.append(inject)
            else:
                # Prefer replacing a dominant-action entry, otherwise replace worst overall.
                dom_idxs = [i for i, pm in enumerate(library) if first_action_from_path(world, robot_pos, pm.path) == dom]
                if dom_idxs:
                    worst_idx = max(dom_idxs, key=lambda i: library[i].score)
                else:
                    worst_idx = max(range(len(library)), key=lambda i: library[i].score)
                replaced = library[worst_idx]
                library[worst_idx] = inject

            library = sorted(library, key=lambda pm: pm.score)[:max_modes]
            distinct2 = sorted(set(first_action_from_path(world, robot_pos, pm.path) for pm in library))

            if log is not None:
                if replaced is None:
                    log.log(
                        f"[ModesRescue][FORCE] t={t} appended a0={inject_a0} score={inject.score:.2f} -> distinct_a0={len(distinct2)}({','.join(distinct2)})",
                        t=t,
                        every=None,
                        lvl=1,
                        force=True,
                    )
                else:
                    rep_a0 = first_action_from_path(world, robot_pos, replaced.path)
                    log.log(
                        f"[ModesRescue][FORCE] t={t} replaced a0={rep_a0} score={replaced.score:.2f} with a0={inject_a0} score={inject.score:.2f} -> distinct_a0={len(distinct2)}({','.join(distinct2)})",
                        t=t,
                        every=None,
                        lvl=1,
                        force=True,
                    )
        else:
            if log is not None:
                log.log(
                    f"[ModesRescue][FORCE][FAIL] t={t} no viable non-{dom} candidates found; keeping degenerate a0",
                    t=t,
                    every=None,
                    lvl=1,
                    force=True,
                )

    if log is not None and log._should_print(t, log.cfg.modes_every_t, lvl=1):
        best = ", ".join(
            f"{belief.mode_names[pm.hypothesis_mode_idx] if 0<=pm.hypothesis_mode_idx<len(belief.mode_names) else pm.hypothesis_mode_idx}:"
            f"w={pm.hypothesis_weight:.3f},cost={pm.total_cost:.1f},score={pm.score:.1f}"
            for pm in library[:min(3, len(library))]
        )
        log.log(
            f"[Modes] t={t} candidates={len(candidates)} clusters={len(best_by_cluster)} kept={len(library)} | top3: {best}",
            t=t,
            every=log.cfg.modes_every_t,
            lvl=1,
        )

        if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
            for i, pm in enumerate(library):
                a0 = first_action_from_path(world, robot_pos, pm.path)
                mname = belief.mode_names[pm.hypothesis_mode_idx] if 0 <= pm.hypothesis_mode_idx < len(belief.mode_names) else str(pm.hypothesis_mode_idx)
                log.log(
                    f"[ModesKept] t={t} #{i+1} mode={mname} hypS={pm.hypothesis_sensor_pos} a0={a0} w={pm.hypothesis_weight:.4f} cost={pm.total_cost:.2f} score={pm.score:.2f}",
                    t=t,
                    every=None,
                    lvl=2,
                )

    return library, ModeLibraryInfo(num_candidates=len(candidates), num_clusters=len(best_by_cluster))


def first_action_from_path(world: GridWorld, start: Tuple[int, int], path: List[Tuple[int, int]]) -> Action:
    if len(path) < 2:
        return "S"
    nxt = path[1]
    dr = nxt[0] - start[0]
    dc = nxt[1] - start[1]
    for a, (adr, adc) in ACTION_DELTAS.items():
        if (dr, dc) == (adr, adc):
            return a
    return "S"


def build_action_prior_from_library(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    library: List[PathMode],
    *,
    temp: float = 25.0,
) -> Dict[Action, float]:
    """Mixture prior over kept modes: weight = hyp_weight * exp(-cost/temp)."""
    if not library:
        return {a: 1.0 / len(ACTIONS) for a in ACTIONS}

    pri_raw: Dict[Action, float] = defaultdict(float)
    for pm in library:
        a0 = first_action_from_path(world, robot_pos, pm.path)
        w = max(1e-12, pm.hypothesis_weight)
        w *= math.exp(-pm.total_cost / max(1e-6, temp))
        pri_raw[a0] += w

    return normalize_probs_dict(pri_raw, ACTIONS)


def prior_entropy(prior: Dict[Action, float]) -> float:
    ent = 0.0
    for a in ACTIONS:
        p = float(prior.get(a, 0.0))
        if p > 1e-12:
            ent -= p * math.log(p)
    return ent


def prior_diagnostics(prior: Dict[Action, float]) -> str:
    ent = prior_entropy(prior)
    return f"H={ent:.3f} top={topk_str(prior, k=3)}"


def prior_check_and_log(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    library: List[PathMode],
    raw_prior: Dict[Action, float],
    smoothed_prior: Dict[Action, float],
    *,
    temp: float,
    eps: float,
    mode_names: List[str],
    warn_maxp: float,
    warn_minH: float,
    t: int,
    log: Optional[Logger],
) -> None:
    """Warn when the library induces a near-delta prior.

    When it triggers, we print *why* (first-action mass + kept-mode breakdown), so you can
    tell whether it's a real collapse or just a legitimately one-sided situation.
    """
    if log is None or not log.cfg.enabled:
        return
    if not library:
        return

    kept_a0: List[Action] = [first_action_from_path(world, robot_pos, pm.path) for pm in library]
    distinct_a0 = sorted(set(kept_a0))

    rawH = prior_entropy(raw_prior)
    rawMax = max(float(raw_prior.get(a, 0.0)) for a in ACTIONS)
    smH = prior_entropy(smoothed_prior)
    smMax = max(float(smoothed_prior.get(a, 0.0)) for a in ACTIONS)

    warn = False
    if len(library) >= 2 and len(distinct_a0) <= 1:
        warn = True
    if rawMax >= warn_maxp:
        warn = True
    if rawH <= warn_minH:
        warn = True

    if not warn:
        return

    log.log(
        f"[PriorCheck][WARN] t={t} kept={len(library)} distinct_a0={len(distinct_a0)}({','.join(distinct_a0) if distinct_a0 else '-'}) "
        f"rawH={rawH:.3f} rawMax={rawMax:.3f} | eps={eps:.2f} smH={smH:.3f} smMax={smMax:.3f}",
        t=t,
        every=None,
        lvl=1,
        force=True,
    )
    log.log(f"[PriorDiagRaw] {prior_diagnostics(raw_prior)}", t=t, every=None, lvl=1, force=True)
    log.log(f"[PriorDiagSm]  eps={eps:.2f} {prior_diagnostics(smoothed_prior)}", t=t, every=None, lvl=1, force=True)

    mix_by_a: Dict[Action, float] = defaultdict(float)
    for pm in library:
        a0 = first_action_from_path(world, robot_pos, pm.path)
        mix_w = max(1e-12, pm.hypothesis_weight) * math.exp(-pm.total_cost / max(1e-6, temp))
        mix_by_a[a0] += mix_w

    mix_norm = normalize_probs_dict(mix_by_a, ACTIONS)
    log.log(f"[PriorCheck] mix_mass(raw): {topk_str(mix_by_a, k=5, fmt='{k}:{v:.3g}')}", t=t, every=None, lvl=1, force=True)
    log.log(f"[PriorCheck] mix_mass(nrm): {topk_str(mix_norm, k=5)}", t=t, every=None, lvl=1, force=True)

    for i, pm in enumerate(library, start=1):
        a0 = first_action_from_path(world, robot_pos, pm.path)
        mname = mode_names[pm.hypothesis_mode_idx] if 0 <= pm.hypothesis_mode_idx < len(mode_names) else str(pm.hypothesis_mode_idx)
        mix_w = max(1e-12, pm.hypothesis_weight) * math.exp(-pm.total_cost / max(1e-6, temp))
        log.log(
            f"[PriorCheck][Kept] t={t} #{i} a0={a0} mix_w={mix_w:.3g} mode={mname} hypS={pm.hypothesis_sensor_pos} "
            f"hyp_w={pm.hypothesis_weight:.4f} cost={pm.total_cost:.2f} score={pm.score:.2f}",
            t=t,
            every=None,
            lvl=1,
            force=True,
        )


# =========================
# GenBR-lite: Belief-space MCTS with PUCT
# =========================

@dataclass
class MCTSNode:
    robot_pos: Tuple[int, int]
    depth: int

    N: int = 0
    N_a: Dict[Action, int] = field(default_factory=lambda: {a: 0 for a in ACTIONS})
    W_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})
    Q_a: Dict[Action, float] = field(default_factory=lambda: {a: 0.0 for a in ACTIONS})

    P_a: Dict[Action, float] = field(default_factory=lambda: {a: 1.0 / len(ACTIONS) for a in ACTIONS})

    children: Dict[Action, "MCTSNode"] = field(default_factory=dict)


def select_action_puct(node: MCTSNode, *, c_puct: float) -> Action:
    """PUCT: argmax_a Q + c*P*sqrt(N)/(1+N_a). Uses node.P_a."""
    sqrtN = math.sqrt(max(1, node.N))

    best_a: Optional[Action] = None
    best_score = -1e18

    for a in ACTIONS:
        q = node.Q_a[a]
        u = c_puct * node.P_a[a] * sqrtN / (1.0 + node.N_a[a])
        score = q + u + random.uniform(-1e-9, 1e-9)
        if score > best_score:
            best_score = score
            best_a = a

    assert best_a is not None
    return best_a


def backprop_path(path: List[Tuple[MCTSNode, Action]], value: float) -> None:
    for node, a in reversed(path):
        node.N += 1
        node.N_a[a] += 1
        node.W_a[a] += value
        node.Q_a[a] = node.W_a[a] / node.N_a[a]


def choose_final_action(root: MCTSNode) -> Action:
    maxN = max(root.N_a.values())
    cand = [a for a in ACTIONS if root.N_a[a] == maxN]
    if len(cand) == 1:
        return cand[0]

    maxQ = max(root.Q_a[a] for a in cand)
    cand2 = [a for a in cand if abs(root.Q_a[a] - maxQ) < 1e-12]
    if len(cand2) == 1:
        return cand2[0]

    maxP = max(root.P_a[a] for a in cand2)
    cand3 = [a for a in cand2 if abs(root.P_a[a] - maxP) < 1e-12]
    return random.choice(cand3)


def rollout_value(
    world: GridWorld,
    robot_pos: Tuple[int, int],
    sensor_pos: Tuple[int, int],
    goal: Tuple[int, int],
    detection: DetectionModel,
    *,
    max_steps: int,
    w_step: float,
    w_detect: float,
) -> float:
    total_cost = 0.0
    cur = robot_pos
    for _ in range(max_steps):
        if cur == goal:
            break
        a = deterministic_greedy_step_toward(world, cur, goal)
        cur = world.step(cur, a)
        total_cost += w_step
        total_cost += w_detect * detection.p_detect(cur, sensor_pos)
    if cur == goal:
        total_cost -= 20.0
    return -total_cost


def simulate_sensor_step(
    world: GridWorld,
    sensor_pos: Tuple[int, int],
    robot_pos: Tuple[int, int],
    t: int,
    mode_idx: int,
    policies: List[SensorModePolicy],
) -> Tuple[int, int]:
    if 0 <= mode_idx < len(policies):
        aS = policies[mode_idx].action(world, sensor_pos, robot_pos, t)
        return world.step(sensor_pos, aS)
    return sensor_pos


def mcts_search_action(
    world: GridWorld,
    belief: BeliefState,
    robot_pos: Tuple[int, int],
    goal: Tuple[int, int],
    sensor_policies: List[SensorModePolicy],
    detection: DetectionModel,
    action_prior: Dict[Action, float],
    *,
    t: int,
    num_sims: int = 250,
    max_depth: int = 10,
    c_puct: float = 1.4,
    w_step: float = 1.0,
    w_detect: float = 7.0,
    gamma: float = 0.95,
    prior_eps: float = 0.10,
    log: Optional[Logger] = None,
) -> Action:
    prior = normalize_probs_dict(action_prior, ACTIONS, eps=prior_eps)

    root = MCTSNode(robot_pos=robot_pos, depth=0)
    root.P_a = prior

    # Prepare particle sampling distribution
    particles = belief.particles if belief.particles else [Particle(sensor_pos=robot_pos, mode_idx=0, weight=1.0)]
    weights = [float(getattr(p, "weight", 1.0)) for p in particles]
    s = sum(weights)
    if s <= 0.0:
        weights = [1.0 / len(weights)] * len(weights)
    else:
        weights = [w / s for w in weights]

    if log is not None and (log._should_print(t, log.cfg.mcts_every_t, lvl=1)):
        log.log(f"[MCTS] t={t} sims={num_sims} depth={max_depth} prior: {prior_diagnostics(prior)}", t=t, every=log.cfg.mcts_every_t, lvl=1)

    for sim in range(num_sims):
        hp = random.choices(particles, weights=weights, k=1)[0]
        sim_sensor = hp.sensor_pos
        sim_mode = hp.mode_idx

        node = root
        cur_robot = robot_pos
        cur_sensor = sim_sensor

        path: List[Tuple[MCTSNode, Action]] = []
        total_return = 0.0
        disc = 1.0

        if cur_robot == goal:
            root.N += 1
            continue

        for depth in range(max_depth):
            if cur_robot == goal:
                total_return += disc * 50.0
                break

            a = select_action_puct(node, c_puct=c_puct)
            path.append((node, a))

            nxt_robot = world.step(cur_robot, a)
            nxt_sensor = simulate_sensor_step(world, cur_sensor, nxt_robot, t + depth, sim_mode, sensor_policies)

            step_cost = w_step + w_detect * detection.p_detect(nxt_robot, nxt_sensor)
            total_return += disc * (-step_cost)
            disc *= gamma

            if a not in node.children:
                child = MCTSNode(robot_pos=nxt_robot, depth=node.depth + 1)
                child.P_a = prior
                node.children[a] = child

                leaf_v = rollout_value(
                    world,
                    nxt_robot,
                    nxt_sensor,
                    goal,
                    detection,
                    max_steps=max_depth - depth - 1,
                    w_step=w_step,
                    w_detect=w_detect,
                )
                total_return += disc * leaf_v
                break

            node = node.children[a]
            cur_robot, cur_sensor = nxt_robot, nxt_sensor

        if path:
            backprop_path(path, total_return)
        else:
            root.N += 1

        if log is not None and log.cfg.level >= 2 and (sim < 2 or ((sim + 1) % log.cfg.mcts_progress_every_sims == 0 and t <= log.cfg.detail_until_t)):
            log.log(f"[MCTS][detail] t={t} sim={sim+1}/{num_sims} root.N={root.N}", t=t, every=None, lvl=2)

    total_edge_visits = sum(root.N_a.values())
    if log is not None and log._should_print(t, log.cfg.mcts_every_t, lvl=1):
        bestN = max(root.N_a, key=lambda a: root.N_a[a])
        log.log(
            f"[MCTS] t={t} root.N={root.N} sum(N_a)={total_edge_visits} best_by_N={bestN}",
            t=t,
            every=log.cfg.mcts_every_t,
            lvl=1,
        )
        if log.cfg.level >= 2 and t <= log.cfg.detail_until_t:
            for a in ACTIONS:
                log.log(f"[MCTS][detail] a={a} N={root.N_a[a]:4d} Q={root.Q_a[a]:+8.3f} P={root.P_a[a]:.3f}", t=t, every=None, lvl=2)

    return choose_final_action(root)


# =========================
# Run-wide plots ("what happened in the whole game")
# =========================

# NOTE: Uses matplotlib, but you only need it for plotting.
# If you don't have it: pip install matplotlib
import os
from dataclasses import dataclass
from typing import Any

import matplotlib.pyplot as plt


@dataclass
class RunHistory:
    mode_names: List[str]
    t: List[int] = field(default_factory=list)

    # positions
    robot_r: List[int] = field(default_factory=list)
    robot_c: List[int] = field(default_factory=list)
    sensor_r: List[int] = field(default_factory=list)
    sensor_c: List[int] = field(default_factory=list)

    # observation
    obs_r: List[int] = field(default_factory=list)
    obs_c: List[int] = field(default_factory=list)

    # core signals
    dist_RS: List[int] = field(default_factory=list)
    p_detect: List[float] = field(default_factory=list)

    # event flags
    caught: List[int] = field(default_factory=list)

    # belief over modes (posterior mass each mode)
    mode_post: Dict[str, List[float]] = field(default_factory=dict)

    # action prior and chosen action
    prior_by_a: Dict[Action, List[float]] = field(default_factory=dict)
    chosen_action: List[Action] = field(default_factory=list)

    def __post_init__(self) -> None:
        self.mode_post = {m: [] for m in self.mode_names}
        self.prior_by_a = {a: [] for a in ACTIONS}

    def add(
        self,
        *,
        t: int,
        robot_pos: Tuple[int, int],
        sensor_pos: Tuple[int, int],
        goal: Tuple[int, int],
        obs: Tuple[int, int],
        mode_post: Dict[str, float],
        action_prior: Dict[Action, float],
        chosen_action: Action,
        p_detect: float,
        dist_RS: int,
        caught: bool,
    ) -> None:
        self.t.append(int(t))
        self.robot_r.append(int(robot_pos[0])); self.robot_c.append(int(robot_pos[1]))
        self.sensor_r.append(int(sensor_pos[0])); self.sensor_c.append(int(sensor_pos[1]))
        self.obs_r.append(int(obs[0])); self.obs_c.append(int(obs[1]))

        self.dist_RS.append(int(dist_RS))
        self.p_detect.append(float(p_detect))
        self.caught.append(1 if caught else 0)

        for m in self.mode_names:
            self.mode_post[m].append(float(mode_post.get(m, 0.0)))

        for a in ACTIONS:
            self.prior_by_a[a].append(float(action_prior.get(a, 0.0)))

        self.chosen_action.append(chosen_action)


def make_artifact_dir(seed: int) -> str:
    out_dir = os.path.join("artifacts", f"run_seed_{seed}")
    os.makedirs(out_dir, exist_ok=True)
    return out_dir


def _savefig(path: str) -> None:
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()


def save_whole_game_plots(out_dir: str, hist: RunHistory, detection: DetectionModel) -> None:
    if not hist.t:
        return

    # ------------------------------------------------------------------
    # Plot 1: "How dangerous was it over time?"  (p_detect + distance)
    # ------------------------------------------------------------------
    plt.figure(figsize=(10.5, 4.5))
    plt.plot(hist.t, hist.p_detect, label="p_detect(robot | sensor)", linewidth=2)
    plt.plot(hist.t, hist.dist_RS, label="Manhattan distance d(R,S)")

    # Mark caught timesteps (if any)
    for i, t in enumerate(hist.t):
        if hist.caught[i]:
            plt.axvline(t, linestyle="--", linewidth=1.5, label="caught timestep" if i == 0 else None)

    plt.axhline(y=detection.p_close, linestyle=":", linewidth=1, label="p_close")
    plt.axhline(y=detection.p_far, linestyle=":", linewidth=1, label="p_far")
    plt.xlabel("t")
    plt.ylabel("risk")
    plt.title("Risk timeline: detection probability and robot–sensor distance")
    plt.legend()
    _savefig(os.path.join(out_dir, "01_risk_timeline.png"))

    # ------------------------------------------------------------------
    # Plot 2: "What did we believe the sensor was doing?" (mode posterior)
    # stacked area = sums to 1, easy to see dominance shifts
    # ------------------------------------------------------------------
    plt.figure(figsize=(10.5, 4.5))
    ys = [hist.mode_post[m] for m in hist.mode_names]
    plt.stackplot(hist.t, *ys, labels=hist.mode_names)
    plt.xlabel("t")
    plt.ylabel("posterior mass")
    plt.ylim(0.0, 1.0)
    plt.title("Belief over sensor modes (particle filter posterior)")
    plt.legend(loc="upper right")
    _savefig(os.path.join(out_dir, "02_mode_posterior_stack.png"))

    # ------------------------------------------------------------------
    # Plot 3: "What was the planner leaning toward?" (action prior + chosen)
    # Show prior curves and put a dot on the action we actually took.
    # ------------------------------------------------------------------
    plt.figure(figsize=(10.5, 4.5))
    for a in ACTIONS:
        plt.plot(hist.t, hist.prior_by_a[a], label=f"prior({a})")

    # Overlay chosen actions as markers at their prior value
    for i, t in enumerate(hist.t):
        a = hist.chosen_action[i]
        y = hist.prior_by_a[a][i]
        plt.scatter([t], [y], marker="o", s=40, label="chosen action" if i == 0 else None)

    plt.xlabel("t")
    plt.ylabel("probability")
    plt.ylim(0.0, 1.0)
    plt.title("Robot action prior over time (and the action actually selected)")
    plt.legend(ncol=3)
    _savefig(os.path.join(out_dir, "03_action_prior_and_choice.png"))


# =========================
# Main Simulation
# =========================

@dataclass
class Config:
    seed: int = 8

    H: int = 12
    W: int = 18
    T: int = 40

    num_particles: int = 250

    # PF
    motion_noise: float = 0.15
    resample_threshold: float = 0.55

    # Mode library
    num_hypotheses: int = 25
    per_hypothesis_attempts: int = 2
    max_modes: int = 6
    w_detect_astar: float = 10.0
    rand_scale: float = 0.45

    # NEW: plausibility-vs-cost tradeoff
    plausibility_lambda: float = 3.0

    # NEW: prior temperature
    prior_temp: float = 25.0

    # Prior smoothing/diagnostics (helps catch "prior collapse" from the mode library)
    prior_eps_main: float = 0.15
    prior_warn_maxp: float = 0.97
    prior_warn_minH: float = 0.05

    # MCTS
    num_sims: int = 250
    max_depth: int = 10
    c_puct: float = 1.4
    w_step_mcts: float = 1.0
    w_detect_mcts: float = 7.0

    # NEW: force a specific true mode for reproducibility
    true_mode_name: Optional[str] = "chase_robot"  # set None for random

    # Debug
    debug: DebugConfig = field(default_factory=DebugConfig)


def make_demo_world(cfg: Config) -> Tuple[GridWorld, Tuple[int, int], Tuple[int, int], Tuple[int, int]]:
    obs = set()
    for c in range(3, 15):
        obs.add((5, c))
    for r in range(1, 9):
        obs.add((r, 9))
    obs.discard((5, 7))
    obs.discard((6, 9))

    world = GridWorld(H=cfg.H, W=cfg.W, obstacles=obs)
    robot_start = (10, 2)
    goal = (1, 16)
    sensor_start = (2, 2)
    return world, robot_start, sensor_start, goal


def main() -> None:
    cfg = Config()
    seed_all(cfg.seed)

    log = Logger(cfg.debug)

    world, robot_pos, sensor_pos, goal = make_demo_world(cfg)

    sensor_modes: List[SensorModePolicy] = [
        PatrolLoopPolicy(name="patrol_left", loop=[(2, 2), (2, 6), (4, 6), (4, 2)]),
        PatrolLoopPolicy(name="patrol_mid", loop=[(2, 10), (2, 13), (4, 13), (4, 10)]),
        ChasePolicy(name="chase_robot"),
        RandomWalkPolicy(name="random_walk"),
    ]
    mode_names = [m.name for m in sensor_modes]

    # Deterministic true mode option
    if cfg.true_mode_name is None:
        true_mode_idx = random.randrange(len(sensor_modes))
    else:
        try:
            true_mode_idx = mode_names.index(cfg.true_mode_name)
        except ValueError:
            raise ValueError(f"true_mode_name={cfg.true_mode_name!r} not in modes {mode_names}")

    detection = DetectionModel()
    obs_model = ObservationModel(sigma=1.3)

    sensor_init_candidates = [(2, 2), (2, 10), (3, 3), (3, 9), (1, 1)]
    mode_prior = [1.0] * len(sensor_modes)
    belief = initialize_belief(world, sensor_init_candidates, mode_prior, mode_names, cfg.num_particles)

    # ----------------------
    # NEW: Run-wide history
    # (these graphs tell you what is happening in the whole game)
    # ----------------------
    hist = RunHistory(mode_names=mode_names)

    log.log(f"[INIT] true_mode={mode_names[true_mode_idx]} robot={robot_pos} sensor={sensor_pos} goal={goal}", force=True)
    log.log(f"[INIT] belief modes: {topk_str(belief.mode_posterior_named(), k=4)}", force=True)

    # Save plots even if we terminate early
    try:
        for t in range(cfg.T):
            show_header = (t <= cfg.debug.detail_until_t) or (
                cfg.debug.enabled and cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0
            )
            if show_header:
                log.log("=" * 80, force=True)
                log.log(f"[TIME] t={t}", force=True)

            if cfg.debug.enabled and (
                t <= cfg.debug.detail_until_t or (cfg.debug.render_every_t > 0 and t % cfg.debug.render_every_t == 0)
            ):
                log.log("[WORLD]", t=t, every=cfg.debug.render_every_t, lvl=1, force=(t <= cfg.debug.detail_until_t))
                world.render(robot=robot_pos, sensor=sensor_pos, goal=goal)

            # ----------------------
            # 1) Noisy observation
            # ----------------------
            obs = obs_model.sample_observation(sensor_pos, world)
            if cfg.debug.enabled and (
                t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)
            ):
                log.log(f"[OBS] obs={obs} (true sensor={sensor_pos})", force=True)

            # ----------------------
            # 2) PF belief update
            # ----------------------
            _ = belief_predict_update(
                belief,
                world,
                robot_pos,
                sensor_modes,
                obs_model,
                obs,
                t,
                motion_noise=cfg.motion_noise,
                resample_threshold=cfg.resample_threshold,
                log=log,
            )

            # ----------------------
            # 3) Mode/path library -> action prior
            # ----------------------
            library, _ = build_multimodal_path_library(
                world,
                belief,
                robot_pos,
                goal,
                detection,
                num_hypotheses=cfg.num_hypotheses,
                per_hypothesis_attempts=cfg.per_hypothesis_attempts,
                w_detect=cfg.w_detect_astar,
                rand_scale=cfg.rand_scale,
                max_modes=cfg.max_modes,
                plausibility_lambda=cfg.plausibility_lambda,
                t=t,
                log=log,
            )

            raw_prior = build_action_prior_from_library(world, robot_pos, library, temp=cfg.prior_temp)
            action_prior = normalize_probs_dict(raw_prior, ACTIONS, eps=cfg.prior_eps_main)

            prior_check_and_log(
                world,
                robot_pos,
                library,
                raw_prior,
                action_prior,
                temp=cfg.prior_temp,
                eps=cfg.prior_eps_main,
                mode_names=belief.mode_names,
                warn_maxp=cfg.prior_warn_maxp,
                warn_minH=cfg.prior_warn_minH,
                t=t,
                log=log,
            )

            if cfg.debug.enabled and (
                t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)
            ):
                log.log(f"[PriorDiag] eps={cfg.prior_eps_main:.2f} {prior_diagnostics(action_prior)}", force=True)

            # ----------------------
            # 4) MCTS chooses robot action
            # ----------------------
            aR = mcts_search_action(
                world,
                belief,
                robot_pos,
                goal,
                sensor_modes,
                detection,
                action_prior,
                t=t,
                num_sims=cfg.num_sims,
                max_depth=cfg.max_depth,
                c_puct=cfg.c_puct,
                w_step=cfg.w_step_mcts,
                w_detect=cfg.w_detect_mcts,
                prior_eps=0.0,
                log=log,
            )

            if cfg.debug.enabled and (
                t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)
            ):
                log.log(f"[ACT] robot aR={aR}", force=True)

            # ----------------------
            # 5) Step the real world (robot then true sensor)
            # ----------------------
            new_robot = world.step(robot_pos, aR)
            aS_true = sensor_modes[true_mode_idx].action(world, sensor_pos, new_robot, t)

            if mode_names[true_mode_idx] == "chase_robot":
                chase_sanity_check(world, sensor_pos, new_robot, aS_true, log=log, t=t, tag="true")

            new_sensor = world.step(sensor_pos, aS_true)
            robot_pos, sensor_pos = new_robot, new_sensor

            # ----------------------
            # 6) Detection event
            # ----------------------
            dist = manhattan(robot_pos, sensor_pos)
            p_det = detection.p_detect(robot_pos, sensor_pos)
            detected = (random.random() < p_det)

            if cfg.debug.enabled and (
                t <= cfg.debug.detail_until_t or (cfg.debug.summary_every_t > 0 and t % cfg.debug.summary_every_t == 0)
            ):
                log.log(
                    f"[STEP] robot={robot_pos} sensor={sensor_pos} aS={aS_true} d={dist} p_det={p_det:.3f} detected={detected}",
                    force=True,
                )

            # ----------------------
            # NEW: Record everything we need for "whole game" plots
            # ----------------------
            hist.add(
                t=t,
                robot_pos=robot_pos,
                sensor_pos=sensor_pos,
                goal=goal,
                obs=obs,
                mode_post=belief.mode_posterior_named(),
                action_prior=action_prior,
                chosen_action=aR,
                p_detect=p_det,
                dist_RS=dist,
                caught=(detected and dist <= detection.d0),
            )

            # Terminate
            if robot_pos == goal:
                log.log("[TERMINAL] reached goal ✅", force=True)
                break
            if detected and dist <= detection.d0:
                log.log("[TERMINAL] caught ❌", force=True)
                break

    finally:
        # ----------------------
        # NEW: Save "whole game" graphs
        # ----------------------
        out_dir = make_artifact_dir(cfg.seed)
        save_whole_game_plots(out_dir, hist, detection)
        log.log(f"[PLOTS] saved to {out_dir}", force=True)

    log.log("[DONE]", force=True)


if __name__ == "__main__":
    main()


[INIT] true_mode=chase_robot robot=(10, 2) sensor=(2, 2) goal=(1, 16)
[INIT] belief modes: patrol_left:0.272 | patrol_mid:0.252 | random_walk:0.240 | chase_robot:0.236
[TIME] t=0
[WORLD]
..................
.........#......G.
..S......#........
.........#........
.........#........
...####.#######...
..................
.........#........
.........#........
..................
..R...............
..................
[OBS] obs=(2, 4) (true sensor=(2, 2))
[PF] t=0 obs=(2, 4) ESS 250.0->103.6 resample=True | modes: patrol_mid:0.416 | patrol_left:0.280 | random_walk:0.176
[ModesRescue][WARN] t=0 all kept paths start with R; attempting forced-first-action rescue
[ModesRescue] t=0 added=12 forced-a0 candidates (excluding R)
[ModesRescue] t=0 new kept distinct_a0=1(R)
[ModesRescue][FORCE] t=0 replaced a0=R score=53.62 with a0=S score=58.91 -> distinct_a0=2(R,S)
[Modes] t=0 candidates=62 clusters=6 kept=6 | top3: patrol_left:w=0.004,cost=35.2,score=51.7, patrol_mid:w=0.004,cost=36.2,score=52.7, pat